<a href="https://colab.research.google.com/github/hosseinmiripy/rezume/blob/main/11%D8%A7%D8%B3%D8%AA%D8%B1%D8%A7%D8%AA%DA%98%DB%8C_%D8%A8%D8%B1%D8%AA%D8%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy ccxt matplotlib mplfinance python-telegram-bot seaborn scikit-learn plotly ta requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 871.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 673.5/673.5 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.2/626.2 kB 21.1 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=6d691050f9a5cf2a23b96c24232653b2fa11f230f6c42e40e4177505b3caf969
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.15
    Uninstalling aiohttp-3.11.15:
      Successfully uninstalled aiohttp-3.11.15


In [ ]:
"""
ربات تحلیل و سیگنال‌دهی ارز دیجیتال
------------------------------------------------
این ربات داده‌های ارزهای دیجیتال را از صرافی‌های مختلف دریافت کرده و با استفاده از استراتژی‌های پیشرفته،
سیگنال‌های معاملاتی را به کانال تلگرام ارسال می‌کند.

ویژگی‌ها:
- تحلیل در تایم‌فریم‌های 15 دقیقه، 1 ساعت، 4 ساعت و روزانه
- استفاده از 12 استراتژی مختلف تحلیل فنی
- رسم نمودارهای با کیفیت بالا
- ارسال سیگنال‌ها به تلگرام همراه با نمودار
- مدیریت هوشمند اتصال به صرافی‌ها
"""

import tempfile
import shutil
import hashlib
import gc
from datetime import datetime, timedelta
import threading
import datetime
import json  # احتمالاً قبلاً در کد وارد شده
import os  # احتمالاً قبلاً در کد وارد شده
import time  # احتمالاً قبلاً در کد وارد شده
import math  # برای تابع ceil در تقسیم‌بندی گروه‌ها
import random
import pandas as pd
import numpy as np
import ccxt
import logging
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
from matplotlib.patches import Rectangle
import seaborn as sns
import telegram
from telegram import InputFile
from sklearn.preprocessing import MinMaxScaler
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from io import BytesIO
import ta
import warnings
import datetime as dt
from scipy.signal import argrelextrema
from collections import deque, defaultdict
import math
import re
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
from threading import Lock
import traceback
from sklearn.cluster import KMeans
import requests

# تنظیم محیط گوگل کولب - اختیاری
COLAB_ENV = False
try:
    import google.colab

    COLAB_ENV = True
    from google.colab import drive

    print("🌟 محیط گوگل کولب شناسایی شد - فعال‌سازی ذخیره‌سازی پایدار...")
    # نصب به گوگل درایو
    drive.mount('/content/drive')
    BASE_DIR = '/content/drive/MyDrive/crypto_analyzer'
    os.makedirs(f"{BASE_DIR}/cache/ohlcv", exist_ok=True)
    os.makedirs(f"{BASE_DIR}/charts", exist_ok=True)
    os.makedirs(f"{BASE_DIR}/logs", exist_ok=True)
except:
    print("🖥️ محیط محلی شناسایی شد - استفاده از ذخیره‌سازی محلی...")
    BASE_DIR = os.path.dirname(os.path.abspath(__file__))
    os.makedirs(f"{BASE_DIR}/cache/ohlcv", exist_ok=True)
    os.makedirs(f"{BASE_DIR}/charts", exist_ok=True)
    os.makedirs(f"{BASE_DIR}/logs", exist_ok=True)

# تنظیمات لاگ
logging.basicConfig(
    filename=f"{BASE_DIR}/logs/crypto_analyzer.log",
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# نمایش لاگ‌ها در کنسول هم
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(console_formatter)
logger.addHandler(console_handler)

# تنظیمات ربات تلگرام - توکن و آیدی چنل خود را اینجا قرار دهید
TELEGRAM_BOT_TOKEN = "7307185304:AAF9zG3vM0Pc74WkiIDVEnRxu41OJYdyql8"
TELEGRAM_CHANNEL_ID = "@hosseinbtb2"  # مثال: "@your_channel_name"

# مخفی کردن هشدارهای غیرضروری
warnings.filterwarnings("ignore")

# متغیرهای گلوبال
memory_cache = {}
memory_cache_access_time = {}
memory_cache_max_size = 500
request_count_lock = Lock()
last_analysis_times = {}
last_signal_times = {}

# تنظیمات صرافی‌ها - با مدیریت پیشرفته محدودیت‌ها
EXCHANGES_CONFIG = {
    'kucoin': {
        'class': ccxt.kucoin,
        'max_requests_per_minute': 15,
        'cooldown_time': 60,
        'weight': 1.0,
        'backoff_factor': 1.5,
        'max_cooldown': 300,
        'min_cooldown': 30,
    },
    'gate': {
        'class': ccxt.gate,
        'max_requests_per_minute': 20,
        'cooldown_time': 50,
        'weight': 0.9,
        'backoff_factor': 1.5,
        'max_cooldown': 300,
        'min_cooldown': 30,
    },
    'mexc': {
        'class': ccxt.mexc,
        'max_requests_per_minute': 30,
        'cooldown_time': 40,
        'weight': 1.2,
        'backoff_factor': 1.5,
        'max_cooldown': 300,
        'min_cooldown': 30,
    }
}

# تنظیم وضعیت اولیه صرافی‌ها
exchange_data = {}
for exchange_id in EXCHANGES_CONFIG:
    exchange_data[exchange_id] = {
        'request_count': 0,
        'last_reset_time': time.time(),
        'success_count': 0,
        'fail_count': 0,
        'in_cooldown': False,
        'cooldown_until': 0,
        'consecutive_failures': 0,
        'last_rate_limit': 0
    }

# تنظیمات بات
BOT_SETTINGS = {
    'memory_management': True,
    'max_memory_cache_size': 500,
    'min_time_between_signals': 1800,  # حداقل فاصله بین سیگنال‌های یک ارز (به ثانیه)
    'signal_confidence_threshold': 75,  # حداقل درصد اطمینان برای ارسال سیگنال
    'max_signals_per_day': 2000,  # حداکثر تعداد سیگنال در روز
    'signal_cooldown_hours': 1,  # زمان انتظار بین سیگنال‌های مشابه
    'disk_cache_dir': os.path.join(BASE_DIR, "temp_cache"),  # دایرکتوری کش دیسک
    'disk_cache_max_age': 24,  # حداکثر عمر فایل‌های کش به ساعت
}
os.makedirs(BOT_SETTINGS['disk_cache_dir'], exist_ok=True)

# تنظیمات ارزها و تایم‌فریم‌ها
symbols = [
    'BTC/USDT', 'ETH/USDT', 'XRP/USDT', 'SOL/USDT', 'ADA/USDT',
    'DOGE/USDT', 'SHIB/USDT', 'TRX/USDT', 'TON/USDT', 'DOT/USDT',
    'AVAX/USDT', 'LINK/USDT', 'PEPE/USDT', 'UNI/USDT', 'LTC/USDT',
    'ATOM/USDT', 'XLM/USDT', 'BCH/USDT', 'NEAR/USDT', 'ETC/USDT',
    'FIL/USDT', 'VET/USDT', 'HBAR/USDT', 'ALGO/USDT',
    'QNT/USDT', 'AAVE/USDT', 'AXS/USDT', 'SAND/USDT', 'MANA/USDT',
    'MEME/USDT', 'EOS/USDT', 'FLOW/USDT', 'RVN/USDT', 'CHZ/USDT',
    'OP/USDT', 'NEO/USDT', 'CRV/USDT', 'ROSE/USDT', 'ZEC/USDT',
    'ICP/USDT', 'ENJ/USDT', 'AR/USDT', 'ONE/USDT', 'BAT/USDT',
    'ZIL/USDT', 'IOTA/USDT', 'LRC/USDT', 'YFI/USDT', 'SNX/USDT',
    'COMP/USDT', 'DASH/USDT', 'MKR/USDT', 'RUNE/USDT', 'CAKE/USDT',
    'DYDX/USDT', 'OMG/USDT', 'SRM/USDT', 'ICX/USDT', 'ONT/USDT',
    'SXP/USDT', 'STORJ/USDT', 'CELO/USDT', 'QTUM/USDT',
    'SUSHI/USDT', 'LDO/USDT', 'ANKR/USDT', 'IOST/USDT'
]

# تایم‌فریم‌ها
timeframes = ['15m', '1h', '4h', '1d']


# تابع کمکی برای ایجاد کلید کش
def get_cache_key(function_name, *args):
    """ایجاد کلید برای کش حافظه"""
    return f"{function_name}_{':'.join(str(arg) for arg in args)}"


# تابع کمکی برای پاکسازی کش حافظه
# تابع بهبود یافته پاکسازی کش حافظه
def clean_memory_cache(max_items=200, force_cleanup=False):
    """
    پاکسازی هوشمند کش حافظه با حفظ آیتم‌های پرکاربرد و پشتیبان‌گیری به دیسک

    Args:
        max_items: حداکثر آیتم‌های مجاز در کش حافظه
        force_cleanup: اجبار به پاکسازی حتی اگر تعداد آیتم‌ها کمتر از حد مجاز باشد
    """
    global memory_cache, memory_cache_access_time

    # اگر پاکسازی اجباری نباشد و تعداد آیتم‌ها کمتر از حد مجاز باشد، کاری انجام نمی‌دهیم
    if not force_cleanup and len(memory_cache) <= max_items:
        return

    # ثبت میزان حافظه آزاد شده برای لاگ
    initial_size = len(memory_cache)

    try:
        # مرتب‌سازی آیتم‌ها بر اساس زمان آخرین دسترسی
        sorted_items = sorted(memory_cache_access_time.items(), key=lambda x: x[1])

        # تعیین تعداد آیتم‌هایی که باید حذف شوند
        if force_cleanup:
            items_to_remove = int(len(memory_cache) * 0.7)  # پاکسازی 70% آیتم‌ها در حالت اجباری
        else:
            items_to_remove = len(memory_cache) - max_items

        # ذخیره آیتم‌های مهم OHLCV در دیسک قبل از حذف
        saved_to_disk = 0
        for i in range(min(items_to_remove, len(sorted_items))):
            key = sorted_items[i][0]
            if key in memory_cache and "load_from_cache" in key:
                try:
                    save_to_disk_cache(key, memory_cache[key])
                    saved_to_disk += 1
                except Exception as e:
                    logger.error(f"خطا در ذخیره به دیسک: {e}")

        # حذف آیتم‌های قدیمی از حافظه
        removed_count = 0
        for i in range(min(items_to_remove, len(sorted_items))):
            key = sorted_items[i][0]
            if key in memory_cache:
                del memory_cache[key]
                removed_count += 1
            if key in memory_cache_access_time:
                del memory_cache_access_time[key]

        # فراخوانی صریح جمع‌آوری زباله
        gc.collect()

        # بررسی میزان حافظه آزاد شده
        freed_items = initial_size - len(memory_cache)
        logger.info(f"کش حافظه پاکسازی شد: {freed_items} آیتم حذف شد، {saved_to_disk} آیتم به دیسک منتقل شد")

    except Exception as e:
        logger.error(f"خطا در پاکسازی کش حافظه: {e}")
        # در صورت خطا، پاکسازی اضطراری انجام می‌دهیم
        try:
            # پاکسازی کامل کش در صورت خطا
            memory_cache.clear()
            memory_cache_access_time.clear()
            gc.collect()
            logger.warning("پاکسازی اضطراری کش حافظه انجام شد")
        except:
            pass


# تابع جدید برای پایش منابع سیستم با مدیریت پیشرفته حافظه
def monitor_resources(critical_threshold=90, warning_threshold=75):
    """
    پایش منابع سیستم و مدیریت خودکار حافظه در صورت نیاز

    Args:
        critical_threshold: آستانه بحرانی استفاده از حافظه (درصد)
        warning_threshold: آستانه هشدار استفاده از حافظه (درصد)

    Returns:
        وضعیت سیستم (dict)
    """
    try:
        import psutil

        # بررسی استفاده از حافظه
        memory = psutil.virtual_memory()
        memory_percent = memory.percent

        # بررسی استفاده از CPU
        cpu_percent = psutil.cpu_percent(interval=0.1)

        # ثبت وضعیت منابع
        system_status = {
            'memory_percent': memory_percent,
            'cpu_percent': cpu_percent,
            'cache_items': len(memory_cache),
            'memory_free_gb': memory.available / (1024 ** 3)
        }

        # اگر استفاده از حافظه بالاتر از آستانه بحرانی است، پاکسازی اضطراری
        if memory_percent >= critical_threshold:
            logger.warning(f"🚨 وضعیت بحرانی حافظه: {memory_percent:.1f}% - پاکسازی اضطراری")
            clean_memory_cache(max_items=100, force_cleanup=True)

            # اگر هنوز حافظه بالاست، پاکسازی کامل کش
            if psutil.virtual_memory().percent >= critical_threshold:
                memory_cache.clear()
                memory_cache_access_time.clear()
                gc.collect()
                logger.warning("پاکسازی کامل کش حافظه به دلیل وضعیت بحرانی")

        # اگر استفاده از حافظه بالاتر از آستانه هشدار است، پاکسازی استاندارد
        elif memory_percent >= warning_threshold:
            logger.warning(f"⚠️ مصرف حافظه بالاست: {memory_percent:.1f}% - پاکسازی استاندارد")
            clean_memory_cache(max_items=200)
            gc.collect()

        # اگر تعداد آیتم‌های کش بیش از حد مجاز است، پاکسازی می‌کنیم
        elif len(memory_cache) > BOT_SETTINGS['max_memory_cache_size']:
            logger.info(f"تعداد آیتم‌های کش ({len(memory_cache)}) بیش از حد مجاز است - پاکسازی")
            clean_memory_cache(max_items=int(BOT_SETTINGS['max_memory_cache_size'] * 0.8))

        return system_status

    except Exception as e:
        logger.error(f"خطا در پایش منابع سیستم: {e}")

        # در صورت خطا، پاکسازی استاندارد انجام می‌دهیم
        clean_memory_cache(force_cleanup=True)
        gc.collect()

        return {'error': str(e)}

def get_disk_cache_key(key):
    """تبدیل کلید کش به نام فایل مناسب برای ذخیره در دیسک"""
    # استفاده از md5 برای تبدیل کلید به رشته امن برای نام فایل
    return hashlib.md5(key.encode()).hexdigest() + ".pkl"


def save_to_disk_cache(key, data):
    """ذخیره داده در کش دیسک"""
    import pickle

    filename = os.path.join(BOT_SETTINGS['disk_cache_dir'], get_disk_cache_key(key))
    try:
        with open(filename, 'wb') as f:
            pickle.dump(data, f, protocol=4)  # پروتکل 4 برای کارایی بیشتر
        return True
    except Exception as e:
        logger.error(f"خطا در ذخیره کش دیسک {filename}: {e}")
        return False


def load_from_disk_cache(key):
    """بارگیری داده از کش دیسک"""
    import pickle

    filename = os.path.join(BOT_SETTINGS['disk_cache_dir'], get_disk_cache_key(key))
    if not os.path.exists(filename):
        return None

    try:
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        return data
    except Exception as e:
        logger.error(f"خطا در بارگیری کش دیسک {filename}: {e}")
        return None


def cleanup_old_disk_cache():
    """حذف فایل‌های کش قدیمی از دیسک"""
    now = time.time()
    max_age_seconds = BOT_SETTINGS['disk_cache_max_age'] * 3600

    try:
        cache_dir = BOT_SETTINGS['disk_cache_dir']
        count = 0
        for filename in os.listdir(cache_dir):
            filepath = os.path.join(cache_dir, filename)
            if os.path.isfile(filepath) and filename.endswith('.pkl'):
                file_age = now - os.path.getmtime(filepath)
                if file_age > max_age_seconds:
                    os.remove(filepath)
                    count += 1

        if count > 0:
            logger.info(f"پاکسازی کش دیسک: {count} فایل قدیمی حذف شد.")
    except Exception as e:
        logger.error(f"خطا در پاکسازی کش دیسک: {e}")

# کد توابع مدیریت اتصال به صرافی‌ها و دریافت داده
def get_best_exchange(symbol, data_type="ohlcv", depth=None):
    """انتخاب بهترین صرافی با مدیریت هوشمند و هماهنگ‌سازی نوع داده"""
    with request_count_lock:
        # بررسی و ریست شمارنده‌ها
        current_time = time.time()

        # اضافه کردن امتیاز جریمه برای صرافی‌هایی که اخیراً محدودیت نرخ داشته‌اند
        penalty_scores = {}
        for ex_id in EXCHANGES_CONFIG:
            # صرافی‌هایی که اخیراً محدودیت نرخ داشته‌اند جریمه بیشتری دریافت می‌کنند
            recent_rate_limit = False
            if current_time - exchange_data[ex_id].get('last_rate_limit', 0) < 300:
                recent_rate_limit = True
                penalty_scores[ex_id] = 0.5  # 50% جریمه
            else:
                penalty_scores[ex_id] = 0

        # منطق انتخاب صرافی
        available_exchanges = []
        for ex_id, ex_config in EXCHANGES_CONFIG.items():
            # بررسی وضعیت کولدان
            if exchange_data[ex_id]['in_cooldown']:
                if current_time > exchange_data[ex_id]['cooldown_until']:
                    # پایان کولدان
                    exchange_data[ex_id]['in_cooldown'] = False
                    exchange_data[ex_id]['consecutive_failures'] = 0
                    logger.info(f"صرافی {ex_id} از حالت کولدان خارج شد")
                else:
                    # هنوز در حالت کولدان است
                    continue

            # بررسی ریست شمارنده
            if current_time - exchange_data[ex_id]['last_reset_time'] > 60:  # یک دقیقه
                exchange_data[ex_id]['request_count'] = 0
                exchange_data[ex_id]['last_reset_time'] = current_time

            # بررسی محدودیت تعداد درخواست با مارجین امنیتی
            request_limit_factor = 0.85
            if data_type == "orderbook":
                # برای دفتر سفارش، محدودیت بیشتری اعمال می‌شود
                request_limit_factor = 0.7

            if exchange_data[ex_id]['request_count'] < (ex_config['max_requests_per_minute'] * request_limit_factor):
                # امتیاز صرافی بر اساس ترکیب وزن و نسبت موفقیت
                success_ratio = 1.0
                if exchange_data[ex_id]['success_count'] + exchange_data[ex_id]['fail_count'] > 0:
                    success_ratio = exchange_data[ex_id]['success_count'] / (
                            exchange_data[ex_id]['success_count'] + exchange_data[ex_id]['fail_count'])

                # امتیاز نهایی با کسر جریمه
                score = (ex_config['weight'] * (0.7 + 0.3 * success_ratio)) * (1 - penalty_scores[ex_id])

                available_exchanges.append((ex_id, score))

        if not available_exchanges:
            # هیچ صرافی در دسترس نیست - انتظار برای خروج از کولدان
            logger.warning("همه صرافی‌ها در حالت کولدان هستند. انتظار برای بازیابی...")
            # انتخاب صرافی با کمترین زمان کولدان باقیمانده
            min_cooldown_ex = min(EXCHANGES_CONFIG.keys(),
                                  key=lambda x: exchange_data[x]['cooldown_until'] if exchange_data[x][
                                      'in_cooldown'] else float('inf'))
            wait_time = max(1, exchange_data[min_cooldown_ex]['cooldown_until'] - current_time)
            time.sleep(wait_time)
            # بازگشت به صورت بازگشتی
            return get_best_exchange(symbol, data_type, depth)

        # انتخاب صرافی با بالاترین امتیاز
        best_exchange, _ = max(available_exchanges, key=lambda x: x[1])

        # افزایش شمارنده درخواست برای صرافی انتخاب شده
        exchange_data[best_exchange]['request_count'] += 1

        return best_exchange


def handle_rate_limit(exchange_id):
    """مدیریت وضعیت صرافی هنگام برخورد با محدودیت نرخ با تنظیم خودکار"""
    with request_count_lock:
        exchange_data[exchange_id]['in_cooldown'] = True
        cooldown_time = EXCHANGES_CONFIG[exchange_id]['cooldown_time']
        exchange_data[exchange_id]['cooldown_until'] = time.time() + cooldown_time
        # ثبت زمان آخرین محدودیت نرخ و افزایش شمارنده
        exchange_data[exchange_id]['last_rate_limit'] = time.time()
        exchange_data[exchange_id]['cooldown_count'] = exchange_data[exchange_id].get('cooldown_count', 0) + 1
        # افزایش خودکار زمان کولدان اگر صرافی به طور مکرر دچار محدودیت شود
        if exchange_data[exchange_id]['cooldown_count'] > 10:
            adjusted_cooldown = cooldown_time * 1.2  # افزایش 20%
            EXCHANGES_CONFIG[exchange_id]['cooldown_time'] = min(int(adjusted_cooldown), 120)  # حداکثر 2 دقیقه
            logger.warning(
                f"افزایش خودکار زمان کولدان برای {exchange_id} به {EXCHANGES_CONFIG[exchange_id]['cooldown_time']} ثانیه")

        # کاهش max_requests_per_minute اگر محدودیت نرخ بیش از حد تکرار شود
        if exchange_data[exchange_id]['cooldown_count'] > 15:
            current_max = EXCHANGES_CONFIG[exchange_id]['max_requests_per_minute']
            adjusted_max = max(int(current_max * 0.8), 10)  # کاهش 20% با حداقل 10
            EXCHANGES_CONFIG[exchange_id]['max_requests_per_minute'] = adjusted_max
            logger.warning(f"کاهش خودکار max_requests_per_minute برای {exchange_id} به {adjusted_max}")

        logger.warning(f"صرافی {exchange_id} به دلیل محدودیت نرخ در حالت کولدان به مدت {cooldown_time} ثانیه قرار گرفت")


def get_cache_filename(symbol, timeframe, exchange_id):
    """ایجاد نام فایل برای ذخیره کش"""
    safe_symbol = symbol.replace('/', '_')
    return os.path.join(BASE_DIR, f"cache/ohlcv/{exchange_id}_{safe_symbol}_{timeframe}.csv")


def load_from_cache(symbol, timeframe, exchange_id, max_age_hours=None, legacy=False):
    """
    بارگیری داده از کش با پشتیبانی از کش دیسک
    """
    # تنظیم عمر کش بر اساس تایم‌فریم اگر مقدار ورودی نباشد
    if max_age_hours is None:
        if timeframe == '15m':
            max_age_hours = 0.25  # 15 دقیقه
        elif timeframe == '1h':
            max_age_hours = 1  # 1 ساعت
        elif timeframe == '4h':
            max_age_hours = 2  # 2 ساعت
        else:
            max_age_hours = 4  # برای 1d و سایر تایم‌فریم‌ها

    # بررسی کش حافظه ابتدا
    cache_key = get_cache_key("load_from_cache", symbol, timeframe, exchange_id)

    if cache_key in memory_cache:
        # بررسی سن داده در کش حافظه
        age_in_hours = (time.time() - memory_cache_access_time[cache_key]) / 3600
        if age_in_hours <= max_age_hours:
            # بروزرسانی زمان دسترسی
            memory_cache_access_time[cache_key] = time.time()
            logger.info(f"داده از کش حافظه بارگیری شد: {symbol}/{timeframe} (سن: {age_in_hours:.2f} ساعت)")
            return memory_cache[cache_key]
        else:
            logger.info(f"داده کش حافظه قدیمی است: {symbol}/{timeframe} (سن: {age_in_hours:.2f} ساعت)")
            # حذف داده قدیمی از کش حافظه
            del memory_cache[cache_key]
            del memory_cache_access_time[cache_key]

    # بررسی کش دیسک
    disk_data = load_from_disk_cache(cache_key)
    if disk_data is not None:
        # ذخیره مجدد در کش حافظه
        memory_cache[cache_key] = disk_data
        memory_cache_access_time[cache_key] = time.time()
        logger.info(f"داده از کش دیسک بارگیری شد: {symbol}/{timeframe}")

        # مدیریت حافظه
        if BOT_SETTINGS['memory_management'] and len(memory_cache) > memory_cache_max_size:
            clean_memory_cache()

        return disk_data

    # اگر exchange_id برابر با "any" باشد، همه کش‌ها را بررسی می‌کنیم
    if exchange_id == "any":
        for ex_id in EXCHANGES_CONFIG:
            result = load_from_cache(symbol, timeframe, ex_id, max_age_hours)
            if result is not None:
                return result

        # اگر کش‌های معمولی یافت نشد و legacy فعال است، از کش قدیمی‌تر استفاده کنیم
        if legacy:
            for ex_id in EXCHANGES_CONFIG:
                filename = get_cache_filename(symbol, timeframe, ex_id)
                if os.path.exists(filename):
                    file_mod_time = os.path.getmtime(filename)
                    # برای کش قدیمی‌تر، آستانه‌های سختگیرانه‌تر برای داده‌های معتبر
                    if (time.time() - file_mod_time) < (max_age_hours * 2 * 3600):  # دو برابر max_age_hours
                        try:
                            df = pd.read_csv(filename)
                            df['timestamp'] = pd.to_datetime(df['timestamp'])
                            df.set_index('timestamp', inplace=True)
                            if len(df) >= 80:  # حداقل 80 کندل
                                logger.warning(f"استفاده از کش legacy برای {symbol}/{timeframe} از {ex_id}")

                                # ذخیره در کش حافظه
                                memory_cache[cache_key] = df
                                memory_cache_access_time[cache_key] = time.time()

                                return df
                        except Exception as e:
                            logger.warning(f"خطا در بارگیری کش legacy: {e}")

        return None

    filename = get_cache_filename(symbol, timeframe, exchange_id)
    if not os.path.exists(filename):
        return None

    # بررسی سن فایل
    file_mod_time = os.path.getmtime(filename)
    file_age_hours = (time.time() - file_mod_time) / 3600

    if file_age_hours > max_age_hours:
        logger.info(f"کش قدیمی است: {filename} (سن: {file_age_hours:.2f} ساعت)")
        return None

    try:
        # بررسی فایل فشرده
        if filename.endswith('.gz'):
            df = pd.read_csv(filename, compression='gzip')
        else:
            df = pd.read_csv(filename)

        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)
        logger.info(f"داده از کش بارگیری شد: {filename} (سن: {file_age_hours:.2f} ساعت)")

        # ذخیره در کش حافظه
        memory_cache[cache_key] = df
        memory_cache_access_time[cache_key] = time.time()

        # مدیریت حافظه
        if BOT_SETTINGS['memory_management'] and len(memory_cache) > memory_cache_max_size:
            clean_memory_cache()

        return df
    except Exception as e:
        logger.error(f"خطا در بارگیری کش {filename}: {e}")
        return None


def save_to_cache(df, symbol, timeframe, exchange_id):
    """ذخیره داده در کش با پشتیبانی از کش دیسک"""
    if df is None or len(df) == 0:
        return False

    # ذخیره در کش حافظه
    cache_key = get_cache_key("load_from_cache", symbol, timeframe, exchange_id)
    memory_cache[cache_key] = df
    memory_cache_access_time[cache_key] = time.time()

    # مدیریت حافظه - درصورت نیاز به کش دیسک منتقل می‌شود
    if BOT_SETTINGS['memory_management'] and len(memory_cache) > memory_cache_max_size:
        clean_memory_cache()

    filename = get_cache_filename(symbol, timeframe, exchange_id)
    try:
        # تبدیل شاخص datetime به ستون برای ذخیره
        df_to_save = df.reset_index()

        # ذخیره با فرمت فشرده برای دیتافریم‌های بزرگ
        if len(df_to_save) > 500:
            df_to_save.to_csv(filename + '.gz', index=False, compression='gzip')
            # اگر نسخه غیرفشرده قبلاً وجود داشته، آن را حذف کنیم
            if os.path.exists(filename):
                os.remove(filename)
            filename = filename + '.gz'
        else:
            df_to_save.to_csv(filename, index=False)

        logger.info(f"داده در کش ذخیره شد: {filename}")

        # ذخیره در کش دیسک برای بازیابی سریع‌تر
        save_to_disk_cache(cache_key, df)

        return True
    except Exception as e:
        logger.error(f"خطا در ذخیره کش {filename}: {e}")
        return False

def validate_data(df):
    """اعتبارسنجی بهبودیافته داده‌های دریافتی"""
    if df is None or len(df) < 20:
        return False

    try:
        # بررسی مقادیر گمشده
        null_count = df.isnull().sum().sum()
        if null_count > 0:
            if null_count < len(df) * 0.03:  # کمتر از 3% داده‌ها
                df.ffill(inplace=True)
                df.bfill(inplace=True)
                if df.isnull().sum().sum() > 0:
                    return False
            else:
                return False

        # بررسی تغییرات قیمت غیرعادی
        price_changes = df['close'].pct_change().abs()
        if price_changes.max() > 0.5:  # تغییر بیش از 50%
            extreme_changes = price_changes[price_changes > 0.25]
            if len(extreme_changes) > len(df) * 0.05:
                return False

        # بررسی حجم صفر
        zero_volume_ratio = (df['volume'] == 0).sum() / len(df)
        if zero_volume_ratio > 0.15:
            return False

        # بررسی تکرار مقادیر یکسان (داده‌های تکراری)
        repeated_close = (df['close'] == df['close'].shift(1)).mean()
        if repeated_close > 0.5:
            return False

        # بررسی فاصله‌های زمانی کندل‌ها
        if df.index.to_series().diff().dt.total_seconds().std() > 60:
            return False

        return True
    except Exception as e:
        logger.error(f"خطا در اعتبارسنجی داده: {e}")
        return False


def analyze_exchange_performance():
    """بررسی عملکرد صرافی‌ها برای بهینه‌سازی خودکار"""
    performance_report = {}

    for ex_id in EXCHANGES_CONFIG:
        total_requests = exchange_data[ex_id]['success_count'] + exchange_data[ex_id]['fail_count']
        success_rate = exchange_data[ex_id]['success_count'] / total_requests if total_requests > 0 else 0
        cooldown_count = exchange_data[ex_id].get('cooldown_count', 0)

        performance_report[ex_id] = {
            'success_rate': success_rate * 100,
            'total_requests': total_requests,
            'cooldown_count': cooldown_count,
            'recommendation': 'Good' if success_rate > 0.8 and cooldown_count < 5 else 'Needs Adjustment'
        }

    print("\n=== گزارش عملکرد صرافی‌ها ===")
    for ex_id, stats in performance_report.items():
        print(
            f"{ex_id}: نرخ موفقیت {stats['success_rate']:.1f}%, تعداد کل درخواست: {stats['total_requests']}, دفعات محدودیت: {stats['cooldown_count']}")
        if stats['recommendation'] == 'Needs Adjustment':
            print(f"  ⚠️ توصیه: کاهش max_requests_per_minute یا افزایش cooldown_time")

    return performance_report


# کش برای افزایش کارایی تابع fetch_data
fetch_data_cache = {}


def fetch_data(symbol, timeframe, limit=1000, max_retries=5, delay=2, force_fresh=False, max_age_hours=1,
               incremental=False):
    """
    دریافت داده با استفاده از سیستم پیشرفته با مدیریت خطا کامل و استراتژی بهینه،
    همراه با قابلیت به‌روزرسانی تدریجی داده‌ها و مدیریت بهینه حافظه دیسک-محور

    Args:
        symbol: نماد ارز
        timeframe: تایم‌فریم
        limit: تعداد کندل‌های درخواستی
        max_retries: حداکثر تعداد تلاش مجدد
        delay: تاخیر بین تلاش‌ها به ثانیه
        force_fresh: اجبار به دریافت داده‌های تازه و عدم استفاده از کش
        max_age_hours: حداکثر سن مجاز برای داده‌های کش به ساعت
        incremental: فعال‌سازی دریافت تدریجی داده‌ها (فقط داده‌های جدید)

    Returns:
        دیتافریم داده‌های بارگیری شده یا None در صورت خطا
    """
    # کلید کش برای این درخواست
    cache_key = get_cache_key("fetch_data", symbol, timeframe, limit)

    try:
        # بررسی کش حافظه در صورت عدم اجبار به داده‌های تازه
        if not force_fresh and cache_key in memory_cache:
            cache_age_seconds = time.time() - memory_cache_access_time[cache_key]
            # بررسی سن کش (اگر کمتر از 5 دقیقه باشد استفاده می‌کنیم)
            if cache_age_seconds < 300:  # 5 دقیقه
                # بروزرسانی زمان دسترسی بدون ایجاد داده جدید
                memory_cache_access_time[cache_key] = time.time()
                return memory_cache[cache_key]
            logger.info(f"کش حافظه قدیمی است: {symbol}/{timeframe} (سن: {cache_age_seconds / 60:.1f} دقیقه)")

        # بررسی کش دیسک اگر در حافظه نباشد
        if not force_fresh:
            disk_data = load_from_disk_cache(cache_key)
            if disk_data is not None:
                # ذخیره مجدد در کش حافظه
                memory_cache[cache_key] = disk_data
                memory_cache_access_time[cache_key] = time.time()
                logger.info(f"داده از کش دیسک بارگیری شد: {symbol}/{timeframe}")

                # مدیریت حافظه
                if BOT_SETTINGS['memory_management'] and len(memory_cache) > BOT_SETTINGS['max_memory_cache_size']:
                    clean_memory_cache()

                return disk_data

        # لیست صرافی‌هایی که تا به حال برای این جفت ارز/تایم‌فریم موفق بوده‌اند
        successful_exchanges_key = f"{symbol}_{timeframe}_successful"
        successful_exchanges = fetch_data_cache.get(successful_exchanges_key, [])

        # لاگینگ کار با صرافی‌های موفق
        if successful_exchanges:
            logger.info(f"صرافی‌های موفق قبلی برای {symbol}/{timeframe}: {successful_exchanges}")

        attempts_failed = []

        # اگر اجبار به داده‌های تازه نیست و حالت تدریجی فعال است، از ترکیب داده‌های کش با داده‌های جدید استفاده می‌کنیم
        if incremental and not force_fresh:
            df_cached = None
            latest_timestamp = None

            # بارگذاری داده‌های موجود در کش
            for ex_id in successful_exchanges:
                df_cached = load_from_cache(symbol, timeframe, ex_id, max_age_hours)
                if df_cached is not None and len(df_cached) >= 50:
                    if validate_data(df_cached):
                        # آخرین تاریخ موجود در کش را ذخیره می‌کنیم
                        latest_timestamp = df_cached.index[-1]
                        logger.info(
                            f"داده‌های کش معتبر برای {symbol}/{timeframe} یافت شد. آخرین تاریخ: {latest_timestamp}")
                        break

            if df_cached is not None and latest_timestamp is not None:
                logger.info(f"دریافت تدریجی داده‌های جدید برای {symbol}/{timeframe} از تاریخ {latest_timestamp}")

                # تبدیل تاریخ آخرین کندل به میلی‌ثانیه برای استفاده در API
                since_timestamp = int(latest_timestamp.timestamp() * 1000)

                # همیشه حداقل چند کندل آخر را مجدداً دریافت کن برای اطمینان از صحت داده‌ها
                # تعیین تعداد کندل‌های مورد نیاز برای دریافت (با مارجین امنیتی)
                fetch_limit = min(20, limit)  # دریافت حداکثر 20 کندل آخر

                # دریافت داده‌های جدید
                df_new = fetch_recent_data(symbol, timeframe, since_timestamp, fetch_limit)

                if df_new is not None and len(df_new) > 0:
                    # ادغام داده‌های جدید با داده‌های قدیمی
                    df_merged = merge_dataframes(df_cached, df_new)

                    if df_merged is not None and validate_data(df_merged):
                        # ذخیره داده‌های ترکیبی در کش
                        save_to_cache(df_merged, symbol, timeframe,
                                      successful_exchanges[0] if successful_exchanges else "kucoin")

                        # ذخیره در کش حافظه
                        memory_cache[cache_key] = df_merged
                        memory_cache_access_time[cache_key] = time.time()

                        logger.info(f"به‌روزرسانی تدریجی موفق برای {symbol}/{timeframe}: {len(df_new)} کندل جدید")

                        # آزادسازی داده‌های موقت
                        del df_new
                        gc.collect()

                        return df_merged

        # اگر به‌روزرسانی تدریجی موفق نبود یا غیرفعال بود، از روش معمولی استفاده می‌کنیم
        # 1. ابتدا از کش می‌خوانیم (اگر اجبار به داده‌های تازه نباشد)
        if not force_fresh:
            for exchange_id in EXCHANGES_CONFIG:
                cached_df = load_from_cache(symbol, timeframe, exchange_id, max_age_hours=max_age_hours)
                if cached_df is not None and len(cached_df) >= 80:
                    # بررسی زمان آخرین کندل
                    latest_candle_time = cached_df.index[-1]
                    current_time = pd.Timestamp.now(latest_candle_time.tz)
                    time_diff = (current_time - latest_candle_time).total_seconds() / 60  # تفاوت به دقیقه

                    # اگر آخرین کندل قدیمی است، کش را نادیده بگیر
                    if time_diff > timeframe_to_minutes(timeframe) * 2:
                        logger.warning(f"آخرین کندل در کش {time_diff:.1f} دقیقه قدیمی است - نیاز به به‌روزرسانی")
                        cached_df = None
                    # اعتبارسنجی داده‌های کش
                    if validate_data(cached_df):
                        # ذخیره در کش حافظه
                        memory_cache[cache_key] = cached_df
                        memory_cache_access_time[cache_key] = time.time()

                        # آزادسازی حافظه
                        gc.collect()
                        return cached_df
                    else:
                        logger.warning(f"داده‌های کش برای {symbol}/{timeframe} از {exchange_id} نامعتبر است")
                        # آزادسازی کش نامعتبر
                        del cached_df

        # لاگ مربوط به داده‌های تازه
        if force_fresh:
            logger.info(f"اجبار به دریافت داده‌های تازه برای {symbol}/{timeframe}")

        # 2. انتخاب بهترین صرافی برای استفاده
        best_exchange = get_best_exchange(symbol)
        logger.info(f"تلاش برای دریافت داده از صرافی برتر: {best_exchange}")

        # 3. تلاش برای دریافت داده از صرافی برتر
        df = fetch_from_single_exchange(symbol, timeframe, best_exchange, limit, max_retries, delay)
        if df is not None and len(df) >= 80:
            if validate_data(df):  # اعتبارسنجی داده
                # بروزرسانی لیست صرافی‌های موفق
                successful_exchanges = [best_exchange] + [ex for ex in successful_exchanges if ex != best_exchange]
                successful_exchanges = successful_exchanges[:3]  # حداکثر 3 صرافی ذخیره شود
                fetch_data_cache[successful_exchanges_key] = successful_exchanges

                save_to_cache(df, symbol, timeframe, best_exchange)
                # ذخیره در کش دیسک برای استفاده بعدی
                save_to_disk_cache(cache_key, df)

                gc.collect()
                return df
            else:
                logger.warning(f"داده‌های دریافتی از {best_exchange} برای {symbol}/{timeframe} نامعتبر است")
                attempts_failed.append(best_exchange)
                # آزادسازی داده نامعتبر
                del df

        # 4. اگر صرافی برتر موفق نبود، سایر صرافی‌ها را امتحان می‌کنیم
        logger.warning(f"صرافی برتر {best_exchange} برای {symbol}/{timeframe} موفق نبود. امتحان سایر صرافی‌ها")

        # لیست همه صرافی‌ها به جز best_exchange
        other_exchanges = [ex for ex in EXCHANGES_CONFIG.keys() if ex != best_exchange]
        random.shuffle(other_exchanges)  # ترتیب تصادفی

        for exchange_id in other_exchanges:
            # افزایش تاخیر برای جلوگیری از rate limit
            current_delay = delay + 1.0
            logger.info(f"تلاش با صرافی جایگزین: {exchange_id}")

            df = fetch_from_single_exchange(symbol, timeframe, exchange_id, limit, max_retries, current_delay)
            if df is not None and len(df) >= 80:
                if validate_data(df):  # اعتبارسنجی داده
                    # بروزرسانی لیست صرافی‌های موفق
                    successful_exchanges = [exchange_id] + [ex for ex in successful_exchanges if ex != exchange_id]
                    successful_exchanges = successful_exchanges[:3]  # حداکثر 3 صرافی ذخیره شود
                    fetch_data_cache[successful_exchanges_key] = successful_exchanges

                    save_to_cache(df, symbol, timeframe, exchange_id)
                    # ذخیره در کش دیسک برای استفاده بعدی
                    save_to_disk_cache(cache_key, df)

                    gc.collect()
                    return df
                else:
                    logger.warning(f"داده‌های دریافتی از {exchange_id} برای {symbol}/{timeframe} نامعتبر است")
                    # آزادسازی داده نامعتبر
                    del df

            attempts_failed.append(exchange_id)

        # 5. در صورت عدم موفقیت همه صرافی‌ها، از کش قدیمی استفاده می‌کنیم (فقط اگر force_fresh نباشد)
        if not force_fresh:
            logger.error(f"❌ همه صرافی‌ها برای {symbol}/{timeframe} شکست خوردند. تلاش برای استفاده از کش قدیمی")

            # استفاده از کش قدیمی‌تر (تا 6 ساعت)
            cached_df = load_from_cache(symbol, timeframe, "any", max_age_hours=6)
            if cached_df is not None and len(cached_df) >= 80:
                logger.warning(f"استفاده از کش قدیمی (تا 6 ساعت) برای {symbol}/{timeframe}")

                gc.collect()
                return cached_df

            # تلاش برای استفاده از کش legacy (تا 12 ساعت)
            cached_df = load_from_cache(symbol, timeframe, "any", max_age_hours=12, legacy=True)
            if cached_df is not None:
                gc.collect()
                return cached_df
        else:
            logger.error(f"❌ همه صرافی‌ها برای {symbol}/{timeframe} شکست خوردند و اجبار به داده‌های تازه فعال است")

        # اگر هیچ داده‌ای پیدا نشد، None برمی‌گردانیم
        logger.error(f"❌ دریافت داده برای {symbol}/{timeframe} کاملاً ناموفق بود")

        # آزادسازی منابع حافظه
        gc.collect()
        return None

    except Exception as e:
        # حتی اگر خطایی در کل فرآیند رخ داد، برنامه نباید متوقف شود
        logger.error(f"🚨 خطای سیستمی در دریافت داده برای {symbol}/{timeframe}: {str(e)}")

        # آزادسازی صریح حافظه در صورت خطا
        gc.collect()

        return None

def timeframe_to_minutes(timeframe):
    """تبدیل تایم‌فریم به دقیقه"""
    try:
        unit = timeframe[-1]
        value = int(timeframe[:-1])

        if unit == 'm':
            return value
        elif unit == 'h':
            return value * 60
        elif unit == 'd':
            return value * 24 * 60
        elif unit == 'w':
            return value * 7 * 24 * 60
        else:
            return 60  # پیش‌فرض: 1 ساعت
    except:
        return 60  # پیش‌فرض: 1 ساعت

def get_triangulated_price(symbol):
    """دریافت قیمت از چندین صرافی و محاسبه میانگین وزنی برای افزایش دقت"""
    prices = []
    weights = []

    # دریافت قیمت از صرافی‌های مختلف
    for exchange_id in ['kucoin', 'gate', 'mexc']:
        try:
            exchange_class = EXCHANGES_CONFIG[exchange_id]['class']
            exchange = exchange_class()
            ticker = exchange.fetch_ticker(symbol)

            if ticker and 'last' in ticker and ticker['last']:
                # افزودن قیمت و وزن آن (بر اساس حجم معاملات)
                prices.append(ticker['last'])
                # وزن بیشتر به صرافی‌های با حجم بالاتر
                weights.append(ticker.get('quoteVolume', 1000000) / 1000000)
        except:
            continue

    if not prices:
        return None

    # میانگین وزنی قیمت‌ها
    if sum(weights) > 0:
        triangulated_price = sum(p * w for p, w in zip(prices, weights)) / sum(weights)
    else:
        # میانگین ساده در صورت نبود اطلاعات حجم
        triangulated_price = sum(prices) / len(prices)

    return triangulated_price


def fetch_recent_data(symbol, timeframe, since_timestamp, limit=100, max_retries=5):
    """
    دریافت داده‌های اخیر از یک تاریخ مشخص با سیستم تأیید چندگانه قیمت و تصحیح خودکار

    Args:
        symbol: نماد ارز
        timeframe: تایم‌فریم
        since_timestamp: تاریخ شروع به میلی‌ثانیه
        limit: تعداد کندل‌های درخواستی
        max_retries: حداکثر تعداد تلاش مجدد

    Returns:
        دیتافریم داده‌های اخیر یا None در صورت خطا
    """
    # لیست صرافی‌هایی که تا به حال برای این جفت ارز/تایم‌فریم موفق بوده‌اند
    successful_exchanges_key = f"{symbol}_{timeframe}_successful"
    successful_exchanges = fetch_data_cache.get(successful_exchanges_key, [])

    # ابتدا یک قیمت مرجع از چندین صرافی دریافت کنیم
    reference_price = get_market_reference_price(symbol)

    # ترجیح دادن صرافی‌های موفق قبلی
    if successful_exchanges:
        exchange_list = successful_exchanges + [ex for ex in EXCHANGES_CONFIG.keys() if ex not in successful_exchanges]
    else:
        exchange_list = list(EXCHANGES_CONFIG.keys())
        random.shuffle(exchange_list)  # ترتیب تصادفی

    all_dataframes = []  # برای ذخیره نتایج از صرافی‌های مختلف

    for exchange_id in exchange_list:
        exchange_class = EXCHANGES_CONFIG[exchange_id]['class']

        try:
            # بررسی کولدان صرافی
            with request_count_lock:
                if exchange_data[exchange_id]['in_cooldown']:
                    current_time = time.time()
                    if current_time <= exchange_data[exchange_id]['cooldown_until']:
                        logger.info(f"صرافی {exchange_id} در حالت کولدان است. تلاش با صرافی بعدی...")
                        continue
                    else:
                        # پایان کولدان
                        exchange_data[exchange_id]['in_cooldown'] = False
                        exchange_data[exchange_id]['consecutive_failures'] = 0
                        logger.info(f"صرافی {exchange_id} از حالت کولدان خارج شد")

            exchange = exchange_class()

            for attempt in range(max_retries):
                try:
                    # یک تاخیر کوچک برای جلوگیری از درخواست‌های همزمان
                    if attempt > 0:
                        time.sleep(1 + attempt)

                    # کاهش زمان since برای دریافت مطمئن‌تر داده‌ها
                    # از ۲ کندل قبل‌تر شروع می‌کنیم تا کندل‌های قبلی هم اگر تغییر کرده‌اند، دریافت شوند
                    tf_seconds = timeframe_to_seconds(timeframe)
                    adjusted_since = since_timestamp - (tf_seconds * 2 * 1000)  # تبدیل به میلی‌ثانیه

                    # دریافت داده با استفاده از پارامتر since
                    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=adjusted_since, limit=limit)

                    if not ohlcv or len(ohlcv) == 0:
                        logger.warning(
                            f"داده‌ای از {exchange_id} برای {symbol}/{timeframe} از {pd.to_datetime(since_timestamp, unit='ms')} دریافت نشد")
                        break

                    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                    df.set_index('timestamp', inplace=True)

                    # بررسی آخرین قیمت با قیمت مرجع (اگر قیمت مرجع موجود باشد)
                    if reference_price and len(df) > 0:
                        last_price = df['close'].iloc[-1]
                        price_difference = abs(last_price / reference_price - 1)

                        if price_difference > 0.05:  # اختلاف بیش از 5%
                            logger.warning(
                                f"⚠️ اختلاف قیمت زیاد در {exchange_id} برای {symbol}: {price_difference * 100:.2f}% (مرجع: {reference_price}, دریافتی: {last_price})")
                            # داده‌ها را ذخیره اما علامت‌گذاری می‌کنیم
                            df.attrs['price_deviation'] = price_difference
                        elif price_difference > 0.01:  # اختلاف بین 1% تا 5%
                            logger.info(
                                f"اختلاف قیمت قابل توجه در {exchange_id} برای {symbol}: {price_difference * 100:.2f}%")
                            df.attrs['price_deviation'] = price_difference
                        else:
                            # قیمت معتبر است
                            df.attrs['price_deviation'] = 0

                    # محاسبه ستون‌های پایه
                    df['price_change'] = df['close'].pct_change() * 100
                    df['volume_change'] = df['volume'].pct_change() * 100
                    df['norm_volume'] = df['volume'] / df['volume'].rolling(window=20).mean()
                    df['volatility'] = (df['high'] - df['low']) / df['low'] * 100
                    df['exchange_source'] = exchange_id  # ذخیره منبع داده

                    # بروزرسانی آمار موفقیت
                    with request_count_lock:
                        exchange_data[exchange_id]['success_count'] += 1
                        exchange_data[exchange_id]['consecutive_failures'] = 0

                    logger.info(
                        f"داده‌های اخیر با موفقیت از {exchange_id} برای {symbol}/{timeframe} دریافت شد: {len(df)} کندل")

                    # ذخیره دیتافریم برای ترکیب بعدی
                    all_dataframes.append(df)

                    # اگر این صرافی اختلاف قیمت کمی دارد، به عنوان اولین نتیجه برگردانیم
                    if hasattr(df, 'attrs') and df.attrs.get('price_deviation', 1) < 0.01:
                        return df

                    # ادامه با صرافی بعدی برای مقایسه
                    break

                except ccxt.RateLimitExceeded:
                    logger.warning(f"⚠️ محدودیت نرخ در {exchange_id} برای دریافت داده‌های اخیر {symbol}/{timeframe}")
                    handle_rate_limit(exchange_id)
                    break

                except Exception as e:
                    logger.warning(
                        f"⚠️ خطا در تلاش {attempt + 1}/{max_retries} برای دریافت داده‌های اخیر از {exchange_id}: {str(e)}")

            # اگر به اینجا رسیده‌ایم، صرافی فعلی موفق نبوده است یا نتیجه‌اش مناسب نبوده
            with request_count_lock:
                if len(all_dataframes) == 0:  # فقط اگر هیچ داده‌ای دریافت نکرده باشیم
                    exchange_data[exchange_id]['fail_count'] += 1

        except Exception as e:
            logger.error(f"🚨 خطای اتصال به {exchange_id} برای دریافت داده‌های اخیر: {str(e)}")
            with request_count_lock:
                exchange_data[exchange_id]['fail_count'] += 1

    # اگر هیچ داده‌ای دریافت نکردیم
    if len(all_dataframes) == 0:
        logger.error(f"❌ دریافت داده‌های اخیر برای {symbol}/{timeframe} از تمام صرافی‌ها ناموفق بود")
        return None

    # اگر بیش از یک دیتافریم دریافت کردیم، بهترین را انتخاب یا ترکیب کنیم
    if len(all_dataframes) > 1:
        return merge_and_correct_data(all_dataframes, reference_price, symbol, timeframe)

    # اگر تنها یک دیتافریم داریم، همان را برگردانیم
    return all_dataframes[0]


def get_market_reference_price(symbol, timeout=3):
    """
    دریافت قیمت مرجع از چندین منبع برای مقایسه

    Args:
        symbol: نماد ارز
        timeout: زمان انتظار برای دریافت (ثانیه)

    Returns:
        قیمت مرجع یا None در صورت عدم دریافت
    """
    prices = []
    weights = []

    # دریافت قیمت از سرویس‌های مختلف
    # 1. استفاده از CCXT - از منابع موفق متعدد
    for exchange_id in ['binance', 'kucoin', 'gate', 'mexc']:
        try:
            exchange_class = getattr(ccxt, exchange_id)
            exchange = exchange_class()
            ticker = exchange.fetch_ticker(symbol)
            if ticker and 'last' in ticker and ticker['last']:
                prices.append(ticker['last'])
                # وزن براساس اعتبار صرافی
                if exchange_id == 'binance':
                    weights.append(10)  # وزن بیشتر برای بایننس به عنوان صرافی مرجع
                else:
                    weights.append(5)
        except:
            pass

    # 2. استفاده از CoinGecko API (اختیاری)
    try:
        coin_id = symbol.split('/')[0].lower()
        response = requests.get(f"https://api.coingecko.com/api/v3/simple/price?ids={coin_id}&vs_currencies=usd",
                                timeout=timeout)
        if response.status_code == 200:
            data = response.json()
            if coin_id in data and 'usd' in data[coin_id]:
                prices.append(data[coin_id]['usd'])
                weights.append(3)  # وزن کمتر برای CoinGecko
    except:
        pass

    # اگر هیچ قیمتی دریافت نکردیم
    if not prices:
        return None

    # محاسبه میانگین وزنی قیمت‌ها
    if sum(weights) > 0:
        reference_price = sum(p * w for p, w in zip(prices, weights)) / sum(weights)
    else:
        # میانگین ساده
        reference_price = sum(prices) / len(prices)

    return reference_price


def merge_and_correct_data(dataframes, reference_price, symbol, timeframe):
    """
    ترکیب و تصحیح داده‌های دریافت شده از منابع مختلف

    Args:
        dataframes: لیست دیتافریم‌های دریافتی
        reference_price: قیمت مرجع برای تصحیح
        symbol: نماد ارز
        timeframe: تایم‌فریم

    Returns:
        دیتافریم ترکیب شده و تصحیح شده
    """
    if len(dataframes) == 0:
        return None

    # مرتب‌سازی براساس اختلاف قیمت (از کمترین به بیشترین)
    sorted_dfs = sorted(dataframes, key=lambda df: df.attrs.get('price_deviation', 1) if hasattr(df, 'attrs') else 1)

    # استفاده از دیتافریم با کمترین اختلاف به عنوان پایه
    base_df = sorted_dfs[0].copy()

    # اگر فقط یک دیتافریم داریم یا قیمت مرجع نداریم
    if len(sorted_dfs) == 1 or not reference_price:
        logger.info(f"استفاده از دیتافریم {base_df['exchange_source'].iloc[0]} برای {symbol}/{timeframe}")
        return base_df

    # ترکیب داده‌ها از منابع مختلف با وزن‌دهی براساس اختلاف قیمت
    for i in range(1, len(sorted_dfs)):
        df = sorted_dfs[i]
        # فقط برای کندل‌های مشترک
        common_indices = base_df.index.intersection(df.index)

        if len(common_indices) > 0:
            # وزن براساس انحراف قیمت - صرافی‌های با انحراف کمتر وزن بیشتری دارند
            weight_base = 1 / (base_df.attrs.get('price_deviation', 0.01) + 0.01)
            weight_df = 1 / (df.attrs.get('price_deviation', 0.01) + 0.01)

            for col in ['open', 'high', 'low', 'close']:
                # میانگین وزنی برای هر ستون
                weights_sum = weight_base + weight_df
                base_df.loc[common_indices, col] = (
                                                           (base_df.loc[common_indices, col] * weight_base) +
                                                           (df.loc[common_indices, col] * weight_df)
                                                   ) / weights_sum

    # تصحیح نهایی قیمت با استفاده از قیمت مرجع (اگر اختلاف زیاد باشد)
    if reference_price:
        last_close = base_df['close'].iloc[-1]
        diff_pct = abs(last_close / reference_price - 1)

        if diff_pct > 0.02:  # اختلاف بیش از 2%
            logger.warning(f"تصحیح نهایی قیمت {symbol} با استفاده از قیمت مرجع. اختلاف: {diff_pct * 100:.2f}%")

            # ضریب تصحیح
            correction_factor = reference_price / last_close

            # اعمال ضریب فقط به آخرین کندل
            base_df.loc[base_df.index[-1], 'open'] *= correction_factor
            base_df.loc[base_df.index[-1], 'high'] *= correction_factor
            base_df.loc[base_df.index[-1], 'low'] *= correction_factor
            base_df.loc[base_df.index[-1], 'close'] *= correction_factor

            # بازمحاسبه ستون‌های مشتق شده
            base_df['price_change'] = base_df['close'].pct_change() * 100
            base_df['volatility'] = (base_df['high'] - base_df['low']) / base_df['low'] * 100

    logger.info(f"ترکیب موفق داده‌ها از {len(sorted_dfs)} منبع برای {symbol}/{timeframe}")
    return base_df


def get_fallback_price(symbol):
    """دریافت قیمت از منابع جایگزین در صورت خطا در منابع اصلی"""
    try:
        # استفاده از API عمومی برای دریافت قیمت
        response = requests.get(
            f"https://api.coingecko.com/api/v3/simple/price?ids={symbol.split('/')[0].lower()}&vs_currencies=usd",
            timeout=5)
        if response.status_code == 200:
            data = response.json()
            if symbol.split('/')[0].lower() in data:
                return data[symbol.split('/')[0].lower()]['usd']
    except:
        pass

    # تلاش برای استفاده از Binance به عنوان منبع مرجع
    try:
        binance = ccxt.binance()
        ticker = binance.fetch_ticker(symbol)
        if ticker and 'last' in ticker:
            return ticker['last']
    except:
        pass

    return None

def merge_dataframes(df_old, df_new):
    """
    ادغام دو دیتافریم با مدیریت همپوشانی

    Args:
        df_old: دیتافریم قدیمی
        df_new: دیتافریم جدید با داده‌های تازه

    Returns:
        دیتافریم ترکیب شده
    """
    try:
        if df_old is None or df_new is None:
            return df_old if df_old is not None else df_new

        # اطمینان از اینکه هر دو دیتافریم timestamp را به عنوان شاخص دارند
        if 'timestamp' in df_old.columns:
            df_old = df_old.set_index('timestamp')

        if 'timestamp' in df_new.columns:
            df_new = df_new.set_index('timestamp')

        # ترکیب دو دیتافریم با حذف همپوشانی
        df_combined = pd.concat([df_old, df_new])

        # حذف ردیف‌های تکراری بر اساس شاخص (timestamp)
        df_combined = df_combined[~df_combined.index.duplicated(keep='last')]

        # مرتب‌سازی بر اساس تاریخ
        df_combined = df_combined.sort_index()

        # پر کردن مقادیر NaN احتمالی
        if df_combined.isnull().sum().sum() > 0:
            df_combined = df_combined.ffill().bfill()

        return df_combined

    except Exception as e:
        logger.error(f"خطا در ادغام دیتافریم‌ها: {e}")
        # در صورت خطا، دیتافریم قدیمی را برگردانیم
        return df_old


def correct_price_data(df, symbol):
    """تصحیح داده‌های قیمت با استفاده از منابع مختلف"""
    if df is None or len(df) == 0:
        return df

    try:
        # دریافت قیمت فعلی بازار
        market_price = get_triangulated_price(symbol)
        if not market_price:
            market_price = get_fallback_price(symbol)

        if market_price:
            last_price = df['close'].iloc[-1]
            # محاسبه درصد اختلاف
            diff_pct = abs(last_price / market_price - 1) * 100

            if diff_pct > 5:
                # اختلاف خیلی زیاد - استفاده کامل از قیمت بازار
                logger.warning(f"اختلاف قیمت بسیار زیاد برای {symbol}: {diff_pct:.2f}% - تصحیح کامل")
                df['close'].iloc[-1] = market_price
                df['open'].iloc[-1] = (df['open'].iloc[-1] * market_price / last_price)
                df['high'].iloc[-1] = max(df['high'].iloc[-1] * market_price / last_price, market_price)
                df['low'].iloc[-1] = min(df['low'].iloc[-1] * market_price / last_price, market_price)
            elif diff_pct > 1:
                # اختلاف متوسط - ترکیب قیمت داده و بازار
                logger.info(f"تصحیح جزئی قیمت {symbol}: {diff_pct:.2f}%")
                weight_market = min(diff_pct / 5, 0.8)  # حداکثر 80% وزن به قیمت بازار
                weight_data = 1 - weight_market
                df['close'].iloc[-1] = (market_price * weight_market) + (last_price * weight_data)
    except Exception as e:
        logger.error(f"خطا در تصحیح قیمت {symbol}: {e}")

    return df

def timeframe_to_seconds(timeframe):
    """
    تبدیل تایم‌فریم به ثانیه

    Args:
        timeframe: تایم‌فریم مانند "15m", "1h", "4h", "1d"

    Returns:
        مقدار معادل به ثانیه
    """
    try:
        unit = timeframe[-1]
        value = int(timeframe[:-1])

        if unit == 'm':
            return value * 60
        elif unit == 'h':
            return value * 60 * 60
        elif unit == 'd':
            return value * 24 * 60 * 60
        elif unit == 'w':
            return value * 7 * 24 * 60 * 60
        else:
            return 3600  # پیش‌فرض: 1 ساعت
    except:
        return 3600  # پیش‌فرض: 1 ساعت


def track_last_analysis_time(symbol, timeframe, timestamp=None):
    """
    ثبت زمان آخرین تحلیل برای هر نماد و تایم‌فریم

    Args:
        symbol: نماد ارز
        timeframe: تایم‌فریم
        timestamp: زمان تحلیل (None برای استفاده از زمان فعلی)

    Returns:
        زمان ثبت شده
    """
    global last_analysis_times

    key = f"{symbol}_{timeframe}"

    # اگر timestamp داده نشده، از زمان فعلی استفاده می‌کنیم
    current_time = timestamp if timestamp is not None else time.time()

    # ثبت زمان
    last_analysis_times[key] = current_time

    return current_time


def get_last_analysis_time(symbol, timeframe):
    """
    دریافت زمان آخرین تحلیل برای یک نماد و تایم‌فریم خاص

    Args:
        symbol: نماد ارز
        timeframe: تایم‌فریم

    Returns:
        زمان آخرین تحلیل یا None اگر تحلیلی انجام نشده است
    """
    global last_analysis_times

    key = f"{symbol}_{timeframe}"

    return last_analysis_times.get(key, None)


def force_latest_data(symbol, timeframe, limit=100):
    """
    دریافت اجباری آخرین داده‌ها از بازار بدون استفاده از کش

    Args:
        symbol: نماد ارز
        timeframe: تایم‌فریم
        limit: تعداد کندل‌های درخواستی

    Returns:
        دیتافریم داده‌های جدید یا None در صورت خطا
    """
    logger.info(f"🔄 دریافت اجباری داده‌های جدید برای {symbol}/{timeframe}")

    # استفاده از بایننس به عنوان منبع اصلی اگر در دسترس باشد
    primary_exchanges = ['binance', 'kucoin', 'gate', 'mexc', 'bitfinex', 'huobi']
    all_results = []

    for exchange_id in primary_exchanges:
        try:
            # ایجاد نمونه صرافی
            if exchange_id not in EXCHANGES_CONFIG and hasattr(ccxt, exchange_id):
                exchange = getattr(ccxt, exchange_id)()
            else:
                exchange_class = EXCHANGES_CONFIG.get(exchange_id, {}).get('class')
                if not exchange_class:
                    continue
                exchange = exchange_class()

            # تنظیم زمان انقضای درخواست
            exchange.options['timeout'] = 10000  # 10 ثانیه

            # دریافت اطلاعات بازار برای اطمینان از پشتیبانی نماد
            if symbol not in exchange.load_markets():
                continue

            # دریافت زمان سرور برای تطبیق زمان
            server_time = exchange.fetch_time() if hasattr(exchange, 'fetch_time') else int(time.time() * 1000)

            # محاسبه زمان شروع براساس تایم‌فریم
            seconds_per_candle = timeframe_to_seconds(timeframe)
            start_timestamp = server_time - (seconds_per_candle * limit * 1000)

            # دریافت کندل‌ها
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=start_timestamp, limit=limit)

            if not ohlcv or len(ohlcv) < limit * 0.8:  # اگر کمتر از 80% داده‌های درخواستی برگردانده شد
                logger.warning(f"داده‌های ناکافی از {exchange_id} برای {symbol}/{timeframe}")
                continue

            # تبدیل به دیتافریم
            df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

            # بررسی تازگی داده‌ها
            current_time = pd.Timestamp.now()
            latest_candle_time = df['timestamp'].max()
            time_diff = (current_time - latest_candle_time).total_seconds() / 60  # تفاوت به دقیقه

            # افزودن اطلاعات کیفیت به دیتافریم
            df['time_lag'] = time_diff
            df['data_source'] = exchange_id

            logger.info(
                f"داده‌ها از {exchange_id} دریافت شد - آخرین کندل: {latest_candle_time}, تاخیر: {time_diff:.1f} دقیقه")

            # ذخیره نتیجه
            all_results.append((df, time_diff))

            # اگر تاخیر کمتر از مدت یک کندل است، نتیجه را مستقیماً برگردان
            if time_diff < seconds_per_candle / 60:
                df.set_index('timestamp', inplace=True)
                return df

        except Exception as e:
            logger.warning(f"خطا در دریافت از {exchange_id}: {str(e)}")
            continue

    # اگر نتیجه‌ای نداریم
    if not all_results:
        logger.error(f"❌ دریافت داده‌های جدید برای {symbol}/{timeframe} ناموفق بود")
        return None

    # انتخاب دیتافریم با کمترین تاخیر
    all_results.sort(key=lambda x: x[1])  # مرتب‌سازی براساس تاخیر
    best_df = all_results[0][0]
    best_df.set_index('timestamp', inplace=True)

    # افزودن ستون‌های مورد نیاز
    best_df['price_change'] = best_df['close'].pct_change() * 100
    best_df['volume_change'] = best_df['volume'].pct_change() * 100
    best_df['norm_volume'] = best_df['volume'] / best_df['volume'].rolling(window=20).mean()
    best_df['volatility'] = (best_df['high'] - best_df['low']) / best_df['low'] * 100

    logger.info(
        f"بهترین داده از {best_df['data_source'].iloc[0]} انتخاب شد - تاخیر: {best_df['time_lag'].iloc[0]:.1f} دقیقه")
    return best_df

def fetch_from_single_exchange(symbol, timeframe, exchange_id, limit=1000, max_retries=3, delay=2):
    """دریافت داده از یک صرافی مشخص با مدیریت خطاها"""
    exchange_class = EXCHANGES_CONFIG[exchange_id]['class']

    try:
        exchange = exchange_class()

        for attempt in range(max_retries):
            try:
                # افزودن تاخیر قبل از هر تلاش (به جز اولین تلاش)
                if attempt > 0:
                    sleep_time = delay * (attempt + 1) + random.uniform(0, 2)
                    time.sleep(sleep_time)

                # تنظیم پارامترهای محدودیت تعداد کندل‌ها
                if timeframe in ['15m']:
                    # برای تایم‌فریم‌های کوتاه، داده‌های بیشتری نیاز داریم
                    actual_limit = min(1500, limit)
                else:
                    actual_limit = limit

                ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=actual_limit)

                if not ohlcv or len(ohlcv) < 80:
                    logger.warning(
                        f"داده ناکافی از {exchange_id} برای {symbol}/{timeframe}: {len(ohlcv) if ohlcv else 0} کندل")
                    continue

                df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                df.set_index('timestamp', inplace=True)

                # محاسبه ستون‌های پایه
                df['price_change'] = df['close'].pct_change() * 100
                df['volume_change'] = df['volume'].pct_change() * 100
                df['norm_volume'] = df['volume'] / df['volume'].rolling(window=20).mean()
                df['volatility'] = (df['high'] - df['low']) / df['low'] * 100

                # بروزرسانی آمار موفقیت
                with request_count_lock:
                    exchange_data[exchange_id]['success_count'] += 1
                    # ریست خطاهای متوالی
                    exchange_data[exchange_id]['consecutive_failures'] = 0

                logger.info(f"داده با موفقیت از {exchange_id} برای {symbol}/{timeframe} دریافت شد: {len(df)} کندل")
                return df

            except ccxt.RateLimitExceeded:
                logger.warning(f"⚠️ محدودیت نرخ در {exchange_id} برای {symbol}/{timeframe}")
                handle_rate_limit(exchange_id)

            except ccxt.RequestTimeout:
                logger.warning(
                    f"⚠️ خطای زمان پاسخگویی از {exchange_id} برای {symbol}/{timeframe}. تلاش {attempt + 1}/{max_retries}")

            except ccxt.ExchangeNotAvailable:
                logger.warning(f"⚠️ صرافی {exchange_id} در دسترس نیست. تلاش {attempt + 1}/{max_retries}")

            except ccxt.DDoSProtection:
                logger.warning(f"⚠️ حفاظت DDoS در {exchange_id}. افزایش تاخیر...")
                time.sleep(delay * 10)  # تاخیر بیشتر

            except Exception as e:
                logger.error(f"🚨 خطا از {exchange_id} برای {symbol}/{timeframe}: {str(e)}")
                break

        # بروزرسانی آمار شکست
        with request_count_lock:
            exchange_data[exchange_id]['fail_count'] += 1

        return None

    except Exception as e:
        logger.error(f"🚨 خطای اتصال به {exchange_id}: {str(e)}")
        with request_count_lock:
            exchange_data[exchange_id]['fail_count'] += 1
        return None


def fetch_order_book(symbol, depth=20):
    """دریافت دفتر سفارش با پشتیبانی از چندین صرافی و مدیریت خطا"""
    # کلید کش
    cache_key = get_cache_key("fetch_order_book", symbol, depth)

    # بررسی کش حافظه (با زمان انقضای کوتاه‌تر برای دفتر سفارش)
    if cache_key in memory_cache and time.time() - memory_cache_access_time[cache_key] < 30:  # 30 ثانیه
        return memory_cache[cache_key]

    # لیست صرافی‌هایی که تا به حال برای این جفت ارز/تایم‌فریم موفق بوده‌اند
    successful_exchanges_key = f"{symbol}_orderbook_successful"
    successful_exchanges = fetch_data_cache.get(successful_exchanges_key, [])

    # ابتدا صرافی‌های موفق قبلی را امتحان می‌کنیم
    if successful_exchanges:
        for ex_id in successful_exchanges:
            try:
                exchange_class = EXCHANGES_CONFIG[ex_id]['class']
                if not exchange_class:
                    continue

                exchange = exchange_class()
                order_book = exchange.fetch_order_book(symbol, limit=depth)

                if order_book and 'bids' in order_book and 'asks' in order_book:
                    if len(order_book['bids']) > 0 and len(order_book['asks']) > 0:
                        with request_count_lock:
                            exchange_data[ex_id]['success_count'] += 1
                        logger.info(f"دفتر سفارش با موفقیت از {ex_id} برای {symbol} دریافت شد")

                        # ذخیره در کش حافظه
                        memory_cache[cache_key] = order_book
                        memory_cache_access_time[cache_key] = time.time()

                        return order_book
            except:
                pass

    # اگر صرافی‌های موفق قبلی کار نکردند، بهترین صرافی را امتحان می‌کنیم
    best_exchange = get_best_exchange(symbol, "orderbook", depth)
    exchanges_to_try = [best_exchange] + [ex for ex in EXCHANGES_CONFIG.keys() if ex != best_exchange]

    for exchange_id in exchanges_to_try:
        try:
            exchange_class = EXCHANGES_CONFIG[exchange_id]['class']
            if not exchange_class:
                continue

            exchange = exchange_class()
            order_book = exchange.fetch_order_book(symbol, limit=depth)

            if order_book and 'bids' in order_book and 'asks' in order_book:
                if len(order_book['bids']) > 0 and len(order_book['asks']) > 0:
                    with request_count_lock:
                        exchange_data[exchange_id]['success_count'] += 1

                    # بروزرسانی لیست صرافی‌های موفق
                    successful_exchanges = [exchange_id] + [ex for ex in successful_exchanges if ex != exchange_id]
                    successful_exchanges = successful_exchanges[:3]  # حداکثر 3 صرافی ذخیره شود
                    fetch_data_cache[successful_exchanges_key] = successful_exchanges

                    logger.info(f"دفتر سفارش با موفقیت از {exchange_id} برای {symbol} دریافت شد")

                    # ذخیره در کش حافظه
                    memory_cache[cache_key] = order_book
                    memory_cache_access_time[cache_key] = time.time()

                    return order_book

            # تاخیر بین درخواست‌ها
            time.sleep(1 + random.uniform(0, 1))

        except ccxt.RateLimitExceeded:
            handle_rate_limit(exchange_id)

        except Exception as e:
            logger.warning(f"خطا در دریافت دفتر سفارش از {exchange_id} برای {symbol}: {str(e)}")
            with request_count_lock:
                exchange_data[exchange_id]['fail_count'] += 1

    logger.error(f"❌ دریافت دفتر سفارش برای {symbol} از تمام صرافی‌ها ناموفق بود")
    return None


# کلاس اصلی تحلیلگر ارز دیجیتال
class CryptoAnalyzer:
    def __init__(self, telegram_bot_token=None, telegram_channel_id=None):
        self.telegram_bot_token = telegram_bot_token
        self.telegram_channel_id = telegram_channel_id
        self.telegram_bot = None
        self.telegram_connected = False  # اضافه شده
        self.signal_count = 0
        self.state_file = f"{BASE_DIR}/bot_state.json"
        self.last_save_time = time.time()
        self.save_interval = 300  # ۵ دقیقه
        self.signal_history = {}
        self.strategy_confidence = {
            'market_structure': 0.92,  # استراتژی ساختار بازار
            'order_block': 0.90,  # استراتژی بلوک سفارش
            'buying_selling_pressure': 0.88,  # استراتژی فشار خرید/فروش
            'whale_activity': 0.87,  # استراتژی فعالیت نهنگ‌ها
            'ema_strategy': 0.85,  # استراتژی میانگین متحرک نمایی
            'rsi_strategy': 0.84,  # استراتژی شاخص قدرت نسبی
            'divergence': 0.92,  # استراتژی واگرایی
            'bbrsi_macd': 0.86,  # استراتژی ترکیبی باندهای بولینگر، RSI و MACD
            'darvas_box': 0.80,  # استراتژی جعبه دارواس
            'ichimoku': 0.88,  # استراتژی ایچیموکو
            'fibonacci': 0.85,  # استراتژی فیبوناچی
            'market_structure_analysis': 0.93,  # استراتژی تحلیل ساختار بازار
            'golden_strategy': 0.95,  # استراتژی طلایی ترکیبی
            'special_strategy': 0.89,  # استراتژی ویژه
        }

        # تنظیم شیوه نمایش برای نمودارها
        plt.style.use('dark_background')
        sns.set_style("darkgrid")

        # تنظیم اتصال به ربات تلگرام
        self.setup_telegram()

        # شمارنده سیگنال‌ها
        self.daily_signal_count = 0
        self.last_signal_reset = time.time()

        logger.info("✅ سیستم تحلیلگر ارز دیجیتال با موفقیت راه‌اندازی شد")

    def setup_telegram(self):
        """تنظیم اتصال به تلگرام با روش HTTP مستقیم بجای کتابخانه"""
        if self.telegram_bot_token and self.telegram_channel_id:
            try:
                # ارسال پیام تست برای بررسی اتصال
                welcome_message = "🤖 ربات تحلیلگر ارز دیجیتال فعال شد\nدر حال بررسی بازار و ارسال سیگنال‌های معاملاتی..."

                url = f'https://api.telegram.org/bot{self.telegram_bot_token}/sendMessage'
                payload = {
                    'chat_id': self.telegram_channel_id,
                    'text': welcome_message,
                    'parse_mode': 'HTML'
                }

                response = requests.post(url, data=payload, timeout=10)

                if response.status_code == 200:
                    logger.info("✅ اتصال به تلگرام با موفقیت برقرار شد و پیام تست ارسال شد")
                    self.telegram_connected = True
                else:
                    logger.error(f"❌ خطا در اتصال به تلگرام: {response.status_code} - {response.text}")
                    self.telegram_connected = False

            except Exception as e:
                logger.error(f"❌ خطا در اتصال به تلگرام: {e}")
                logger.error(traceback.format_exc())
                self.telegram_connected = False
        else:
            logger.warning("⚠️ مقادیر توکن و آیدی کانال تلگرام تنظیم نشده است")
            self.telegram_connected = False

    def send_telegram_message(self, message, image_path=None):
        """ارسال پیام به کانال تلگرام با تصویر اختیاری - با استفاده از روش HTTP مستقیم"""
        if not self.telegram_connected:
            logger.warning("⚠️ اتصال تلگرام برقرار نیست")
            return False

        try:
            # تنظیم حداکثر طول مجاز پیام
            MAX_MESSAGE_LENGTH = 4000
            RETRY_COUNT = 3
            RETRY_DELAY = 2

            # اگر تصویر داریم، از متد ارسال عکس استفاده می‌کنیم
            if image_path:
                if not os.path.exists(image_path):
                    logger.error(f"❌ فایل تصویر یافت نشد: {image_path}")
                    return False

                # بررسی حجم تصویر و کاهش آن در صورت نیاز
                file_size = os.path.getsize(image_path) / (1024 * 1024)  # به مگابایت
                if file_size > 5:
                    logger.warning(f"⚠️ حجم تصویر زیاد است ({file_size:.1f} MB). در حال کاهش کیفیت...")
                    # کاهش کیفیت تصویر با PIL
                    try:
                        from PIL import Image
                        img = Image.open(image_path)

                        # محاسبه نسبت کاهش سایز
                        quality = min(95, int(5 * 100 / file_size))

                        # ایجاد نام فایل جدید
                        new_path = image_path.replace('.png', '_compressed.jpg')
                        img.save(new_path, 'JPEG', quality=quality)
                        image_path = new_path
                        logger.info(f"✅ حجم تصویر کاهش یافت. فایل جدید: {image_path}")
                    except Exception as img_err:
                        logger.error(f"❌ خطا در کاهش حجم تصویر: {img_err}")

                # تقسیم پیام اگر طولانی است
                if len(message) > MAX_MESSAGE_LENGTH:
                    # ابتدا تصویر را با بخش اول پیام ارسال می‌کنیم
                    first_part = message[:MAX_MESSAGE_LENGTH] + "\n...(ادامه دارد)"

                    # ارسال تصویر با بخش اول پیام
                    url = f'https://api.telegram.org/bot{self.telegram_bot_token}/sendPhoto'

                    for attempt in range(RETRY_COUNT):
                        try:
                            with open(image_path, 'rb') as photo:
                                files = {'photo': photo}
                                data = {
                                    'chat_id': self.telegram_channel_id,
                                    'caption': first_part,
                                    'parse_mode': 'HTML'
                                }
                                response = requests.post(url, data=data, files=files, timeout=30)

                                if response.status_code != 200:
                                    logger.error(f"❌ خطا در ارسال تصویر: {response.status_code} - {response.text}")
                                    time.sleep(RETRY_DELAY)
                                    continue
                                else:
                                    # بخش‌های باقیمانده پیام را در قالب متن ارسال می‌کنیم
                                    remaining_text = message[MAX_MESSAGE_LENGTH:]

                                    # اگر متن باقیمانده داریم، آن را ارسال می‌کنیم
                                    if remaining_text:
                                        text_url = f'https://api.telegram.org/bot{self.telegram_bot_token}/sendMessage'
                                        text_data = {
                                            'chat_id': self.telegram_channel_id,
                                            'text': "(ادامه) " + remaining_text,
                                            'parse_mode': 'HTML'
                                        }
                                        text_response = requests.post(text_url, data=text_data, timeout=10)

                                        if text_response.status_code != 200:
                                            logger.error(f"❌ خطا در ارسال ادامه پیام: {text_response.status_code}")
                                            return False

                                    logger.info(f"✅ پیام با تصویر با موفقیت به تلگرام ارسال شد")
                                    return True

                        except Exception as e:
                            logger.error(f"❌ خطا در ارسال تصویر (تلاش {attempt + 1}): {e}")
                            time.sleep(RETRY_DELAY)

                    return False
                else:
                    # پیام کوتاه است، مستقیماً با تصویر ارسال می‌کنیم
                    url = f'https://api.telegram.org/bot{self.telegram_bot_token}/sendPhoto'

                    for attempt in range(RETRY_COUNT):
                        try:
                            with open(image_path, 'rb') as photo:
                                files = {'photo': photo}
                                data = {
                                    'chat_id': self.telegram_channel_id,
                                    'caption': message,
                                    'parse_mode': 'HTML'
                                }
                                response = requests.post(url, data=data, files=files, timeout=30)

                                if response.status_code != 200:
                                    logger.error(f"❌ خطا در ارسال تصویر: {response.status_code} - {response.text}")
                                    if "Bad Request: can't parse entities" in response.text:
                                        logger.error("مشکل در تگ‌های HTML پیام. ارسال بدون parse_mode...")
                                        data['parse_mode'] = None
                                        response = requests.post(url, data=data, files=files, timeout=30)
                                        if response.status_code == 200:
                                            logger.info(f"✅ پیام با تصویر بدون parse_mode با موفقیت ارسال شد")
                                            return True
                                    time.sleep(RETRY_DELAY)
                                    continue
                                else:
                                    logger.info(f"✅ پیام با تصویر با موفقیت به تلگرام ارسال شد")
                                    return True

                        except Exception as e:
                            logger.error(f"❌ خطا در ارسال تصویر (تلاش {attempt + 1}): {e}")
                            time.sleep(RETRY_DELAY)

                    return False

            else:
                # ارسال پیام متنی بدون تصویر
                url = f'https://api.telegram.org/bot{self.telegram_bot_token}/sendMessage'

                # تقسیم پیام‌های طولانی
                if len(message) <= MAX_MESSAGE_LENGTH:
                    message_parts = [message]
                else:
                    message_parts = []
                    # تقسیم پیام با دقت در محل های مناسب
                    while len(message) > MAX_MESSAGE_LENGTH:
                        # یافتن محل مناسب برای تقسیم (پایان خط)
                        split_index = message[:MAX_MESSAGE_LENGTH].rfind('\n')
                        if split_index == -1:  # اگر پایان خط پیدا نشد
                            split_index = MAX_MESSAGE_LENGTH

                        message_parts.append(message[:split_index])
                        message = message[split_index:].strip()

                    if message:  # اضافه کردن بخش باقیمانده
                        message_parts.append(message)

                # ارسال هر بخش از پیام
                success = True
                for i, msg_part in enumerate(message_parts):
                    clean_message = msg_part
                    if i < len(message_parts) - 1:
                        clean_message += "\n(ادامه دارد...)"
                    elif i > 0:
                        clean_message = "(ادامه) " + clean_message

                    # ارسال پیام با تلاش مجدد در صورت خطا
                    sent = False
                    for attempt in range(RETRY_COUNT):
                        try:
                            payload = {
                                'chat_id': self.telegram_channel_id,
                                'text': clean_message,
                                'parse_mode': 'HTML'
                            }
                            response = requests.post(url, data=payload, timeout=10)

                            if response.status_code != 200:
                                # بررسی خطای تگ HTML
                                if "Bad Request: can't parse entities" in response.text:
                                    logger.error("مشکل در تگ‌های HTML پیام. ارسال بدون parse_mode...")
                                    payload['parse_mode'] = None
                                    response = requests.post(url, data=payload, timeout=10)
                                    if response.status_code == 200:
                                        logger.info(
                                            f"✅ بخش {i + 1}/{len(message_parts)} پیام بدون parse_mode با موفقیت ارسال شد")
                                        sent = True
                                        break

                                error_msg = f"خطای API تلگرام ({response.status_code}): {response.text}"
                                logger.error(error_msg)
                                time.sleep(RETRY_DELAY)
                            else:
                                logger.info(f"✅ بخش {i + 1}/{len(message_parts)} پیام با موفقیت ارسال شد")
                                sent = True
                                break
                        except Exception as e:
                            logger.error(f"❌ خطا در ارسال پیام (تلاش {attempt + 1}): {e}")
                            time.sleep(RETRY_DELAY)

                    if not sent:
                        success = False

                return success

        except Exception as e:
            logger.error(f"❌ خطای کلی در ارسال پیام به تلگرام: {e}")
            logger.error(traceback.format_exc())
            return False

    def send_signal(self, symbol, timeframe, signal_type, entry_price, stop_loss, take_profit,
                    strategies, confidence, description, chart_path=None):
        """ارسال سیگنال به کانال تلگرام"""

        # بررسی زمان آخرین سیگنال برای این ارز
        last_signal_key = f"{symbol}_{signal_type}"
        current_time = time.time()

        if last_signal_key in self.signal_history:
            last_time = self.signal_history[last_signal_key]
            # اگر کمتر از زمان تعیین شده از آخرین سیگنال گذشته باشد، سیگنال جدید ارسال نمی‌شود
            if current_time - last_time < BOT_SETTINGS['signal_cooldown_hours'] * 3600:
                hours_ago = (current_time - last_time) / 3600
                logger.info(
                    f"⏳ سیگنال {signal_type} برای {symbol} {hours_ago:.1f} ساعت پیش ارسال شده و در حالت انتظار است")
                return False

        # بررسی محدودیت تعداد سیگنال روزانه
        if current_time - self.last_signal_reset > 86400:  # 24 ساعت
            self.daily_signal_count = 0
            self.last_signal_reset = current_time

        if self.daily_signal_count >= BOT_SETTINGS['max_signals_per_day']:
            logger.warning(f"⚠️ محدودیت تعداد سیگنال روزانه ({BOT_SETTINGS['max_signals_per_day']}) رسیده است")
            return False

        # ساخت پیام سیگنال
        signal_emoji = "🟢" if signal_type == "خرید" else "🔴"

        message = f"{signal_emoji} <b>سیگنال {signal_type} | {symbol} | {timeframe}</b>\n\n"
        message += f"⚡️ <b>درصد اطمینان:</b> {confidence}%\n"
        message += f"💵 <b>قیمت ورود:</b> {entry_price:.4f}\n"
        message += f"🛑 <b>حد ضرر:</b> {stop_loss:.4f}\n"
        message += f"🎯 <b>حد سود:</b> {take_profit:.4f}\n\n"

        message += "<b>استراتژی‌های تأییدکننده:</b>\n"
        for strategy, conf in strategies.items():
            emoji = "✅" if conf >= 80 else "⚠️"
            message += f"{emoji} {strategy}: {conf}%\n"

        message += f"\n<b>توضیحات:</b>\n{description}\n\n"
        message += f"❗️ <i>این سیگنال‌ها صرفاً جنبه آموزشی داشته و توصیه سرمایه‌گذاری نیست.</i>"

        # ارسال پیام به تلگرام
        send_success = self.send_telegram_message(message, chart_path)

        if send_success:
            # فقط در صورت موفقیت، به‌روزرسانی تاریخچه سیگنال‌ها و افزایش شمارنده
            self.signal_history[last_signal_key] = current_time
            self.daily_signal_count += 1
            self.signal_count += 1
            logger.info(f"🚀 سیگنال {signal_type} برای {symbol} در تایم‌فریم {timeframe} با موفقیت ارسال شد")
            return True
        else:
            logger.error(f"❌ خطا در ارسال سیگنال {signal_type} برای {symbol} در تایم‌فریم {timeframe}")
            return False

    def send_conditional_signal(self, symbol, timeframe, signal_type, entry_condition, entry_price,
                                stop_loss, take_profit_levels, strategies, confidence, description, chart_path=None):
        """
        ارسال سیگنال شرطی به کانال تلگرام

        Args:
            symbol: نماد ارز
            timeframe: تایم‌فریم
            signal_type: نوع سیگنال (خرید یا فروش)
            entry_condition: شرط ورود به معامله
            entry_price: قیمت ورود
            stop_loss: حد ضرر
            take_profit_levels: لیستی از حدهای سود در سطوح مختلف (list)
            strategies: استراتژی‌های تأییدکننده (dict)
            confidence: درصد اطمینان
            description: توضیحات
            chart_path: مسیر فایل نمودار (اختیاری)

        Returns:
            نتیجه ارسال سیگنال (True یا False)
        """
        # بررسی زمان آخرین سیگنال برای این ارز
        last_signal_key = f"{symbol}_{signal_type}_conditional"
        current_time = time.time()

        if last_signal_key in self.signal_history:
            last_time = self.signal_history[last_signal_key]
            # اگر کمتر از زمان تعیین شده از آخرین سیگنال گذشته باشد، سیگنال جدید ارسال نمی‌شود
            if current_time - last_time < BOT_SETTINGS['signal_cooldown_hours'] * 3600:
                hours_ago = (current_time - last_time) / 3600
                logger.info(
                    f"⏳ سیگنال شرطی {signal_type} برای {symbol} {hours_ago:.1f} ساعت پیش ارسال شده و در حالت انتظار است")
                return False

        # بررسی محدودیت تعداد سیگنال روزانه
        if current_time - self.last_signal_reset > 86400:  # 24 ساعت
            self.daily_signal_count = 0
            self.last_signal_reset = current_time

        if self.daily_signal_count >= BOT_SETTINGS['max_signals_per_day']:
            logger.warning(f"⚠️ محدودیت تعداد سیگنال روزانه ({BOT_SETTINGS['max_signals_per_day']}) رسیده است")
            return False

        # ساخت پیام سیگنال
        signal_emoji = "🟢" if signal_type == "خرید" else "🔴"
        condition_emoji = "⚡"

        message = f"{signal_emoji} <b>سیگنال شرطی {signal_type} | {symbol} | {timeframe}</b>\n\n"
        message += f"{condition_emoji} <b>شرط ورود:</b> {entry_condition}\n"
        message += f"⚡️ <b>درصد اطمینان:</b> {confidence}%\n"
        message += f"💵 <b>قیمت ورود پیشنهادی:</b> {entry_price:.6f}\n"
        message += f"🛑 <b>حد ضرر:</b> {stop_loss:.6f}\n"

        # اضافه کردن حدهای سود چندگانه
        for i, tp in enumerate(take_profit_levels, 1):
            message += f"🎯 <b>حد سود {i}:</b> {tp:.6f}\n"

        message += "\n<b>استراتژی‌های تأییدکننده:</b>\n"
        for strategy, conf in strategies.items():
            emoji = "✅" if conf >= 80 else "⚠️"
            message += f"{emoji} {strategy}: {conf}%\n"

        message += f"\n<b>توضیحات:</b>\n{description}\n\n"
        message += f"❗️ <i>این سیگنال‌ها صرفاً جنبه آموزشی داشته و توصیه سرمایه‌گذاری نیست.</i>"

        # ارسال پیام به تلگرام
        send_success = self.send_telegram_message(message, chart_path)

        if send_success:
            # فقط در صورت موفقیت، به‌روزرسانی تاریخچه سیگنال‌ها و افزایش شمارنده
            self.signal_history[last_signal_key] = current_time
            self.daily_signal_count += 1
            self.signal_count += 1
            logger.info(f"🚀 سیگنال شرطی {signal_type} برای {symbol} در تایم‌فریم {timeframe} با موفقیت ارسال شد")
            return True
        else:
            logger.error(f"❌ خطا در ارسال سیگنال شرطی {signal_type} برای {symbol} در تایم‌فریم {timeframe}")
            return False

    def add_colab_survival_mechanisms(self):
        """افزودن مکانیسم‌های بقا در محیط گوگل کولب"""
        self.state_file = f"{BASE_DIR}/bot_state.json"
        self.last_save_time = time.time()
        self.save_interval = 300  # ۵ دقیقه

        # بررسی و ایجاد پوشه‌های لازم
        if COLAB_ENV:
            import IPython
            from IPython.display import display, Javascript
            import threading

            logger.info("🛡️ مکانیسم‌های محافظت از قطع اتصال کولب فعال شد")

            # ۱. تابع بازیابی وضعیت قبلی
            self.load_state()

            # ۲. تابع زنده نگه داشتن کولب
            def keep_alive():
                # کد جاوااسکریپت برای جلوگیری از قطع شدن اتصال کولب با شبیه‌سازی فعالیت کاربر
                display(Javascript('''
                function ClickConnect(){
                    console.log("کلیک اتوماتیک");
                    document.querySelector("colab-toolbar-button#connect").click()
                }
                setInterval(ClickConnect, 60000)

                // شبیه‌سازی فعالیت کاربر با اسکرول کردن صفحه
                function keepAlive() {
                    document.body.dispatchEvent(new KeyboardEvent('keydown', {key: 'ArrowUp'}));
                    setTimeout(function() {
                        document.body.dispatchEvent(new KeyboardEvent('keydown', {key: 'ArrowDown'}));
                    }, 500);
                }
                setInterval(keepAlive, 30000);
                '''))

                # زنده نگه داشتن اتصال با درخواست‌های منظم
                while True:
                    try:
                        # این کد پاسخ نخواهد داد اما باعث فعالیت می‌شود
                        temp_var = 1 + 1
                        # ذخیره منظم وضعیت
                        current_time = time.time()
                        if current_time - self.last_save_time > self.save_interval:
                            self.save_state()
                            self.last_save_time = current_time

                        # هر ۵ دقیقه یک گزارش سلامت ارسال می‌کنیم
                        if int(current_time) % 300 < 10:
                            logger.info(f"💓 ربات فعال است - سیگنال‌های ارسال شده: {self.signal_count}")

                        time.sleep(30)  # هر ۳۰ ثانیه بررسی می‌کنیم
                    except Exception as e:
                        logger.error(f"خطا در مکانیسم نگهداری اتصال: {e}")
                        time.sleep(60)

            # شروع تابع زنده نگه داشتن در یک ترد جداگانه
            keep_alive_thread = threading.Thread(target=keep_alive, daemon=True)
            keep_alive_thread.start()

            # ۳. شروع یک ترد برای نمایش گزارش های وضعیت
            def show_status():
                while True:
                    try:
                        print(
                            f"\r📡 ربات فعال - آخرین ذخیره: {time.strftime('%H:%M:%S', time.localtime(self.last_save_time))} - سیگنال‌ها: {self.signal_count}",
                            end="")
                        time.sleep(10)
                    except:
                        time.sleep(30)

            status_thread = threading.Thread(target=show_status, daemon=True)
            status_thread.start()

    def save_state(self):
        """ذخیره وضعیت فعلی ربات برای بازیابی در صورت قطع اتصال"""
        try:
            state = {
                'signal_count': self.signal_count,
                'signal_history': self.signal_history,
                'last_signal_reset': self.last_signal_reset,
                'daily_signal_count': self.daily_signal_count,
                'last_analysis_times': last_analysis_times,
                'timestamp': time.time()
            }

            with open(self.state_file, 'w') as f:
                json.dump(state, f)

            # ذخیره وضعیت صرافی‌ها
            exchange_state_file = f"{BASE_DIR}/exchange_state.json"
            with open(exchange_state_file, 'w') as f:
                json.dump(exchange_data, f)

            logger.info(f"💾 وضعیت ربات ذخیره شد - {time.strftime('%Y-%m-%d %H:%M:%S')}")
        except Exception as e:
            logger.error(f"❌ خطا در ذخیره وضعیت ربات: {e}")

    def load_state(self):
        """بازیابی وضعیت ذخیره شده قبلی در صورت وجود"""
        global last_analysis_times

        try:
            if os.path.exists(self.state_file):
                with open(self.state_file, 'r') as f:
                    state = json.load(f)

                # بررسی تازگی وضعیت (حداکثر ۶ ساعت)
                if time.time() - state.get('timestamp', 0) < 6 * 3600:
                    self.signal_count = state.get('signal_count', 0)
                    self.signal_history = state.get('signal_history', {})
                    self.last_signal_reset = state.get('last_signal_reset', time.time())
                    self.daily_signal_count = state.get('daily_signal_count', 0)
                    last_analysis_times = state.get('last_analysis_times', {})

                    # نمایش پیام بازیابی
                    logger.info(f"🔄 وضعیت قبلی ربات بازیابی شد - {len(self.signal_history)} سیگنال در تاریخچه")

                    # ارسال پیام بازیابی به تلگرام
                    if self.telegram_connected:
                        restart_message = f"🔄 ربات پس از قطع اتصال مجددا راه‌اندازی شد\n"
                        restart_message += f"📊 تعداد سیگنال‌های ارسال شده تا کنون: {self.signal_count}\n"
                        restart_message += f"⏰ زمان بازیابی: {time.strftime('%Y-%m-%d %H:%M:%S')}"

                        self.send_telegram_message(restart_message)
                else:
                    logger.warning("⚠️ وضعیت ذخیره شده قدیمی است (بیش از ۶ ساعت) و بازیابی نشد")

                # بازیابی وضعیت صرافی‌ها
                exchange_state_file = f"{BASE_DIR}/exchange_state.json"
                if os.path.exists(exchange_state_file):
                    with open(exchange_state_file, 'r') as f:
                        global exchange_data
                        exchange_data = json.load(f)
        except Exception as e:
            logger.error(f"❌ خطا در بازیابی وضعیت ربات: {e}")


    def diagnose_signals(self):
        """بررسی وضعیت سیگنال‌ها و شمارنده‌ها"""
        logger.info(f"📊 تعداد کل سیگنال‌های ارسال شده: {self.signal_count}")
        logger.info(f"📊 تعداد سیگنال‌های امروز: {self.daily_signal_count}/{BOT_SETTINGS['max_signals_per_day']}")
        logger.info(
            f"📊 زمان آخرین ریست شمارنده: {dt.datetime.fromtimestamp(self.last_signal_reset).strftime('%Y-%m-%d %H:%M:%S')}")

        # بررسی تاریخچه سیگنال‌ها
        logger.info(f"📊 تعداد سیگنال‌های در تاریخچه: {len(self.signal_history)}")

        # بررسی اتصال به تلگرام
        if self.telegram_bot:
            logger.info("✅ اتصال به ربات تلگرام برقرار است")
            # ارسال یک پیام تست فقط برای تشخیص
            try:
                self.telegram_bot.get_me()
                logger.info("✅ API تلگرام به درستی پاسخ می‌دهد")
            except Exception as e:
                logger.error(f"❌ خطا در اتصال به API تلگرام: {e}")
        else:
            logger.error("❌ اتصال به ربات تلگرام برقرار نیست")

    def create_candlestick_chart(self, df, symbol, timeframe, strategies=None, indicators=None,
                                 support_resistance=None, patterns=None, entry_exit=None, title=None):
        """
        ایجاد نمودار کندل‌استیک با همه اندیکاتورها و علامت‌گذاری‌های لازم - بهبود یافته و با مدیریت خطا

        Args:
            df: دیتافریم داده‌ها
            symbol: نماد ارز
            timeframe: تایم‌فریم
            strategies: استراتژی‌ها و نتایج آن‌ها (dict)
            indicators: اندیکاتورهای محاسبه شده (dict)
            support_resistance: سطوح حمایت و مقاومت (dict)
            patterns: الگوهای شناسایی شده (dict)
            entry_exit: نقاط ورود و خروج (dict)
            title: عنوان نمودار (str)

        Returns:
            مسیر فایل نمودار ذخیره شده
        """
        try:
            # گرفتن زیرمجموعه آخر داده‌ها برای نمایش بهتر در نمودار
            if len(df) > 150:
                plot_df = df.iloc[-150:]
            else:
                plot_df = df

            # بررسی معتبر بودن داده‌ها
            if len(plot_df) < 5:
                logger.warning(f"داده‌های ناکافی برای رسم نمودار {symbol} - {timeframe}")
                # رسم یک نمودار ساده بدون اندیکاتورها
                chart_filename = f"{BASE_DIR}/charts/{symbol.replace('/', '_')}_{timeframe}_{int(time.time())}.png"
                plt.figure(figsize=(10, 6))
                plt.title(f"{symbol} - {timeframe} - داده‌های ناکافی")
                plt.text(0.5, 0.5, "داده‌های ناکافی برای رسم نمودار",
                         horizontalalignment='center', verticalalignment='center',
                         transform=plt.gca().transAxes, fontsize=14)
                plt.savefig(chart_filename, dpi=200)
                plt.close()
                return chart_filename

            # تبدیل داده‌ها به فرمت مناسب برای mplfinance
            ohlc_data = plot_df[['open', 'high', 'low', 'close', 'volume']].copy()

            # تنظیم استایل نمودار
            mc = mpf.make_marketcolors(up='#00b060', down='#fe3032',
                                       edge='inherit',
                                       wick={'up': '#00b060', 'down': '#fe3032'},
                                       volume={'up': '#007046', 'down': '#a11d20'})

            s = mpf.make_mpf_style(base_mpf_style='charles', marketcolors=mc,
                                   rc={'font.size': 10, 'axes.labelsize': 12},
                                   y_on_right=False)

            # تنظیم پنل‌های اضافی برای اندیکاتورها
            additional_panels = []

            # افزودن اندیکاتورهای مختلف به نمودار
            apds = []

            # رنگ‌ها
            colors = ['#f5d742', '#5cc9f5', '#da70d6', '#ff7f0e', '#e377c2', '#bcbd22', '#17becf']

            # نمایش میانگین‌های متحرک اگر موجود باشند
            if indicators and 'ema' in indicators and indicators['ema']:
                ema_data = []
                for i, period in enumerate([8, 21, 50, 200]):
                    try:
                        if f'ema_{period}' in plot_df.columns:
                            ema_series = plot_df[f'ema_{period}']
                        elif f'ema_{period}' in df.columns:
                            # اگر در plot_df نیست ولی در df اصلی هست، محاسبه کنیم
                            ema_series = df[f'ema_{period}'].iloc[-len(plot_df):]
                        else:
                            # اگر در هیچکدام نیست، محاسبه کنیم
                            ema_series = ta.trend.ema_indicator(plot_df['close'], window=period)

                        # بررسی معتبر بودن داده‌ها
                        if not ema_series.isnull().all() and len(ema_series.dropna()) > 0:
                            # مقادیر NaN را با میانگین جایگزین کنیم
                            mean_value = ema_series.mean()
                            ema_series = ema_series.fillna(mean_value)

                            color = colors[i % len(colors)]
                            ema_data.append(
                                mpf.make_addplot(ema_series, color=color, width=1,
                                                 label=f'EMA {period}')
                            )
                            plot_df[f'ema_{period}'] = ema_series
                    except Exception as e:
                        logger.warning(f"خطا در محاسبه EMA {period}: {e}")

                if ema_data:
                    apds.extend(ema_data)

            # نمایش باندهای بولینگر اگر موجود باشند
            if indicators and 'bollinger' in indicators and indicators['bollinger']:
                try:
                    if not all(x in plot_df.columns for x in ['bb_upper', 'bb_middle', 'bb_lower']):
                        # محاسبه باندهای بولینگر
                        bb_ind = ta.volatility.BollingerBands(plot_df['close'], window=20, window_dev=2)
                        plot_df['bb_upper'] = bb_ind.bollinger_hband()
                        plot_df['bb_middle'] = bb_ind.bollinger_mavg()
                        plot_df['bb_lower'] = bb_ind.bollinger_lband()

                    # بررسی معتبر بودن داده‌ها
                    for bb_col in ['bb_upper', 'bb_middle', 'bb_lower']:
                        if not plot_df[bb_col].isnull().all() and len(plot_df[bb_col].dropna()) > 0:
                            # مقادیر NaN را با میانگین جایگزین کنیم
                            mean_value = plot_df[bb_col].mean()
                            plot_df[bb_col] = plot_df[bb_col].fillna(mean_value)

                    apds.append(mpf.make_addplot(plot_df['bb_upper'], color='#6fa8dc', width=0.7, linestyle='--'))
                    apds.append(mpf.make_addplot(plot_df['bb_middle'], color='#6fa8dc', width=0.7))
                    apds.append(mpf.make_addplot(plot_df['bb_lower'], color='#6fa8dc', width=0.7, linestyle='--'))
                except Exception as e:
                    logger.warning(f"خطا در محاسبه باندهای بولینگر: {e}")

            # نمایش سطوح فیبوناچی اگر موجود باشند
            if indicators and 'fibonacci' in indicators and indicators['fibonacci'] and 'fibonacci_data' in indicators:
                try:
                    fib_data = indicators['fibonacci_data']
                    if 'fibonacci_retracements' in fib_data and fib_data['fibonacci_retracements']:
                        # رسم آخرین retracement
                        last_retracement = fib_data['fibonacci_retracements'][-1]
                        fib_levels = last_retracement['levels']

                        for level, value in fib_levels.items():
                            if isinstance(level, (int, float)):  # اطمینان از اینکه کلید یک عدد است
                                level_label = f"Fib {level}"
                                color = '#edb879' if level != 0.5 else '#e36f7a'  # سطح 0.5 را با رنگ متفاوت نمایش می‌دهیم
                                # بررسی معتبر بودن مقدار
                                if not np.isnan(value) and np.isfinite(value):
                                    apds.append(
                                        mpf.make_addplot([value] * len(plot_df), color=color, width=0.7,
                                                         linestyle='--', panel=0)
                                    )
                except Exception as e:
                    logger.warning(f"خطا در رسم سطوح فیبوناچی: {e}")

            # نمایش ایچیموکو اگر موجود باشد
            if indicators and 'ichimoku' in indicators and indicators['ichimoku']:
                try:
                    # محاسبه خطوط ایچیموکو اگر موجود نیستند
                    if not all(x in plot_df.columns for x in ['tenkan_sen', 'kijun_sen']):
                        # تنکان-سن (Tenkan-sen) - خط تبدیل
                        high_9 = plot_df['high'].rolling(window=9).max()
                        low_9 = plot_df['low'].rolling(window=9).min()
                        plot_df['tenkan_sen'] = (high_9 + low_9) / 2

                        # کیجون-سن (Kijun-sen) - خط پایه
                        high_26 = plot_df['high'].rolling(window=26).max()
                        low_26 = plot_df['low'].rolling(window=26).min()
                        plot_df['kijun_sen'] = (high_26 + low_26) / 2

                    # بررسی معتبر بودن داده‌ها
                    for col in ['tenkan_sen', 'kijun_sen']:
                        if col in plot_df.columns and not plot_df[col].isnull().all() and len(
                                plot_df[col].dropna()) > 0:
                            # مقادیر NaN را با میانگین جایگزین کنیم
                            mean_value = plot_df[col].mean()
                            plot_df[col] = plot_df[col].fillna(mean_value)

                            if col == 'tenkan_sen':
                                apds.append(mpf.make_addplot(plot_df[col], color='#f5d742', width=0.7))
                            else:  # kijun_sen
                                apds.append(mpf.make_addplot(plot_df[col], color='#5cc9f5', width=0.7))

                    # Cloud (فقط اگر هر دو ستون senkou_span_a و senkou_span_b موجود باشند یا بتوانیم محاسبه کنیم)
                    if not all(x in plot_df.columns for x in ['senkou_span_a', 'senkou_span_b']):
                        # محاسبه senkou span
                        if 'tenkan_sen' in plot_df.columns and 'kijun_sen' in plot_df.columns:
                            plot_df['senkou_span_a'] = ((plot_df['tenkan_sen'] + plot_df['kijun_sen']) / 2).shift(26)

                            # سنکو اسپن B
                            high_52 = plot_df['high'].rolling(window=52).max()
                            low_52 = plot_df['low'].rolling(window=52).min()
                            plot_df['senkou_span_b'] = ((high_52 + low_52) / 2).shift(26)

                    # فقط اگر داده‌ها کافی باشند، اضافه می‌کنیم
                    span_cols = ['senkou_span_a', 'senkou_span_b']
                    for i, col in enumerate(span_cols):
                        if col in plot_df.columns and not plot_df[col].isnull().all() and len(
                                plot_df[col].dropna()) > 0:
                            # مقادیر NaN را با میانگین جایگزین کنیم یا حذف کنیم
                            non_null_values = plot_df[col].dropna().values
                            if len(non_null_values) > 0:
                                # فقط در صورتی که مقادیر غیر NaN داریم
                                mean_value = non_null_values.mean()
                                filled_series = plot_df[col].fillna(mean_value)

                                color = '#11a171' if i == 0 else '#e36f7a'
                                apds.append(mpf.make_addplot(filled_series, color=color, width=0.5))
                except Exception as e:
                    logger.warning(f"خطا در محاسبه ایچیموکو: {e}")

            # نمایش RSI در یک پنل جداگانه
            if indicators and 'rsi' in indicators and indicators['rsi']:
                try:
                    if 'rsi' not in plot_df.columns:
                        plot_df['rsi'] = ta.momentum.rsi(plot_df['close'], window=14)

                    # بررسی معتبر بودن داده‌ها
                    if not plot_df['rsi'].isnull().all() and len(plot_df['rsi'].dropna()) > 0:
                        # مقادیر NaN را با میانگین جایگزین کنیم
                        rsi_mean = plot_df['rsi'].dropna().mean()
                        plot_df['rsi'] = plot_df['rsi'].fillna(rsi_mean)

                        apds.append(
                            mpf.make_addplot(plot_df['rsi'], panel=1, color='#edb879', width=1,
                                             ylim=(0, 100))
                        )
                        # خطوط سطوح اشباع خرید و فروش
                        apds.append(mpf.make_addplot([70] * len(plot_df), panel=1, color='#e36f7a',
                                                     width=0.5, linestyle='--'))
                        apds.append(mpf.make_addplot([30] * len(plot_df), panel=1, color='#11a171',
                                                     width=0.5, linestyle='--'))
                        additional_panels.append('RSI')
                except Exception as e:
                    logger.warning(f"خطا در محاسبه RSI: {e}")

            # نمایش MACD در یک پنل جداگانه
            if indicators and 'macd' in indicators and indicators['macd']:
                try:
                    # محاسبه MACD اگر موجود نیست
                    if not all(x in plot_df.columns for x in ['macd', 'macd_signal']):
                        macd_ind = ta.trend.MACD(plot_df['close'], window_slow=26, window_fast=12, window_sign=9)
                        plot_df['macd'] = macd_ind.macd()
                        plot_df['macd_signal'] = macd_ind.macd_signal()
                        plot_df['macd_diff'] = macd_ind.macd_diff()

                    # بررسی معتبر بودن داده‌ها
                    valid_macd = (
                            not plot_df['macd'].isnull().all() and
                            not plot_df['macd_signal'].isnull().all() and
                            len(plot_df['macd'].dropna()) > 0 and
                            len(plot_df['macd_signal'].dropna()) > 0
                    )

                    if valid_macd:
                        # مقادیر NaN را با میانگین جایگزین کنیم
                        for col in ['macd', 'macd_signal']:
                            mean_value = plot_df[col].dropna().mean()
                            plot_df[col] = plot_df[col].fillna(mean_value)

                        macd_panel = 2 if 'RSI' in additional_panels else 1

                        apds.append(
                            mpf.make_addplot(plot_df['macd'], panel=macd_panel, color='#5cc9f5', width=1)
                        )
                        apds.append(
                            mpf.make_addplot(plot_df['macd_signal'], panel=macd_panel, color='#f5d742', width=1)
                        )

                        # ستون‌های هیستوگرام MACD
                        macd_hist = plot_df['macd'] - plot_df['macd_signal']

                        # جایگزینی مقادیر NaN
                        macd_hist = macd_hist.fillna(0)

                        apds.append(
                            mpf.make_addplot(macd_hist, type='bar', panel=macd_panel,
                                             color=['#11a171' if x > 0 else '#e36f7a' for x in macd_hist])
                        )

                        additional_panels.append('MACD')
                except Exception as e:
                    logger.warning(f"خطا در محاسبه MACD: {e}")

            # نمایش حجم معاملات در یک پنل جداگانه
            volume_panel = len(additional_panels) + 1
            if 'volume' in plot_df.columns and not plot_df['volume'].isnull().all() and plot_df['volume'].sum() > 0:
                try:
                    # برای اطمینان از معتبر بودن داده‌های حجم
                    plot_df['volume'] = plot_df['volume'].fillna(0)

                    # رنگ‌بندی ستون‌های حجم
                    volume_colors = []
                    for i in range(len(plot_df)):
                        if i >= len(plot_df) or not all(col in plot_df.columns for col in ['open', 'close']):
                            volume_colors.append('#11a171')  # پیش‌فرض سبز
                        else:
                            color = '#11a171' if plot_df['close'].iloc[i] >= plot_df['open'].iloc[i] else '#e36f7a'
                            volume_colors.append(color)

                    apds.append(
                        mpf.make_addplot(plot_df['volume'], panel=volume_panel, type='bar',
                                         color=volume_colors)
                    )
                    additional_panels.append('Volume')
                except Exception as e:
                    logger.warning(f"خطا در نمایش حجم معاملات: {e}")

            # اگر هیچ پنل اضافی نداریم، حداقل یک پنل خالی اضافه کنیم
            if not additional_panels:
                additional_panels.append('No Data')

            # تنظیم عنوان نمودار
            if not title:
                title = f"{symbol} - {timeframe} - {plot_df.index[-1].strftime('%Y-%m-%d %H:%M')}"

            # ایجاد نمودار
            fig, axes = mpf.plot(
                ohlc_data,
                type='candle',
                style=s,
                figsize=(14, 12 + len(additional_panels) * 2),
                title=title,
                ylabel='قیمت',
                volume=False,  # از پنل حجم داخلی استفاده نمی‌کنیم، زیرا خودمان آن را اضافه می‌کنیم
                panel_ratios=(4, *([1] * len(additional_panels))),
                addplot=apds,
                returnfig=True
            )

            # افزودن برچسب‌ها برای پنل‌های اضافی
            for i, panel_name in enumerate(additional_panels):
                if i + 1 < len(axes):  # بررسی محدوده آرایه
                    axes[i + 1].set_ylabel(panel_name)

            # افزودن نقاط ورود و خروج، سطوح حمایت و مقاومت و غیره
            ax = axes[0]  # اولین محور برای نمودار اصلی

            # ==== افزودن سطوح حمایت و مقاومت ====
            if support_resistance:
                for level_type, levels in support_resistance.items():
                    color = '#11a171' if level_type == 'support' else '#e36f7a'
                    for level in levels:
                        # بررسی معتبر بودن مقدار
                        if not np.isnan(level) and np.isfinite(level):
                            ax.axhline(level, color=color, linestyle='--', alpha=0.7, linewidth=1)

            # ==== افزودن الگوهای شناسایی شده ====
            if patterns:
                for pattern_type, pattern_points in patterns.items():
                    for point in pattern_points:
                        try:
                            # تطبیق با ساختار داده‌های MSB و Order Block
                            if pattern_type == 'market_structure_break_bullish':
                                # اگر point یک tuple است (index, price)
                                if isinstance(point, tuple) and len(point) >= 2:
                                    orig_idx, price = point[0], point[1]
                                    # بررسی معتبر بودن مقادیر
                                    if not isinstance(orig_idx, (int, np.integer)) or np.isnan(
                                            price) or not np.isfinite(price):
                                        continue

                                    # بررسی شاخص در محدوده
                                    if orig_idx >= len(df) - len(plot_df) and orig_idx < len(df):
                                        idx = orig_idx - (len(df) - len(plot_df))
                                        ax.scatter(idx, price, color='green', marker='^', s=120, alpha=0.8)
                                        ax.annotate('MSB', xy=(idx, price), xytext=(idx, price * 1.02),
                                                    color='green', fontsize=8, ha='center')
                                # اگر point یک dictionary است
                                elif isinstance(point, dict) and 'index' in point and 'price' in point:
                                    orig_idx, price = point['index'], point['price']
                                    # بررسی معتبر بودن مقادیر
                                    if not isinstance(orig_idx, (int, np.integer)) or np.isnan(
                                            price) or not np.isfinite(price):
                                        continue

                                    if orig_idx >= len(df) - len(plot_df) and orig_idx < len(df):
                                        idx = orig_idx - (len(df) - len(plot_df))
                                        ax.scatter(idx, price, color='green', marker='^', s=120, alpha=0.8)
                                        ax.annotate('MSB', xy=(idx, price), xytext=(idx, price * 1.02),
                                                    color='green', fontsize=8, ha='center')

                            elif pattern_type == 'market_structure_break_bearish':
                                # اگر point یک tuple است (index, price)
                                if isinstance(point, tuple) and len(point) >= 2:
                                    orig_idx, price = point[0], point[1]
                                    # بررسی معتبر بودن مقادیر
                                    if not isinstance(orig_idx, (int, np.integer)) or np.isnan(
                                            price) or not np.isfinite(price):
                                        continue

                                    # بررسی شاخص در محدوده
                                    if orig_idx >= len(df) - len(plot_df) and orig_idx < len(df):
                                        idx = orig_idx - (len(df) - len(plot_df))
                                        ax.scatter(idx, price, color='red', marker='v', s=120, alpha=0.8)
                                        ax.annotate('MSB', xy=(idx, price), xytext=(idx, price * 0.98),
                                                    color='red', fontsize=8, ha='center')
                                # اگر point یک dictionary است
                                elif isinstance(point, dict) and 'index' in point and 'price' in point:
                                    orig_idx, price = point['index'], point['price']
                                    # بررسی معتبر بودن مقادیر
                                    if not isinstance(orig_idx, (int, np.integer)) or np.isnan(
                                            price) or not np.isfinite(price):
                                        continue

                                    if orig_idx >= len(df) - len(plot_df) and orig_idx < len(df):
                                        idx = orig_idx - (len(df) - len(plot_df))
                                        ax.scatter(idx, price, color='red', marker='v', s=120, alpha=0.8)
                                        ax.annotate('MSB', xy=(idx, price), xytext=(idx, price * 0.98),
                                                    color='red', fontsize=8, ha='center')

                            elif pattern_type == 'order_block_bullish':
                                # بررسی اگر point یک dict است
                                if isinstance(point, dict) and 'index' in point and 'high' in point and 'low' in point:
                                    orig_idx = point['index']
                                    high = point['high']
                                    low = point['low']

                                    # بررسی معتبر بودن مقادیر
                                    if (not isinstance(orig_idx, (int, np.integer)) or
                                            np.isnan(high) or not np.isfinite(high) or
                                            np.isnan(low) or not np.isfinite(low)):
                                        continue

                                    # بررسی شاخص در محدوده
                                    if orig_idx >= len(df) - len(plot_df) and orig_idx < len(df):
                                        idx = orig_idx - (len(df) - len(plot_df))
                                        rect = Rectangle((idx - 0.4, low), 0.8, high - low,
                                                         color='green', alpha=0.3)
                                        ax.add_patch(rect)
                                        ax.annotate('OB', xy=(idx, low), xytext=(idx, low * 0.99),
                                                    color='green', fontsize=8, ha='center')
                                        # اضافه کردن خط میتسوبو
                                        if 'mitsubo' in point and not np.isnan(point['mitsubo']) and np.isfinite(
                                                point['mitsubo']):
                                            ax.axhline(y=point['mitsubo'], linestyle='--', color='green', alpha=0.6,
                                                       xmin=(idx - 1) / (len(plot_df)),
                                                       xmax=(idx + 1) / (len(plot_df)))

                            elif pattern_type == 'order_block_bearish':
                                # بررسی اگر point یک dict است
                                if isinstance(point, dict) and 'index' in point and 'high' in point and 'low' in point:
                                    orig_idx = point['index']
                                    high = point['high']
                                    low = point['low']

                                    # بررسی معتبر بودن مقادیر
                                    if (not isinstance(orig_idx, (int, np.integer)) or
                                            np.isnan(high) or not np.isfinite(high) or
                                            np.isnan(low) or not np.isfinite(low)):
                                        continue

                                    # بررسی شاخص در محدوده
                                    if orig_idx >= len(df) - len(plot_df) and orig_idx < len(df):
                                        idx = orig_idx - (len(df) - len(plot_df))
                                        rect = Rectangle((idx - 0.4, low), 0.8, high - low,
                                                         color='red', alpha=0.3)
                                        ax.add_patch(rect)
                                        ax.annotate('OB', xy=(idx, high), xytext=(idx, high * 1.01),
                                                    color='red', fontsize=8, ha='center')
                                        # اضافه کردن خط میتسوبو
                                        if 'mitsubo' in point and not np.isnan(point['mitsubo']) and np.isfinite(
                                                point['mitsubo']):
                                            ax.axhline(y=point['mitsubo'], linestyle='--', color='red', alpha=0.6,
                                                       xmin=(idx - 1) / (len(plot_df)),
                                                       xmax=(idx + 1) / (len(plot_df)))
                        except Exception as e:
                            logger.warning(f"خطا در رسم الگوی {pattern_type}: {e}")

            # ==== اگر نقاط ورود و خروج تعریف شده باشند، آن‌ها را به نمودار اضافه می‌کنیم ====
            if entry_exit:
                try:
                    if 'entries' in entry_exit:
                        for entry in entry_exit['entries']:
                            # بررسی اینکه entry یک dictionary است یا یک tuple
                            if isinstance(entry, dict):
                                orig_idx = entry.get('index')
                                price = entry.get('price')
                                if orig_idx is None or price is None:
                                    continue
                            elif isinstance(entry, tuple) and len(entry) >= 2:
                                orig_idx = entry[0]
                                price = entry[1]
                            else:
                                continue

                            # بررسی معتبر بودن مقادیر
                            if not isinstance(orig_idx, (int, np.integer)) or np.isnan(price) or not np.isfinite(price):
                                continue

                            # بررسی می‌کنیم که آیا شاخص در محدوده plot_df است
                            if orig_idx >= len(df) - len(plot_df) and orig_idx < len(df):
                                # نگاشت شاخص از داده‌های اصلی به plot_df
                                idx = orig_idx - (len(df) - len(plot_df))

                                # رسم نقطه ورود
                                ax.scatter(idx, price, color='blue', marker='o', s=100, alpha=0.8)
                                ax.annotate('ورود', xy=(idx, price),
                                            xytext=(idx, price * 1.02),
                                            color='blue', fontsize=9, ha='center',
                                            arrowprops=dict(facecolor='blue', shrink=0.05, alpha=0.6))

                    # رسم حد ضرر
                    if 'stop_loss' in entry_exit and not np.isnan(entry_exit['stop_loss']) and np.isfinite(
                            entry_exit['stop_loss']):
                        stop_loss = entry_exit['stop_loss']
                        ax.axhline(y=stop_loss, color='red', linestyle='--', alpha=0.8, linewidth=1)
                        ax.text(len(plot_df) - 10, stop_loss, 'حد ضرر', color='red', fontsize=9, ha='right')

                    # رسم حدهای سود
                    if 'take_profit' in entry_exit:
                        if isinstance(entry_exit['take_profit'], list):
                            # اگر چندین حد سود داریم
                            for i, tp in enumerate(entry_exit['take_profit'], 1):
                                if not np.isnan(tp) and np.isfinite(tp):
                                    ax.axhline(y=tp, color='green', linestyle='--', alpha=0.8, linewidth=1)
                                    ax.text(len(plot_df) - 10, tp, f'حد سود {i}', color='green', fontsize=9, ha='right')
                        else:
                            # اگر یک حد سود داریم
                            tp = entry_exit['take_profit']
                            if not np.isnan(tp) and np.isfinite(tp):
                                ax.axhline(y=tp, color='green', linestyle='--', alpha=0.8, linewidth=1)
                                ax.text(len(plot_df) - 10, tp, 'حد سود', color='green', fontsize=9, ha='right')

                    if 'exits' in entry_exit:
                        for exit in entry_exit['exits']:
                            # بررسی اینکه exit یک dictionary است یا یک tuple
                            if isinstance(exit, dict):
                                orig_idx = exit.get('index')
                                if orig_idx is None:
                                    continue
                            elif isinstance(exit, tuple) and len(exit) >= 1:
                                orig_idx = exit[0]
                            else:
                                continue

                            # بررسی معتبر بودن مقدار
                            if not isinstance(orig_idx, (int, np.integer)):
                                continue

                            # بررسی می‌کنیم که آیا شاخص در محدوده plot_df است
                            if orig_idx >= len(df) - len(plot_df) and orig_idx < len(df):
                                # نگاشت شاخص از داده‌های اصلی به plot_df
                                idx = orig_idx - (len(df) - len(plot_df))

                                # بررسی اینکه آیا idx در محدوده معتبر است
                                if 0 <= idx < len(plot_df):
                                    high_val = plot_df['high'].iloc[idx]
                                    if not np.isnan(high_val) and np.isfinite(high_val):
                                        ax.annotate('خروج', xy=(idx, high_val * 1.005),
                                                    xytext=(idx, high_val * 1.015),
                                                    color='#e36f7a', fontsize=9, ha='center')
                except Exception as e:
                    logger.warning(f"خطا در رسم نقاط ورود و خروج: {e}")

            # افزودن اطلاعات استراتژی به نمودار
            if strategies:
                try:
                    # جابجا کردن متن استراتژی به بالا سمت چپ نمودار
                    info_text = ""
                    total_strength = 0
                    count = 0

                    for name, strength in strategies.items():
                        # بررسی معتبر بودن مقدار
                        if not isinstance(strength, (int, float)) or np.isnan(strength) or not np.isfinite(strength):
                            continue

                        info_text += f"{name}: {strength:.0f}%\n"
                        total_strength += strength
                        count += 1

                    if count > 0:
                        avg_strength = total_strength / count
                        info_text += f"\nاطمینان کلی: {avg_strength:.0f}%"

                    # نمایش متن در سمت چپ بالا
                    if info_text:  # فقط اگر متن داریم
                        plt.figtext(0.02, 0.95, info_text, fontsize=9, bbox=dict(facecolor='white', alpha=0.7))
                except Exception as e:
                    logger.warning(f"خطا در نمایش اطلاعات استراتژی: {e}")

            # ذخیره نمودار
            chart_filename = f"{BASE_DIR}/charts/{symbol.replace('/', '_')}_{timeframe}_{int(time.time())}.png"
            plt.savefig(chart_filename, dpi=180, bbox_inches='tight')
            plt.close(fig)

            # فراخوانی صریح جمع‌آوری زباله و آزادسازی حافظه
            plt.close('all')
            gc.collect()

            return chart_filename

        except Exception as e:
            # در صورت هر گونه خطا، یک نمودار ساده بدون اندیکاتورها برمیگردانیم
            logger.error(f"خطا در رسم نمودار {symbol} - {timeframe}: {e}")
            logger.error(traceback.format_exc())

            try:
                # رسم یک نمودار ساده با پیام خطا
                chart_filename = f"{BASE_DIR}/charts/{symbol.replace('/', '_')}_{timeframe}_{int(time.time())}_error.png"
                plt.figure(figsize=(10, 6))
                plt.title(f"{symbol} - {timeframe} - خطا در رسم نمودار")
                plt.text(0.5, 0.5, f"خطا در رسم نمودار: {str(e)}",
                         horizontalalignment='center', verticalalignment='center',
                         transform=plt.gca().transAxes, fontsize=12)
                plt.savefig(chart_filename, dpi=200)
                plt.close()
                return chart_filename
            except:
                # اگر حتی در ساخت نمودار خطا هم مشکل داشتیم، آدرس خالی برمیگردانیم
                return None

    def analyze_market_structure(self, df, window=10, fib_factor=0.273):
        """
        تحلیل ساختار بازار و شناسایی شکست‌های ساختاری و بلوک‌های سفارش

        Args:
            df: دیتافریم داده‌ها
            window: پنجره برای شناسایی سوینگ‌های قیمتی
            fib_factor: ضریب فیبوناچی برای محاسبه بلوک‌های سفارش (0.273 برای تطابق با TradingView)

        Returns:
            نتایج تحلیل ساختار بازار و بلوک‌های سفارش
        """
        # ایجاد یک کپی از دیتافریم
        df_ms = df.copy()

        # یافتن نقاط سوینگ (اوج و حضیض محلی)
        df_ms['swing_high'] = df_ms['high'].rolling(window=window, center=True).apply(
            lambda x: 1 if x.iloc[window // 2] == max(x) else 0, raw=False)
        df_ms['swing_low'] = df_ms['low'].rolling(window=window, center=True).apply(
            lambda x: 1 if x.iloc[window // 2] == min(x) else 0, raw=False)

        # ساختارهای بازار را ذخیره می‌کنیم
        highs = []  # لیست اوج‌ها
        lows = []  # لیست حضیض‌ها

        # پیدا کردن تمام نقاط سوینگ و ذخیره آنها
        for i in range(window, len(df_ms) - window):
            if df_ms['swing_high'].iloc[i] == 1:
                # بررسی اینکه آیا این اوج محلی معتبر است
                is_valid = True
                # بررسی اوج‌های قبلی نزدیک - حذف نقاط سوینگ خیلی نزدیک به هم
                for h in reversed(highs):
                    if abs(i - h[0]) < window and abs(df_ms['high'].iloc[i] - h[1]) / h[1] < 0.005:
                        is_valid = False
                        break

                if is_valid:
                    highs.append((i, df_ms['high'].iloc[i]))

            if df_ms['swing_low'].iloc[i] == 1:
                # بررسی اینکه آیا این حضیض محلی معتبر است
                is_valid = True
                # بررسی حضیض‌های قبلی نزدیک
                for l in reversed(lows):
                    if abs(i - l[0]) < window and abs(df_ms['low'].iloc[i] - l[1]) / l[1] < 0.005:
                        is_valid = False
                        break

                if is_valid:
                    lows.append((i, df_ms['low'].iloc[i]))

        # برچسب‌زنی روند با استفاده از اوج‌ها و حضیض‌ها
        df_ms['higher_high'] = np.nan
        df_ms['lower_low'] = np.nan
        df_ms['higher_low'] = np.nan
        df_ms['lower_high'] = np.nan
        df_ms['msb_bullish'] = np.nan  # شکست ساختاری صعودی
        df_ms['msb_bearish'] = np.nan  # شکست ساختاری نزولی
        df_ms['orderblock_bullish'] = np.nan  # بلوک سفارش صعودی
        df_ms['orderblock_bearish'] = np.nan  # بلوک سفارش نزولی

        # بررسی روندها
        for i in range(2, len(highs)):
            # اوج بالاتر (Higher High)
            if highs[i][1] > highs[i - 1][1]:
                df_ms['higher_high'].iloc[highs[i][0]] = highs[i][1]
            # اوج پایین‌تر (Lower High)
            else:
                df_ms['lower_high'].iloc[highs[i][0]] = highs[i][1]

        for i in range(2, len(lows)):
            # حضیض بالاتر (Higher Low)
            if lows[i][1] > lows[i - 1][1]:
                df_ms['higher_low'].iloc[lows[i][0]] = lows[i][1]
            # حضیض پایین‌تر (Lower Low)
            else:
                df_ms['lower_low'].iloc[lows[i][0]] = lows[i][1]

        # شناسایی شکست ساختار بازار (Market Structure Break) - تغییر الگوریتم مطابق با TradingView
        msb_bullish = []
        msb_bearish = []

        # بررسی الگوی MSB صعودی و نزولی
        for i in range(2, len(highs)):
            # شکست صعودی: اوج جدید بالاتر از اوج قبلی + یک دره در بین آنها
            if highs[i][1] > highs[i - 1][1]:
                # بررسی آیا یک دره بین این دو اوج وجود دارد
                for l in lows:
                    if highs[i - 1][0] < l[0] < highs[i][0]:
                        df_ms['msb_bullish'].iloc[highs[i][0]] = df_ms['high'].iloc[highs[i][0]]
                        msb_bullish.append((highs[i][0], df_ms['high'].iloc[highs[i][0]]))
                        break

        for i in range(2, len(lows)):
            # شکست نزولی: حضیض جدید پایین‌تر از حضیض قبلی + یک اوج در بین آنها
            if lows[i][1] < lows[i - 1][1]:
                # بررسی آیا یک اوج بین این دو حضیض وجود دارد
                for h in highs:
                    if lows[i - 1][0] < h[0] < lows[i][0]:
                        df_ms['msb_bearish'].iloc[lows[i][0]] = df_ms['low'].iloc[lows[i][0]]
                        msb_bearish.append((lows[i][0], df_ms['low'].iloc[lows[i][0]]))
                        break

        # شناسایی بلوک‌های سفارش (Order Blocks) با الگوریتم بهبود یافته
        orderblock_bullish = []
        orderblock_bearish = []

        # یافتن بلوک سفارش صعودی (قبل از MSB نزولی)
        for msb_idx, _ in msb_bearish:
            # بررسی ۵ کندل قبل از MSB برای یافتن بهترین کندل صعودی
            ob_idx = max(0, msb_idx - 5)
            best_ob_idx = -1
            max_body_size = 0

            for i in range(max(0, msb_idx - 5), msb_idx):
                # اگر کندل صعودی است (بسته شدن بالاتر از قیمت باز شدن)
                if df_ms['close'].iloc[i] > df_ms['open'].iloc[i]:
                    body_size = abs(df_ms['close'].iloc[i] - df_ms['open'].iloc[i])
                    if body_size > max_body_size:
                        max_body_size = body_size
                        best_ob_idx = i

            if best_ob_idx != -1:
                ob_high = df_ms['high'].iloc[best_ob_idx]
                ob_low = df_ms['low'].iloc[best_ob_idx]
                # محاسبه ناحیه میتسوبو (mitsubo zone) که در TradingView استفاده می‌شود
                mitsubo_level = ob_low + (ob_high - ob_low) * fib_factor

                orderblock_bullish.append({
                    'index': best_ob_idx,
                    'high': ob_high,
                    'low': ob_low,
                    'mitsubo': mitsubo_level,
                    'body_size': max_body_size
                })
                df_ms['orderblock_bullish'].iloc[best_ob_idx] = ob_low

        # یافتن بلوک سفارش نزولی (قبل از MSB صعودی)
        for msb_idx, _ in msb_bullish:
            # بررسی ۵ کندل قبل از MSB برای یافتن بهترین کندل نزولی
            ob_idx = max(0, msb_idx - 5)
            best_ob_idx = -1
            max_body_size = 0

            for i in range(max(0, msb_idx - 5), msb_idx):
                # اگر کندل نزولی است (بسته شدن پایین‌تر از قیمت باز شدن)
                if df_ms['close'].iloc[i] < df_ms['open'].iloc[i]:
                    body_size = abs(df_ms['close'].iloc[i] - df_ms['open'].iloc[i])
                    if body_size > max_body_size:
                        max_body_size = body_size
                        best_ob_idx = i

            if best_ob_idx != -1:
                ob_high = df_ms['high'].iloc[best_ob_idx]
                ob_low = df_ms['low'].iloc[best_ob_idx]
                # محاسبه ناحیه میتسوبو (mitsubo zone) که در TradingView استفاده می‌شود
                mitsubo_level = ob_high - (ob_high - ob_low) * fib_factor

                orderblock_bearish.append({
                    'index': best_ob_idx,
                    'high': ob_high,
                    'low': ob_low,
                    'mitsubo': mitsubo_level,
                    'body_size': max_body_size
                })
                df_ms['orderblock_bearish'].iloc[best_ob_idx] = ob_high

        # تحلیل وضعیت کلی ساختار بازار
        msb_status = 'روند نامشخص'
        msb_strength = 0

        # آخرین شکست‌های ساختاری
        last_bullish_msb = None if not msb_bullish else msb_bullish[-1]
        last_bearish_msb = None if not msb_bearish else msb_bearish[-1]

        # اگر اخیراً یک شکست ساختاری صعودی داشته‌ایم
        if last_bullish_msb and (not last_bearish_msb or last_bullish_msb[0] > last_bearish_msb[0]):
            # چک می‌کنیم آیا قیمت بالای آخرین اوج شکسته شده است
            if df_ms['close'].iloc[-1] > last_bullish_msb[1]:
                msb_status = 'صعودی - تأیید شده'
                msb_strength = 90
            else:
                msb_status = 'صعودی - در انتظار تأیید'
                msb_strength = 70

        # اگر اخیراً یک شکست ساختاری نزولی داشته‌ایم
        elif last_bearish_msb and (not last_bullish_msb or last_bearish_msb[0] > last_bullish_msb[0]):
            # چک می‌کنیم آیا قیمت زیر آخرین حضیض شکسته شده است
            if df_ms['close'].iloc[-1] < last_bearish_msb[1]:
                msb_status = 'نزولی - تأیید شده'
                msb_strength = 90
            else:
                msb_status = 'نزولی - در انتظار تأیید'
                msb_strength = 70

        # شمارش تعداد ساختارهای بازار برای تحلیل روند
        hh_count = df_ms['higher_high'].count()
        lh_count = df_ms['lower_high'].count()
        hl_count = df_ms['higher_low'].count()
        ll_count = df_ms['lower_low'].count()

        trend_score = (hh_count + hl_count - lh_count - ll_count) / max(1,
                                                                        hh_count + hl_count + lh_count + ll_count) * 100

        # اضافه کردن نقاط ورود با استفاده از بلوک‌های سفارش
        entry_points = []
        current_price = df_ms['close'].iloc[-1]

        # برای بلوک‌های سفارش صعودی
        for ob in orderblock_bullish:
            # اگر قیمت در نزدیکی بلوک سفارش صعودی است
            if current_price <= ob['high'] * 1.05 and current_price >= ob['low'] * 0.95:
                # ورود: نزدیک به سطح میتسوبو
                entry_price = ob['mitsubo']
                # حد ضرر: کمی زیر کف بلوک سفارش
                stop_loss = ob['low'] * 0.995
                # حد سود: براساس ساختار بازار (فاصله تا بلوک بعدی یا شکست ساختاری بعدی)
                risk = entry_price - stop_loss
                take_profit1 = entry_price + risk  # 1:1
                take_profit2 = entry_price + risk * 2  # 1:2
                take_profit3 = entry_price + risk * 3  # 1:3

                entry_points.append({
                    'type': 'conditional_long',
                    'index': ob['index'],
                    'price': entry_price,
                    'stop_loss': stop_loss,
                    'take_profit1': take_profit1,
                    'take_profit2': take_profit2,
                    'take_profit3': take_profit3,
                    'condition': f'قیمت به {entry_price:.6f} یا کمتر برسد',
                    'strength': 85
                })

        # برای بلوک‌های سفارش نزولی
        for ob in orderblock_bearish:
            # اگر قیمت در نزدیکی بلوک سفارش نزولی است
            if current_price >= ob['low'] * 0.95 and current_price <= ob['high'] * 1.05:
                # ورود: نزدیک به سطح میتسوبو
                entry_price = ob['mitsubo']
                # حد ضرر: کمی بالای سقف بلوک سفارش
                stop_loss = ob['high'] * 1.005
                # حد سود: براساس ساختار بازار (فاصله تا بلوک بعدی یا شکست ساختاری بعدی)
                risk = stop_loss - entry_price
                take_profit1 = entry_price - risk  # 1:1
                take_profit2 = entry_price - risk * 2  # 1:2
                take_profit3 = entry_price - risk * 3  # 1:3

                entry_points.append({
                    'type': 'conditional_short',
                    'index': ob['index'],
                    'price': entry_price,
                    'stop_loss': stop_loss,
                    'take_profit1': take_profit1,
                    'take_profit2': take_profit2,
                    'take_profit3': take_profit3,
                    'condition': f'قیمت به {entry_price:.6f} یا بیشتر برسد',
                    'strength': 85
                })

        # نتایج نهایی
        result = {
            'market_structure': {
                'status': msb_status,
                'strength': msb_strength,
                'trend_score': trend_score,
                'higher_highs': hh_count,
                'lower_highs': lh_count,
                'higher_lows': hl_count,
                'lower_lows': ll_count
            },
            'msb_bullish': msb_bullish,
            'msb_bearish': msb_bearish,
            'orderblock_bullish': orderblock_bullish,
            'orderblock_bearish': orderblock_bearish,
            'entry_points': entry_points,
            'dataframe': df_ms
        }

        return result

    def analyze_buying_selling_pressure(self, df, volume_threshold=1.5, price_threshold=1.0):
        """
        تحلیل فشار خرید یا فروش با استفاده از الگوهای حجم و قیمت

        Args:
            df: دیتافریم داده‌ها
            volume_threshold: آستانه افزایش حجم نسبت به میانگین
            price_threshold: آستانه تغییرات قیمت

        Returns:
            نتایج تحلیل فشار خرید/فروش
        """
        # کپی از دیتافریم
        df_pressure = df.copy()

        # محاسبه میانگین حجم برای مقایسه
        df_pressure['volume_sma20'] = df_pressure['volume'].rolling(window=20).mean()
        df_pressure['volume_ratio'] = df_pressure['volume'] / df_pressure['volume_sma20']

        # محاسبه تغییرات قیمت
        df_pressure['price_change_pct'] = df_pressure['close'].pct_change() * 100

        # شناسایی کندل‌های با فشار خرید
        df_pressure['buying_pressure'] = ((df_pressure['close'] > df_pressure['open']) &
                                          (df_pressure['volume_ratio'] > volume_threshold) &
                                          (df_pressure['price_change_pct'] > price_threshold))

        # شناسایی کندل‌های با فشار فروش
        df_pressure['selling_pressure'] = ((df_pressure['close'] < df_pressure['open']) &
                                           (df_pressure['volume_ratio'] > volume_threshold) &
                                           (df_pressure['price_change_pct'] < -price_threshold))

        # شناسایی الگوهای فشار خرید متوالی
        consecutive_buying = 0
        buying_patterns = []

        for i in range(1, len(df_pressure)):
            if df_pressure['buying_pressure'].iloc[i]:
                consecutive_buying += 1

                # اگر حداقل دو کندل فشار خرید متوالی داشتیم، الگو را ثبت می‌کنیم
                if consecutive_buying >= 2:
                    buying_patterns.append({
                        'index': i,
                        'strength': min(100, consecutive_buying * 20 + df_pressure['volume_ratio'].iloc[i] * 10),
                        'volume_ratio': df_pressure['volume_ratio'].iloc[i]
                    })
            else:
                consecutive_buying = 0

        # شناسایی الگوهای فشار فروش متوالی
        consecutive_selling = 0
        selling_patterns = []

        for i in range(1, len(df_pressure)):
            if df_pressure['selling_pressure'].iloc[i]:
                consecutive_selling += 1

                # اگر حداقل دو کندل فشار فروش متوالی داشتیم، الگو را ثبت می‌کنیم
                if consecutive_selling >= 2:
                    selling_patterns.append({
                        'index': i,
                        'strength': min(100, consecutive_selling * 20 + df_pressure['volume_ratio'].iloc[i] * 10),
                        'volume_ratio': df_pressure['volume_ratio'].iloc[i]
                    })
            else:
                consecutive_selling = 0

        # آنالیز فشار در 10 کندل آخر
        recent_candles = 10
        recent_buying = df_pressure['buying_pressure'].iloc[-recent_candles:].sum()
        recent_selling = df_pressure['selling_pressure'].iloc[-recent_candles:].sum()

        # تعیین وضعیت کلی فشار بازار
        pressure_status = 'خنثی'
        pressure_strength = 50

        if recent_buying > recent_selling:
            # فشار خرید بیشتر است
            pressure_status = 'خرید'
            pressure_strength = 50 + (recent_buying - recent_selling) * 10

            # بررسی یک الگوی قوی‌تر: فشار خرید بعد از یک روند نزولی
            if df_pressure['close'].iloc[-15:-5].mean() > df_pressure['close'].iloc[-5:].mean():
                pressure_status = 'خرید پس از اصلاح'
                pressure_strength += 10

        elif recent_selling > recent_buying:
            # فشار فروش بیشتر است
            pressure_status = 'فروش'
            pressure_strength = 50 + (recent_selling - recent_buying) * 10

            # بررسی یک الگوی قوی‌تر: فشار فروش بعد از یک روند صعودی
            if df_pressure['close'].iloc[-15:-5].mean() < df_pressure['close'].iloc[-5:].mean():
                pressure_status = 'فروش پس از صعود'
                pressure_strength += 10

        # نتایج نهایی
        result = {
            'pressure_status': pressure_status,
            'pressure_strength': min(100, pressure_strength),
            'recent_buying': recent_buying,
            'recent_selling': recent_selling,
            'buying_patterns': buying_patterns,
            'selling_patterns': selling_patterns,
            'dataframe': df_pressure
        }

        return result

    def detect_whale_activity(self, df, volume_multiplier=3.0, price_impact=1.5):
        """
        تشخیص فعالیت نهنگ‌ها بدون نیاز به API خارجی با تحلیل حجم و قیمت

        Args:
            df: دیتافریم داده‌ها
            volume_multiplier: ضریب افزایش حجم برای تشخیص فعالیت نهنگ‌ها
            price_impact: حداقل تأثیر قیمتی برای تشخیص فعالیت نهنگ‌ها

        Returns:
            نتایج تشخیص فعالیت نهنگ‌ها
        """
        # کپی از دیتافریم
        df_whale = df.copy()

        # اضافه کردن ستون‌های مورد نیاز که ممکن است در دیتافریم اصلی نباشند
        # محاسبه تغییرات قیمت به صورت درصد
        df_whale['price_change_pct'] = df_whale['close'].pct_change() * 100

        # محاسبه میانگین‌های متحرک حجم
        df_whale['volume_sma20'] = df_whale['volume'].rolling(window=20).mean()
        df_whale['volume_sma50'] = df_whale['volume'].rolling(window=50).mean()

        # محاسبه انحراف معیار حجم و قیمت
        df_whale['volume_std20'] = df_whale['volume'].rolling(window=20).std()
        df_whale['close_std20'] = df_whale['close'].rolling(window=20).std()

        # شناسایی کندل‌های با حجم غیرعادی (نهنگ‌ها)
        df_whale['abnormal_volume'] = df_whale['volume'] > (
                df_whale['volume_sma50'] + df_whale['volume_std20'] * volume_multiplier)

        # شناسایی تأثیر قیمتی مهم
        df_whale['price_impact'] = np.abs(df_whale['close'] - df_whale['open']) / df_whale['open'] * 100
        df_whale['significant_impact'] = df_whale['price_impact'] > price_impact

        # شناسایی فعالیت نهنگ‌های خریدار
        df_whale['whale_buying'] = ((df_whale['abnormal_volume']) &
                                    (df_whale['significant_impact']) &
                                    (df_whale['close'] > df_whale['open']))

        # شناسایی فعالیت نهنگ‌های فروشنده
        df_whale['whale_selling'] = ((df_whale['abnormal_volume']) &
                                     (df_whale['significant_impact']) &
                                     (df_whale['close'] < df_whale['open']))

        # تحلیل تجمعی فعالیت نهنگ‌ها در بازه اخیر
        recent_whale_buying = []
        recent_whale_selling = []

        for i in range(max(50, len(df_whale) - 100), len(df_whale)):
            if df_whale['whale_buying'].iloc[i]:
                recent_whale_buying.append({
                    'index': i,
                    'date': df_whale.index[i],
                    'volume': df_whale['volume'].iloc[i],
                    'volume_ratio': df_whale['volume'].iloc[i] / df_whale['volume_sma50'].iloc[i],
                    'price_impact': df_whale['price_impact'].iloc[i]
                })

            if df_whale['whale_selling'].iloc[i]:
                recent_whale_selling.append({
                    'index': i,
                    'date': df_whale.index[i],
                    'volume': df_whale['volume'].iloc[i],
                    'volume_ratio': df_whale['volume'].iloc[i] / df_whale['volume_sma50'].iloc[i],
                    'price_impact': df_whale['price_impact'].iloc[i]
                })

        # تحلیل الگوی انباشت یا توزیع
        accumulation_pattern = False
        distribution_pattern = False
        consolidation_after_whale = False

        # انباشت: چندین فعالیت خرید بزرگ در بازه کوتاه با قیمت‌های نسبتاً ثابت
        if len(recent_whale_buying) >= 2:
            recent_buying_indices = [item['index'] for item in recent_whale_buying[-3:]]
            if max(recent_buying_indices) - min(recent_buying_indices) < 20:  # در 20 کندل اخیر
                price_range = df_whale['close'].iloc[min(recent_buying_indices):max(recent_buying_indices) + 1].max() / \
                              df_whale['close'].iloc[
                              min(recent_buying_indices):max(recent_buying_indices) + 1].min() - 1

                if price_range < 0.05:  # محدوده قیمت کمتر از 5%
                    accumulation_pattern = True

        # توزیع: چندین فعالیت فروش بزرگ در بازه کوتاه با قیمت‌های نسبتاً ثابت
        if len(recent_whale_selling) >= 2:
            recent_selling_indices = [item['index'] for item in recent_whale_selling[-3:]]
            if max(recent_selling_indices) - min(recent_selling_indices) < 20:  # در 20 کندل اخیر
                price_range = df_whale['close'].iloc[
                              min(recent_selling_indices):max(recent_selling_indices) + 1].max() / \
                              df_whale['close'].iloc[
                              min(recent_selling_indices):max(recent_selling_indices) + 1].min() - 1

                if price_range < 0.05:  # محدوده قیمت کمتر از 5%
                    distribution_pattern = True

        # بررسی تحکیم پس از فعالیت نهنگ‌ها
        if recent_whale_buying or recent_whale_selling:
            last_whale_idx = max([item['index'] for item in recent_whale_buying + recent_whale_selling])
            if len(df_whale) - last_whale_idx > 5:  # حداقل 5 کندل بعد از آخرین فعالیت
                recent_volatility = df_whale['close_std20'].iloc[-1] / df_whale['close'].iloc[-1]
                if recent_volatility < 0.02:  # نوسان کم پس از فعالیت نهنگ
                    consolidation_after_whale = True

        # تعیین وضعیت کلی فعالیت نهنگ‌ها
        whale_status = 'بدون فعالیت نهنگ قابل توجه'
        whale_strength = 0
        whale_signal = 'خنثی'

        if len(recent_whale_buying) > len(recent_whale_selling) and len(recent_whale_buying) > 0:
            whale_status = 'فعالیت خرید نهنگ‌ها'
            whale_strength = min(100, 50 + len(recent_whale_buying) * 10)
            whale_signal = 'خرید'

            if accumulation_pattern:
                whale_status += ' - الگوی انباشت'
                whale_strength += 20

            if consolidation_after_whale:
                whale_status += ' - دوره تحکیم پس از خرید'
                whale_strength += 10

        elif len(recent_whale_selling) > len(recent_whale_buying) and len(recent_whale_selling) > 0:
            whale_status = 'فعالیت فروش نهنگ‌ها'
            whale_strength = min(100, 50 + len(recent_whale_selling) * 10)
            whale_signal = 'فروش'

            if distribution_pattern:
                whale_status += ' - الگوی توزیع'
                whale_strength += 20

            if consolidation_after_whale:
                whale_status += ' - دوره تحکیم پس از فروش'
                whale_strength += 10

        # بررسی لیکوئیدیشن (با تحلیل تغییرات قیمت ناگهانی)
        liquidation_events = []

        for i in range(20, len(df_whale)):
            # تغییر قیمت ناگهانی بیش از 3% در یک کندل با حجم بالا
            if abs(df_whale['price_change_pct'].iloc[i]) > 3 and df_whale['volume'].iloc[i] > \
                    df_whale['volume_sma20'].iloc[i] * 2:
                direction = 'Long Liquidation' if df_whale['close'].iloc[i] < df_whale['open'].iloc[
                    i] else 'Short Liquidation'
                liquidation_events.append({
                    'index': i,
                    'date': df_whale.index[i],
                    'direction': direction,
                    'price_change': df_whale['price_change_pct'].iloc[i],
                    'volume_ratio': df_whale['volume'].iloc[i] / df_whale['volume_sma20'].iloc[i]
                })

        # نتایج نهایی
        result = {
            'whale_status': whale_status,
            'whale_strength': min(100, whale_strength),
            'whale_signal': whale_signal,
            'recent_whale_buying': recent_whale_buying,
            'recent_whale_selling': recent_whale_selling,
            'accumulation_pattern': accumulation_pattern,
            'distribution_pattern': distribution_pattern,
            'consolidation_after_whale': consolidation_after_whale,
            'liquidation_events': liquidation_events,
            'dataframe': df_whale
        }

        return result

    def analyze_ema_strategy(self, df, short_period=8, mid_period=21, long_period=50, trend_period=200):
        """
        استراتژی EMA برای تشخیص روند و سیگنال‌دهی

        Args:
            df: دیتافریم داده‌ها
            short_period: دوره EMA کوتاه‌مدت
            mid_period: دوره EMA میان‌مدت
            long_period: دوره EMA بلندمدت
            trend_period: دوره EMA برای تشخیص روند اصلی

        Returns:
            نتایج استراتژی EMA
        """
        # محاسبه میانگین‌های متحرک نمایی (EMA)
        df_ema = df.copy()

        df_ema[f'ema_{short_period}'] = ta.trend.ema_indicator(df_ema['close'], window=short_period)
        df_ema[f'ema_{mid_period}'] = ta.trend.ema_indicator(df_ema['close'], window=mid_period)
        df_ema[f'ema_{long_period}'] = ta.trend.ema_indicator(df_ema['close'], window=long_period)
        df_ema[f'ema_{trend_period}'] = ta.trend.ema_indicator(df_ema['close'], window=trend_period)

        # شناسایی کراس‌اوور‌ها
        # کراس صعودی: EMA کوتاه‌مدت بالای EMA میان‌مدت
        df_ema['ema_short_above_mid'] = df_ema[f'ema_{short_period}'] > df_ema[f'ema_{mid_period}']
        df_ema['ema_bullish_cross'] = np.logical_and(
            df_ema['ema_short_above_mid'],
            np.logical_not(df_ema['ema_short_above_mid'].shift(1))
        )

        # کراس نزولی: EMA کوتاه‌مدت زیر EMA میان‌مدت
        df_ema['ema_short_below_mid'] = df_ema[f'ema_{short_period}'] < df_ema[f'ema_{mid_period}']
        df_ema['ema_bearish_cross'] = np.logical_and(
            df_ema['ema_short_below_mid'],
            np.logical_not(df_ema['ema_short_below_mid'].shift(1))
        )

        # شناسایی فاصله میانگین‌ها به عنوان شاخص قدرت روند
        df_ema['ema_spread'] = (df_ema[f'ema_{short_period}'] - df_ema[f'ema_{long_period}']) / df_ema[
            f'ema_{long_period}'] * 100

        # تشخیص همسویی EMAها برای روند قدرتمند
        df_ema['ema_aligned_bullish'] = (df_ema[f'ema_{short_period}'] > df_ema[f'ema_{mid_period}']) & \
                                        (df_ema[f'ema_{mid_period}'] > df_ema[f'ema_{long_period}']) & \
                                        (df_ema['close'] > df_ema[f'ema_{short_period}'])

        df_ema['ema_aligned_bearish'] = (df_ema[f'ema_{short_period}'] < df_ema[f'ema_{mid_period}']) & \
                                        (df_ema[f'ema_{mid_period}'] < df_ema[f'ema_{long_period}']) & \
                                        (df_ema['close'] < df_ema[f'ema_{short_period}'])

        # شناسایی روند بلندمدت (باتوجه به EMA بلندمدت)
        df_ema['long_term_uptrend'] = df_ema['close'] > df_ema[f'ema_{trend_period}']

        # جمع‌آوری سیگنال‌ها
        bullish_signals = []
        bearish_signals = []

        for i in range(max(trend_period, mid_period, long_period) + 10, len(df_ema)):
            # سیگنال‌های صعودی
            if df_ema['ema_bullish_cross'].iloc[i]:
                # سیگنال صعودی قوی: کراس صعودی در روند صعودی
                if df_ema['long_term_uptrend'].iloc[i]:
                    strength = 80
                    description = 'کراس صعودی قوی در روند اصلی صعودی'
                else:
                    strength = 60
                    description = 'کراس صعودی در روند نزولی (احتمال ریباند)'

                # افزایش قدرت سیگنال در صورت همسویی همه EMAها
                if df_ema['ema_aligned_bullish'].iloc[i:i + 3].any():
                    strength += 10
                    description += ' - همسویی میانگین‌های متحرک'

                # سیگنال بازگشت روند: قیمت از زیر EMA بلندمدت به بالای آن حرکت می‌کند
                if not df_ema['long_term_uptrend'].iloc[i - 1] and df_ema['long_term_uptrend'].iloc[i]:
                    strength += 15
                    description += ' - بازگشت روند بلندمدت به صعودی'

                bullish_signals.append({
                    'index': i,
                    'date': df_ema.index[i],
                    'price': df_ema['close'].iloc[i],
                    'strength': min(100, strength),
                    'description': description
                })

            # سیگنال‌های نزولی
            if df_ema['ema_bearish_cross'].iloc[i]:
                # سیگنال نزولی قوی: کراس نزولی در روند نزولی
                if not df_ema['long_term_uptrend'].iloc[i]:
                    strength = 80
                    description = 'کراس نزولی قوی در روند اصلی نزولی'
                else:
                    strength = 60
                    description = 'کراس نزولی در روند صعودی (احتمال اصلاح)'

                # افزایش قدرت سیگنال در صورت همسویی همه EMAها
                if df_ema['ema_aligned_bearish'].iloc[i:i + 3].any():
                    strength += 10
                    description += ' - همسویی میانگین‌های متحرک'

                # سیگنال بازگشت روند: قیمت از بالای EMA بلندمدت به زیر آن حرکت می‌کند
                if df_ema['long_term_uptrend'].iloc[i - 1] and not df_ema['long_term_uptrend'].iloc[i]:
                    strength += 15
                    description += ' - بازگشت روند بلندمدت به نزولی'

                bearish_signals.append({
                    'index': i,
                    'date': df_ema.index[i],
                    'price': df_ema['close'].iloc[i],
                    'strength': min(100, strength),
                    'description': description
                })

        # تحلیل وضعیت فعلی
        current_ema_status = 'خنثی'
        current_ema_strength = 50

        # بررسی روند فعلی
        if df_ema['long_term_uptrend'].iloc[-1]:
            base_trend = 'صعودی'
        else:
            base_trend = 'نزولی'

        # بررسی همسویی EMAها
        if df_ema['ema_aligned_bullish'].iloc[-1]:
            current_ema_status = f'صعودی قوی (روند {base_trend})'
            current_ema_strength = 85
        elif df_ema['ema_aligned_bearish'].iloc[-1]:
            current_ema_status = f'نزولی قوی (روند {base_trend})'
            current_ema_strength = 15
        elif df_ema['ema_short_above_mid'].iloc[-1]:
            if df_ema['long_term_uptrend'].iloc[-1]:
                current_ema_status = f'صعودی (روند {base_trend})'
                current_ema_strength = 70
            else:
                current_ema_status = f'صعودی کوتاه‌مدت در روند {base_trend}'
                current_ema_strength = 60
        elif df_ema['ema_short_below_mid'].iloc[-1]:
            if not df_ema['long_term_uptrend'].iloc[-1]:
                current_ema_status = f'نزولی (روند {base_trend})'
                current_ema_strength = 30
            else:
                current_ema_status = f'نزولی کوتاه‌مدت در روند {base_trend}'
                current_ema_strength = 40

        # استراتژی ورود و خروج
        entry_points = []
        exit_points = []

        # استراتژی ورود: قیمت از زیر EMA میانی به بالای آن می‌رود در روند صعودی
        for i in range(max(trend_period, mid_period, long_period) + 10, len(df_ema) - 1):
            # ورود صعودی
            if (df_ema['close'].iloc[i - 1] < df_ema[f'ema_{mid_period}'].iloc[i - 1] and
                    df_ema['close'].iloc[i] > df_ema[f'ema_{mid_period}'].iloc[i] and
                    df_ema['long_term_uptrend'].iloc[i]):
                entry_points.append({
                    'index': i,
                    'type': 'long',
                    'price': df_ema['close'].iloc[i],
                    'stop_loss': df_ema[f'ema_{long_period}'].iloc[i],
                    'take_profit': df_ema['close'].iloc[i] + (
                            df_ema['close'].iloc[i] - df_ema[f'ema_{long_period}'].iloc[i]) * 2
                })

            # ورود نزولی
            if (df_ema['close'].iloc[i - 1] > df_ema[f'ema_{mid_period}'].iloc[i - 1] and
                    df_ema['close'].iloc[i] < df_ema[f'ema_{mid_period}'].iloc[i] and
                    not df_ema['long_term_uptrend'].iloc[i]):
                entry_points.append({
                    'index': i,
                    'type': 'short',
                    'price': df_ema['close'].iloc[i],
                    'stop_loss': df_ema[f'ema_{long_period}'].iloc[i],
                    'take_profit': df_ema['close'].iloc[i] - (
                            df_ema[f'ema_{long_period}'].iloc[i] - df_ema['close'].iloc[i]) * 2
                })

            # خروج از موقعیت صعودی: قیمت زیر EMA کوتاه‌مدت
            if (df_ema['close'].iloc[i - 1] > df_ema[f'ema_{short_period}'].iloc[i - 1] and
                    df_ema['close'].iloc[i] < df_ema[f'ema_{short_period}'].iloc[i]):
                exit_points.append({
                    'index': i,
                    'type': 'exit_long',
                    'price': df_ema['close'].iloc[i]
                })

            # خروج از موقعیت نزولی: قیمت بالای EMA کوتاه‌مدت
            if (df_ema['close'].iloc[i - 1] < df_ema[f'ema_{short_period}'].iloc[i - 1] and
                    df_ema['close'].iloc[i] > df_ema[f'ema_{short_period}'].iloc[i]):
                exit_points.append({
                    'index': i,
                    'type': 'exit_short',
                    'price': df_ema['close'].iloc[i]
                })

        # آماده‌سازی سیگنال آخر
        latest_signal = None
        latest_signal_type = 'خنثی'
        latest_signal_strength = 50

        if bullish_signals and (not bearish_signals or bullish_signals[-1]['index'] > bearish_signals[-1]['index']):
            latest_signal = bullish_signals[-1]
            latest_signal_type = 'خرید'
            latest_signal_strength = latest_signal['strength']

            # اگر سیگنال خرید مربوط به کندل‌های اخیر باشد، قدرت سیگنال را افزایش می‌دهیم
            if len(df_ema) - latest_signal['index'] <= 3:
                latest_signal_strength = min(100, latest_signal_strength + 10)

        elif bearish_signals and (not bullish_signals or bearish_signals[-1]['index'] > bullish_signals[-1]['index']):
            latest_signal = bearish_signals[-1]
            latest_signal_type = 'فروش'
            latest_signal_strength = latest_signal['strength']

            # اگر سیگنال فروش مربوط به کندل‌های اخیر باشد، قدرت سیگنال را افزایش می‌دهیم
            if len(df_ema) - latest_signal['index'] <= 3:
                latest_signal_strength = min(100, latest_signal_strength + 10)

        # نتایج نهایی
        result = {
            'current_status': current_ema_status,
            'current_strength': current_ema_strength,
            'latest_signal_type': latest_signal_type,
            'latest_signal_strength': latest_signal_strength,
            'latest_signal': latest_signal,
            'bullish_signals': bullish_signals,
            'bearish_signals': bearish_signals,
            'entry_points': entry_points,
            'exit_points': exit_points,
            'dataframe': df_ema
        }

        return result

    def analyze_rsi_strategy(self, df, period=14, overbought=70, oversold=30):
        """
        استراتژی شاخص قدرت نسبی (RSI) با تشخیص بهترین نقاط ورود

        Args:
            df: دیتافریم داده‌ها
            period: دوره محاسبه RSI
            overbought: سطح اشباع خرید
            oversold: سطح اشباع فروش

        Returns:
            نتایج استراتژی RSI
        """
        # محاسبه RSI
        df_rsi = df.copy()
        df_rsi['rsi'] = ta.momentum.rsi(df_rsi['close'], window=period)

        # محاسبه میانگین متحرک ساده RSI برای تعیین روند RSI
        df_rsi['rsi_sma5'] = df_rsi['rsi'].rolling(window=5).mean()

        # شناسایی مناطق اشباع خرید و فروش
        df_rsi['overbought'] = df_rsi['rsi'] > overbought
        df_rsi['oversold'] = df_rsi['rsi'] < oversold

        # شناسایی خروج از مناطق اشباع
        df_rsi['exit_overbought'] = (df_rsi['rsi'] < overbought) & (df_rsi['rsi'].shift(1) >= overbought)
        df_rsi['exit_oversold'] = (df_rsi['rsi'] > oversold) & (df_rsi['rsi'].shift(1) <= oversold)

        # شناسایی روند قیمت (میانگین متحرک 50)
        df_rsi['sma50'] = df_rsi['close'].rolling(window=50).mean()
        df_rsi['price_uptrend'] = df_rsi['close'] > df_rsi['sma50']

        # شناسایی واگرایی‌های ساده
        df_rsi['rsi_higher_high'] = (df_rsi['rsi'] > df_rsi['rsi'].shift(1)) & (
                df_rsi['rsi'].shift(1) > df_rsi['rsi'].shift(2))
        df_rsi['price_higher_high'] = (df_rsi['high'] > df_rsi['high'].shift(1)) & (
                df_rsi['high'].shift(1) > df_rsi['high'].shift(2))

        df_rsi['rsi_lower_low'] = (df_rsi['rsi'] < df_rsi['rsi'].shift(1)) & (
                df_rsi['rsi'].shift(1) < df_rsi['rsi'].shift(2))
        df_rsi['price_lower_low'] = (df_rsi['low'] < df_rsi['low'].shift(1)) & (
                df_rsi['low'].shift(1) < df_rsi['low'].shift(2))

        # شناسایی سیگنال‌های خرید RSI
        bullish_signals = []

        for i in range(period + 10, len(df_rsi)):
            # نوع اول: خروج از منطقه اشباع فروش
            if df_rsi['exit_oversold'].iloc[i]:
                strength = 70
                description = 'خروج از منطقه اشباع فروش'

                # تقویت سیگنال اگر در روند صعودی قیمت باشیم
                if df_rsi['price_uptrend'].iloc[i]:
                    strength += 15
                    description += ' در روند صعودی قیمت'

                # شناسایی بازگشت‌های قوی‌تر از سطوح بسیار پایین
                if df_rsi['rsi'].iloc[i - 1] < 20:
                    strength += 10
                    description += ' (RSI بسیار پایین)'

                bullish_signals.append({
                    'index': i,
                    'date': df_rsi.index[i],
                    'price': df_rsi['close'].iloc[i],
                    'rsi': df_rsi['rsi'].iloc[i],
                    'strength': min(100, strength),
                    'description': description,
                    'type': 'exit_oversold'
                })

            # نوع دوم: RSI در حال صعود از منطقه میانی و قیمت در روند صعودی
            if (50 > df_rsi['rsi'].iloc[i - 1] > 45 and df_rsi['rsi'].iloc[i] > 50 and
                    df_rsi['rsi'].iloc[i] > df_rsi['rsi'].iloc[i - 1] and df_rsi['price_uptrend'].iloc[i]):
                bullish_signals.append({
                    'index': i,
                    'date': df_rsi.index[i],
                    'price': df_rsi['close'].iloc[i],
                    'rsi': df_rsi['rsi'].iloc[i],
                    'strength': 65,
                    'description': 'شکست RSI از سطح میانی در روند صعودی',
                    'type': 'rsi_mid_break'
                })

        # شناسایی سیگنال‌های فروش RSI
        bearish_signals = []

        for i in range(period + 10, len(df_rsi)):
            # نوع اول: خروج از منطقه اشباع خرید
            if df_rsi['exit_overbought'].iloc[i]:
                strength = 70
                description = 'خروج از منطقه اشباع خرید'

                # تقویت سیگنال اگر در روند نزولی قیمت باشیم
                if not df_rsi['price_uptrend'].iloc[i]:
                    strength += 15
                    description += ' در روند نزولی قیمت'

                # شناسایی بازگشت‌های قوی‌تر از سطوح بسیار بالا
                if df_rsi['rsi'].iloc[i - 1] > 80:
                    strength += 10
                    description += ' (RSI بسیار بالا)'

                bearish_signals.append({
                    'index': i,
                    'date': df_rsi.index[i],
                    'price': df_rsi['close'].iloc[i],
                    'rsi': df_rsi['rsi'].iloc[i],
                    'strength': min(100, strength),
                    'description': description,
                    'type': 'exit_overbought'
                })

            # نوع دوم: RSI در حال نزول از منطقه میانی و قیمت در روند نزولی
            if (50 < df_rsi['rsi'].iloc[i - 1] < 55 and df_rsi['rsi'].iloc[i] < 50 and
                    df_rsi['rsi'].iloc[i] < df_rsi['rsi'].iloc[i - 1] and not df_rsi['price_uptrend'].iloc[i]):
                bearish_signals.append({
                    'index': i,
                    'date': df_rsi.index[i],
                    'price': df_rsi['close'].iloc[i],
                    'rsi': df_rsi['rsi'].iloc[i],
                    'strength': 65,
                    'description': 'شکست RSI از سطح میانی در روند نزولی',
                    'type': 'rsi_mid_break'
                })

        # شناسایی واگرایی‌های پیشرفته
        bullish_divergences = []
        bearish_divergences = []

        # بررسی دوره‌های زمانی ۳۰ کندلی برای الگوهای واگرایی
        for i in range(period + 30, len(df_rsi)):
            window = 30

            # واگرایی مثبت: قیمت‌های پایین‌تر اما RSI بالاتر (نشانه خرید)
            if (df_rsi['low'].iloc[i] < df_rsi['low'].iloc[i - window:i].min() and
                    df_rsi['rsi'].iloc[i] > df_rsi['rsi'].iloc[i - window:i].min()):

                # محاسبه میزان واگرایی
                price_change = (df_rsi['low'].iloc[i] / df_rsi['low'].iloc[i - window:i].min() - 1) * 100
                rsi_change = (df_rsi['rsi'].iloc[i] / df_rsi['rsi'].iloc[i - window:i].min() - 1) * 100

                # اگر واگرایی معنادار باشد، ثبت می‌کنیم
                if price_change < -1 and rsi_change > 5:
                    strength = 75 + min(25, abs(rsi_change - price_change) / 2)

                    bullish_divergences.append({
                        'index': i,
                        'date': df_rsi.index[i],
                        'price': df_rsi['close'].iloc[i],
                        'rsi': df_rsi['rsi'].iloc[i],
                        'strength': min(100, strength),
                        'description': f'واگرایی مثبت RSI (قیمت پایین‌تر، RSI بالاتر)',
                        'price_change': price_change,
                        'rsi_change': rsi_change
                    })

            # واگرایی منفی: قیمت‌های بالاتر اما RSI پایین‌تر (نشانه فروش)
            if (df_rsi['high'].iloc[i] > df_rsi['high'].iloc[i - window:i].max() and
                    df_rsi['rsi'].iloc[i] < df_rsi['rsi'].iloc[i - window:i].max()):

                # محاسبه میزان واگرایی
                price_change = (df_rsi['high'].iloc[i] / df_rsi['high'].iloc[i - window:i].max() - 1) * 100
                rsi_change = (df_rsi['rsi'].iloc[i] / df_rsi['rsi'].iloc[i - window:i].max() - 1) * 100

                # اگر واگرایی معنادار باشد، ثبت می‌کنیم
                if price_change > 1 and rsi_change < -5:
                    strength = 75 + min(25, abs(rsi_change - price_change) / 2)

                    bearish_divergences.append({
                        'index': i,
                        'date': df_rsi.index[i],
                        'price': df_rsi['close'].iloc[i],
                        'rsi': df_rsi['rsi'].iloc[i],
                        'strength': min(100, strength),
                        'description': f'واگرایی منفی RSI (قیمت بالاتر، RSI پایین‌تر)',
                        'price_change': price_change,
                        'rsi_change': rsi_change
                    })

        # تحلیل وضعیت فعلی RSI
        current_rsi = df_rsi['rsi'].iloc[-1]
        current_rsi_status = 'خنثی'
        current_signal_type = 'خنثی'
        current_signal_strength = 50

        # بررسی وضعیت اخیر
        if current_rsi > overbought:
            current_rsi_status = 'اشباع خرید'
            current_signal_type = 'فروش'
            current_signal_strength = 70

            # اگر RSI بسیار بالاست، سیگنال قوی‌تر است
            if current_rsi > 80:
                current_signal_strength = 85
                current_rsi_status = 'اشباع خرید شدید'

        elif current_rsi < oversold:
            current_rsi_status = 'اشباع فروش'
            current_signal_type = 'خرید'
            current_signal_strength = 70

            # اگر RSI بسیار پایین است، سیگنال قوی‌تر است
            if current_rsi < 20:
                current_signal_strength = 85
                current_rsi_status = 'اشباع فروش شدید'

        else:
            # در منطقه میانی، به روند RSI نگاه می‌کنیم
            rsi_trend = df_rsi['rsi'].iloc[-5:].mean() - df_rsi['rsi'].iloc[-10:-5].mean()

            if rsi_trend > 3:
                current_rsi_status = 'صعودی'
                current_signal_type = 'خرید'
                current_signal_strength = 60
            elif rsi_trend < -3:
                current_rsi_status = 'نزولی'
                current_signal_type = 'فروش'
                current_signal_strength = 60

        # بررسی سیگنال‌های اخیر
        recent_bullish = [s for s in bullish_signals if len(df_rsi) - s['index'] <= 3]
        recent_bearish = [s for s in bearish_signals if len(df_rsi) - s['index'] <= 3]

        recent_bull_div = [s for s in bullish_divergences if len(df_rsi) - s['index'] <= 5]
        recent_bear_div = [s for s in bearish_divergences if len(df_rsi) - s['index'] <= 5]

        if recent_bullish:
            current_signal_type = 'خرید'
            current_signal_strength = max([s['strength'] for s in recent_bullish])
        elif recent_bearish:
            current_signal_type = 'فروش'
            current_signal_strength = max([s['strength'] for s in recent_bearish])

        # واگرایی‌ها اولویت بالاتری دارند
        if recent_bull_div:
            current_signal_type = 'خرید'
            current_signal_strength = max([s['strength'] for s in recent_bull_div])
        elif recent_bear_div:
            current_signal_type = 'فروش'
            current_signal_strength = max([s['strength'] for s in recent_bear_div])

        # نتایج نهایی
        result = {
            'current_rsi': current_rsi,
            'current_status': current_rsi_status,
            'current_signal_type': current_signal_type,
            'current_signal_strength': current_signal_strength,
            'bullish_signals': bullish_signals,
            'bearish_signals': bearish_signals,
            'bullish_divergences': bullish_divergences,
            'bearish_divergences': bearish_divergences,
            'dataframe': df_rsi
        }

        return result

    def detect_divergence(self, df, indicator='rsi', period=14, window_size=20):
        """
        تشخیص واگرایی‌های مخفی و آشکار بین قیمت و اندیکاتور

        Args:
            df: دیتافریم داده‌ها
            indicator: نوع اندیکاتور (rsi, macd, etc.)
            period: دوره محاسبه اندیکاتور
            window_size: اندازه پنجره برای جستجوی واگرایی

        Returns:
            نتایج تشخیص واگرایی
        """
        # کپی از دیتافریم
        df_div = df.copy()

        # محاسبه اندیکاتور مورد نظر
        if indicator == 'rsi':
            df_div['indicator'] = ta.momentum.rsi(df_div['close'], window=period)
        elif indicator == 'macd':
            df_div['indicator'] = ta.trend.macd(df_div['close'], window_slow=26, window_fast=12)
        elif indicator == 'stoch':
            df_div['indicator'] = ta.momentum.stoch(df_div['high'], df_div['low'], df_div['close'],
                                                    window=5, smooth_window=3)
        else:
            df_div['indicator'] = ta.momentum.rsi(df_div['close'], window=period)

        # یافتن نقاط اوج و حضیض محلی برای قیمت
        df_div['price_high'] = df_div['close'].rolling(window=window_size, center=True).apply(
            lambda x: 1 if x.iloc[window_size // 2] == max(x) else 0, raw=False)

        df_div['price_low'] = df_div['close'].rolling(window=window_size, center=True).apply(
            lambda x: 1 if x.iloc[window_size // 2] == min(x) else 0, raw=False)

        # یافتن نقاط اوج و حضیض محلی برای اندیکاتور
        df_div['ind_high'] = df_div['indicator'].rolling(window=window_size, center=True).apply(
            lambda x: 1 if x.iloc[window_size // 2] == max(x) else 0, raw=False)

        df_div['ind_low'] = df_div['indicator'].rolling(window=window_size, center=True).apply(
            lambda x: 1 if x.iloc[window_size // 2] == min(x) else 0, raw=False)

        # شناسایی واگرایی‌ها
        regular_bullish = []  # واگرایی صعودی عادی
        regular_bearish = []  # واگرایی نزولی عادی
        hidden_bullish = []  # واگرایی صعودی مخفی
        hidden_bearish = []  # واگرایی نزولی مخفی

        # لیست نقاط اوج و حضیض برای قیمت و اندیکاتور
        price_highs = []
        price_lows = []
        ind_highs = []
        ind_lows = []

        for i in range(window_size, len(df_div) - window_size):
            if df_div['price_high'].iloc[i] == 1:
                price_highs.append((i, df_div['close'].iloc[i]))
            if df_div['price_low'].iloc[i] == 1:
                price_lows.append((i, df_div['close'].iloc[i]))
            if df_div['ind_high'].iloc[i] == 1:
                ind_highs.append((i, df_div['indicator'].iloc[i]))
            if df_div['ind_low'].iloc[i] == 1:
                ind_lows.append((i, df_div['indicator'].iloc[i]))

        # شناسایی واگرایی صعودی عادی (Regular Bullish)
        # قیمت: کف‌های پایین‌تر، اندیکاتور: کف‌های بالاتر
        for i in range(1, len(price_lows)):
            for j in range(1, len(ind_lows)):
                # اگر تفاوت زمانی بین دو نقطه کمتر از حد مجاز باشد
                if abs(price_lows[i][0] - ind_lows[j][0]) <= window_size // 4:
                    # چک کردن واگرایی صعودی عادی
                    if (i > 0 and j > 0 and
                            price_lows[i][1] < price_lows[i - 1][1] and
                            ind_lows[j][1] > ind_lows[j - 1][1]):
                        # محاسبه قدرت واگرایی براساس میزان تفاوت
                        price_change = (price_lows[i][1] / price_lows[i - 1][1] - 1) * 100
                        ind_change = (ind_lows[j][1] / ind_lows[j - 1][1] - 1) * 100
                        strength = min(100, 70 + abs(ind_change - price_change))

                        regular_bullish.append({
                            'price_idx1': price_lows[i - 1][0],
                            'price_idx2': price_lows[i][0],
                            'ind_idx1': ind_lows[j - 1][0],
                            'ind_idx2': ind_lows[j][0],
                            'price1': price_lows[i - 1][1],
                            'price2': price_lows[i][1],
                            'ind1': ind_lows[j - 1][1],
                            'ind2': ind_lows[j][1],
                            'strength': strength,
                            'description': 'واگرایی صعودی عادی (کف‌های پایین‌تر قیمت، کف‌های بالاتر اندیکاتور)'
                        })

        # شناسایی واگرایی نزولی عادی (Regular Bearish)
        # قیمت: سقف‌های بالاتر، اندیکاتور: سقف‌های پایین‌تر
        for i in range(1, len(price_highs)):
            for j in range(1, len(ind_highs)):
                # اگر تفاوت زمانی بین دو نقطه کمتر از حد مجاز باشد
                if abs(price_highs[i][0] - ind_highs[j][0]) <= window_size // 4:
                    # چک کردن واگرایی نزولی عادی
                    if (i > 0 and j > 0 and
                            price_highs[i][1] > price_highs[i - 1][1] and
                            ind_highs[j][1] < ind_highs[j - 1][1]):
                        # محاسبه قدرت واگرایی براساس میزان تفاوت
                        price_change = (price_highs[i][1] / price_highs[i - 1][1] - 1) * 100
                        ind_change = (ind_highs[j][1] / ind_highs[j - 1][1] - 1) * 100
                        strength = min(100, 70 + abs(ind_change - price_change))

                        regular_bearish.append({
                            'price_idx1': price_highs[i - 1][0],
                            'price_idx2': price_highs[i][0],
                            'ind_idx1': ind_highs[j - 1][0],
                            'ind_idx2': ind_highs[j][0],
                            'price1': price_highs[i - 1][1],
                            'price2': price_highs[i][1],
                            'ind1': ind_highs[j - 1][1],
                            'ind2': ind_highs[j][1],
                            'strength': strength,
                            'description': 'واگرایی نزولی عادی (سقف‌های بالاتر قیمت، سقف‌های پایین‌تر اندیکاتور)'
                        })

        # شناسایی واگرایی صعودی مخفی (Hidden Bullish)
        # قیمت: کف‌های بالاتر، اندیکاتور: کف‌های پایین‌تر
        for i in range(1, len(price_lows)):
            for j in range(1, len(ind_lows)):
                # اگر تفاوت زمانی بین دو نقطه کمتر از حد مجاز باشد
                if abs(price_lows[i][0] - ind_lows[j][0]) <= window_size // 4:
                    # چک کردن واگرایی صعودی مخفی
                    if (i > 0 and j > 0 and
                            price_lows[i][1] > price_lows[i - 1][1] and
                            ind_lows[j][1] < ind_lows[j - 1][1]):
                        # محاسبه قدرت واگرایی براساس میزان تفاوت
                        price_change = (price_lows[i][1] / price_lows[i - 1][1] - 1) * 100
                        ind_change = (ind_lows[j][1] / ind_lows[j - 1][1] - 1) * 100
                        strength = min(100, 70 + abs(ind_change - price_change))

                        hidden_bullish.append({
                            'price_idx1': price_lows[i - 1][0],
                            'price_idx2': price_lows[i][0],
                            'ind_idx1': ind_lows[j - 1][0],
                            'ind_idx2': ind_lows[j][0],
                            'price1': price_lows[i - 1][1],
                            'price2': price_lows[i][1],
                            'ind1': ind_lows[j - 1][1],
                            'ind2': ind_lows[j][1],
                            'strength': strength,
                            'description': 'واگرایی صعودی مخفی (کف‌های بالاتر قیمت، کف‌های پایین‌تر اندیکاتور)'
                        })

        # شناسایی واگرایی نزولی مخفی (Hidden Bearish)
        # قیمت: سقف‌های پایین‌تر، اندیکاتور: سقف‌های بالاتر
        for i in range(1, len(price_highs)):
            for j in range(1, len(ind_highs)):
                # اگر تفاوت زمانی بین دو نقطه کمتر از حد مجاز باشد
                if abs(price_highs[i][0] - ind_highs[j][0]) <= window_size // 4:
                    # چک کردن واگرایی نزولی مخفی
                    if (i > 0 and j > 0 and
                            price_highs[i][1] < price_highs[i - 1][1] and
                            ind_highs[j][1] > ind_highs[j - 1][1]):
                        # محاسبه قدرت واگرایی براساس میزان تفاوت
                        price_change = (price_highs[i][1] / price_highs[i - 1][1] - 1) * 100
                        ind_change = (ind_highs[j][1] / ind_highs[j - 1][1] - 1) * 100
                        strength = min(100, 70 + abs(ind_change - price_change))

                        hidden_bearish.append({
                            'price_idx1': price_highs[i - 1][0],
                            'price_idx2': price_highs[i][0],
                            'ind_idx1': ind_highs[j - 1][0],
                            'ind_idx2': ind_highs[j][0],
                            'price1': price_highs[i - 1][1],
                            'price2': price_highs[i][1],
                            'ind1': ind_highs[j - 1][1],
                            'ind2': ind_highs[j][1],
                            'strength': strength,
                            'description': 'واگرایی نزولی مخفی (سقف‌های پایین‌تر قیمت، سقف‌های بالاتر اندیکاتور)'
                        })

        # تعیین نوع واگرایی اخیر
        current_divergence = 'بدون واگرایی'
        current_strength = 0
        signal_type = 'خنثی'

        # بررسی واگرایی‌های اخیر (در 10 کندل آخر)
        recent_rb = [d for d in regular_bullish if d['price_idx2'] >= len(df_div) - 10]
        recent_hb = [d for d in hidden_bullish if d['price_idx2'] >= len(df_div) - 10]
        recent_rs = [d for d in regular_bearish if d['price_idx2'] >= len(df_div) - 10]
        recent_hs = [d for d in hidden_bearish if d['price_idx2'] >= len(df_div) - 10]

        if recent_rb:
            current_divergence = 'واگرایی صعودی عادی'
            current_strength = max([d['strength'] for d in recent_rb])
            signal_type = 'خرید'
        elif recent_hb:
            current_divergence = 'واگرایی صعودی مخفی'
            current_strength = max([d['strength'] for d in recent_hb])
            signal_type = 'خرید'
        elif recent_rs:
            current_divergence = 'واگرایی نزولی عادی'
            current_strength = max([d['strength'] for d in recent_rs])
            signal_type = 'فروش'
        elif recent_hs:
            current_divergence = 'واگرایی نزولی مخفی'
            current_strength = max([d['strength'] for d in recent_hs])
            signal_type = 'فروش'

        # نتایج نهایی
        result = {
            'current_divergence': current_divergence,
            'current_strength': current_strength,
            'signal_type': signal_type,
            'regular_bullish': regular_bullish,
            'regular_bearish': regular_bearish,
            'hidden_bullish': hidden_bullish,
            'hidden_bearish': hidden_bearish,
            'indicator_type': indicator,
            'dataframe': df_div
        }

        return result

    def analyze_bbrsi_macd_strategy(self, df):
        """
        استراتژی ترکیبی باندهای بولینگر، RSI و MACD

        Args:
            df: دیتافریم داده‌ها

        Returns:
            نتایج استراتژی ترکیبی
        """
        # ایجاد کپی از دیتافریم
        df_combo = df.copy()

        # محاسبه باندهای بولینگر
        df_combo['bb_upper'] = ta.volatility.bollinger_hband(df_combo['close'], window=20, window_dev=2)
        df_combo['bb_middle'] = ta.volatility.bollinger_mavg(df_combo['close'], window=20)
        df_combo['bb_lower'] = ta.volatility.bollinger_lband(df_combo['close'], window=20, window_dev=2)

        # محاسبه باند‌های بولینگر با انحراف معیار کمتر (1.5) برای تشخیص دقیق‌تر
        df_combo['bb_upper_tight'] = ta.volatility.bollinger_hband(df_combo['close'], window=20, window_dev=1.5)
        df_combo['bb_lower_tight'] = ta.volatility.bollinger_lband(df_combo['close'], window=20, window_dev=1.5)

        # محاسبه پهنای باندهای بولینگر (نشانگر نوسان)
        df_combo['bb_width'] = (df_combo['bb_upper'] - df_combo['bb_lower']) / df_combo['bb_middle']

        # محاسبه RSI
        df_combo['rsi'] = ta.momentum.rsi(df_combo['close'], window=14)

        # محاسبه MACD
        df_combo['macd'] = ta.trend.macd(df_combo['close'], window_slow=26, window_fast=12)
        df_combo['macd_signal'] = ta.trend.macd_signal(df_combo['close'], window_slow=26, window_fast=12, window_sign=9)
        df_combo['macd_hist'] = df_combo['macd'] - df_combo['macd_signal']

        # تشخیص قیمت نسبت به باندهای بولینگر
        df_combo['price_above_upper'] = df_combo['close'] > df_combo['bb_upper']
        df_combo['price_below_lower'] = df_combo['close'] < df_combo['bb_lower']
        df_combo['price_above_upper_tight'] = df_combo['close'] > df_combo['bb_upper_tight']
        df_combo['price_below_lower_tight'] = df_combo['close'] < df_combo['bb_lower_tight']

        # شناسایی واکنش قیمت به باندها
        df_combo['reaction_lower'] = (df_combo['price_below_lower'].shift(1) & ~df_combo['price_below_lower'])
        df_combo['reaction_upper'] = (df_combo['price_above_upper'].shift(1) & ~df_combo['price_above_upper'])

        # تشخیص شرایط اشباع خرید/فروش RSI
        df_combo['rsi_oversold'] = df_combo['rsi'] < 30
        df_combo['rsi_overbought'] = df_combo['rsi'] > 70

        # تشخیص کراس MACD
        df_combo['macd_bullish_cross'] = (df_combo['macd'] > df_combo['macd_signal']) & (
                df_combo['macd'].shift(1) <= df_combo['macd_signal'].shift(1))
        df_combo['macd_bearish_cross'] = (df_combo['macd'] < df_combo['macd_signal']) & (
                df_combo['macd'].shift(1) >= df_combo['macd_signal'].shift(1))

        # شناسایی همگرایی سیگنال‌ها

        # سیگنال‌های خرید قوی:
        # 1. قیمت زیر باند پایین + RSI اشباع فروش + MACD در حال افزایش
        df_combo['strong_buy'] = (
                df_combo['price_below_lower'] &
                df_combo['rsi_oversold'] &
                (df_combo['macd_hist'] > df_combo['macd_hist'].shift(1))
        )

        # 2. واکنش به باند پایین + RSI از اشباع فروش خارج شده + MACD کراس صعودی
        df_combo['strong_buy2'] = (
                df_combo['reaction_lower'] &
                (df_combo['rsi'] > 30) & (df_combo['rsi'].shift(1) <= 30) &
                df_combo['macd_bullish_cross']
        )

        # سیگنال‌های فروش قوی:
        # 1. قیمت بالای باند بالا + RSI اشباع خرید + MACD در حال کاهش
        df_combo['strong_sell'] = (
                df_combo['price_above_upper'] &
                df_combo['rsi_overbought'] &
                (df_combo['macd_hist'] < df_combo['macd_hist'].shift(1))
        )

        # 2. واکنش به باند بالا + RSI از اشباع خرید خارج شده + MACD کراس نزولی
        df_combo['strong_sell2'] = (
                df_combo['reaction_upper'] &
                (df_combo['rsi'] < 70) & (df_combo['rsi'].shift(1) >= 70) &
                df_combo['macd_bearish_cross']
        )

        # سیگنال‌های میانه:
        # خرید: قیمت نزدیک باند پایین + RSI زیر 40 + MACD کراس صعودی
        df_combo['moderate_buy'] = (
                (df_combo['close'] < df_combo['bb_lower_tight']) &
                (df_combo['rsi'] < 40) &
                df_combo['macd_bullish_cross']
        )

        # فروش: قیمت نزدیک باند بالا + RSI بالای 60 + MACD کراس نزولی
        df_combo['moderate_sell'] = (
                (df_combo['close'] > df_combo['bb_upper_tight']) &
                (df_combo['rsi'] > 60) &
                df_combo['macd_bearish_cross']
        )

        # جمع‌آوری سیگنال‌های خرید
        buy_signals = []
        for i in range(50, len(df_combo)):
            if df_combo['strong_buy'].iloc[i]:
                buy_signals.append({
                    'index': i,
                    'date': df_combo.index[i],
                    'price': df_combo['close'].iloc[i],
                    'strength': 90,
                    'description': 'سیگنال خرید قوی: قیمت زیر باند پایین + RSI اشباع فروش + MACD در حال افزایش',
                    'type': 'strong_buy'
                })
            elif df_combo['strong_buy2'].iloc[i]:
                buy_signals.append({
                    'index': i,
                    'date': df_combo.index[i],
                    'price': df_combo['close'].iloc[i],
                    'strength': 85,
                    'description': 'سیگنال خرید قوی: واکنش به باند پایین + خروج از اشباع فروش RSI + کراس صعودی MACD',
                    'type': 'strong_buy2'
                })
            elif df_combo['moderate_buy'].iloc[i]:
                buy_signals.append({
                    'index': i,
                    'date': df_combo.index[i],
                    'price': df_combo['close'].iloc[i],
                    'strength': 75,
                    'description': 'سیگنال خرید متوسط: قیمت نزدیک باند پایین + RSI زیر 40 + کراس صعودی MACD',
                    'type': 'moderate_buy'
                })

        # جمع‌آوری سیگنال‌های فروش
        sell_signals = []
        for i in range(50, len(df_combo)):
            if df_combo['strong_sell'].iloc[i]:
                sell_signals.append({
                    'index': i,
                    'date': df_combo.index[i],
                    'price': df_combo['close'].iloc[i],
                    'strength': 90,
                    'description': 'سیگنال فروش قوی: قیمت بالای باند بالا + RSI اشباع خرید + MACD در حال کاهش',
                    'type': 'strong_sell'
                })
            elif df_combo['strong_sell2'].iloc[i]:
                sell_signals.append({
                    'index': i,
                    'date': df_combo.index[i],
                    'price': df_combo['close'].iloc[i],
                    'strength': 85,
                    'description': 'سیگنال فروش قوی: واکنش به باند بالا + خروج از اشباع خرید RSI + کراس نزولی MACD',
                    'type': 'strong_sell2'
                })
            elif df_combo['moderate_sell'].iloc[i]:
                sell_signals.append({
                    'index': i,
                    'date': df_combo.index[i],
                    'price': df_combo['close'].iloc[i],
                    'strength': 75,
                    'description': 'سیگنال فروش متوسط: قیمت نزدیک باند بالا + RSI بالای 60 + کراس نزولی MACD',
                    'type': 'moderate_sell'
                })

        # بررسی شرایط فعلی
        current_status = 'خنثی'
        current_signal = 'خنثی'
        current_strength = 50

        # بررسی آخرین وضعیت نشانگرها
        last_idx = len(df_combo) - 1

        # شرایط فعلی باندهای بولینگر
        if df_combo['price_above_upper'].iloc[last_idx]:
            bb_status = 'بالای باند بالایی (احتمال اشباع خرید)'
            bb_signal = 'فروش'
            bb_strength = 70
        elif df_combo['price_below_lower'].iloc[last_idx]:
            bb_status = 'زیر باند پایینی (احتمال اشباع فروش)'
            bb_signal = 'خرید'
            bb_strength = 70
        elif df_combo['price_above_upper_tight'].iloc[last_idx]:
            bb_status = 'نزدیک به باند بالایی'
            bb_signal = 'فروش'
            bb_strength = 60
        elif df_combo['price_below_lower_tight'].iloc[last_idx]:
            bb_status = 'نزدیک به باند پایینی'
            bb_signal = 'خرید'
            bb_strength = 60
        else:
            bb_status = 'در محدوده میانی باندها'
            bb_signal = 'خنثی'
            bb_strength = 50

        # شرایط فعلی RSI
        if df_combo['rsi'].iloc[last_idx] > 70:
            rsi_status = 'اشباع خرید'
            rsi_signal = 'فروش'
            rsi_strength = 70
        elif df_combo['rsi'].iloc[last_idx] < 30:
            rsi_status = 'اشباع فروش'
            rsi_signal = 'خرید'
            rsi_strength = 70
        elif df_combo['rsi'].iloc[last_idx] > 60:
            rsi_status = 'بالا (احتمال روند صعودی)'
            rsi_signal = 'فروش'
            rsi_strength = 60
        elif df_combo['rsi'].iloc[last_idx] < 40:
            rsi_status = 'پایین (احتمال روند نزولی)'
            rsi_signal = 'خرید'
            rsi_strength = 60
        else:
            rsi_status = 'منطقه میانی'
            rsi_signal = 'خنثی'
            rsi_strength = 50

        # شرایط فعلی MACD
        if df_combo['macd_bullish_cross'].iloc[last_idx]:
            macd_status = 'کراس صعودی'
            macd_signal = 'خرید'
            macd_strength = 70
        elif df_combo['macd_bearish_cross'].iloc[last_idx]:
            macd_status = 'کراس نزولی'
            macd_signal = 'فروش'
            macd_strength = 70
        elif df_combo['macd'].iloc[last_idx] > df_combo['macd_signal'].iloc[last_idx]:
            macd_status = 'بالای خط سیگنال (روند صعودی)'
            macd_signal = 'خرید'
            macd_strength = 60
        else:
            macd_status = 'زیر خط سیگنال (روند نزولی)'
            macd_signal = 'فروش'
            macd_strength = 60

        # تعیین سیگنال نهایی با توجه به همگرایی نشانگرها
        if bb_signal == rsi_signal == macd_signal == 'خرید':
            current_status = 'همگرایی کامل نشانگرها (خرید)'
            current_signal = 'خرید'
            current_strength = (bb_strength + rsi_strength + macd_strength) / 3 + 10
        elif bb_signal == rsi_signal == macd_signal == 'فروش':
            current_status = 'همگرایی کامل نشانگرها (فروش)'
            current_signal = 'فروش'
            current_strength = (bb_strength + rsi_strength + macd_strength) / 3 + 10
        elif bb_signal == rsi_signal == 'خرید' or bb_signal == macd_signal == 'خرید' or rsi_signal == macd_signal == 'خرید':
            current_status = 'همگرایی نسبی نشانگرها (خرید)'
            current_signal = 'خرید'
            current_strength = (bb_strength + rsi_strength + macd_strength) / 3 + 5
        elif bb_signal == rsi_signal == 'فروش' or bb_signal == macd_signal == 'فروش' or rsi_signal == macd_signal == 'فروش':
            current_status = 'همگرایی نسبی نشانگرها (فروش)'
            current_signal = 'فروش'
            current_strength = (bb_strength + rsi_strength + macd_strength) / 3 + 5

        # بررسی سیگنال‌های اخیر
        recent_buy = [s for s in buy_signals if len(df_combo) - s['index'] <= 3]
        recent_sell = [s for s in sell_signals if len(df_combo) - s['index'] <= 3]

        if recent_buy:
            current_signal = 'خرید'
            current_strength = max([s['strength'] for s in recent_buy])
            current_status = 'سیگنال اخیر خرید - ' + max(recent_buy, key=lambda x: x['strength'])['description']
        elif recent_sell:
            current_signal = 'فروش'
            current_strength = max([s['strength'] for s in recent_sell])
            current_status = 'سیگنال اخیر فروش - ' + max(recent_sell, key=lambda x: x['strength'])['description']

        # تعیین نقاط ورود و خروج
        entry_points = []
        exit_points = []

        for signal in buy_signals:
            # محاسبه حد ضرر و حد سود برای سیگنال‌های خرید
            stop_loss = signal['price'] * 0.97  # 3% زیر قیمت ورود
            take_profit = signal['price'] * 1.06  # 6% بالای قیمت ورود

            entry_points.append({
                'index': signal['index'],
                'price': signal['price'],
                'stop_loss': stop_loss,
                'take_profit': take_profit,
                'type': 'long',
                'strength': signal['strength']
            })

        for signal in sell_signals:
            # محاسبه حد ضرر و حد سود برای سیگنال‌های فروش
            stop_loss = signal['price'] * 1.03  # 3% بالای قیمت ورود
            take_profit = signal['price'] * 0.94  # 6% زیر قیمت ورود

            entry_points.append({
                'index': signal['index'],
                'price': signal['price'],
                'stop_loss': stop_loss,
                'take_profit': take_profit,
                'type': 'short',
                'strength': signal['strength']
            })

        # خروج: واکنش قیمت به باندها یا تغییر وضعیت MACD
        for i in range(50, len(df_combo)):
            if df_combo['reaction_upper'].iloc[i] or df_combo['macd_bearish_cross'].iloc[i]:
                exit_points.append({
                    'index': i,
                    'price': df_combo['close'].iloc[i],
                    'type': 'exit_long'
                })

            if df_combo['reaction_lower'].iloc[i] or df_combo['macd_bullish_cross'].iloc[i]:
                exit_points.append({
                    'index': i,
                    'price': df_combo['close'].iloc[i],
                    'type': 'exit_short'
                })

        # نتایج نهایی
        result = {
            'current_status': current_status,
            'current_signal': current_signal,
            'current_strength': min(100, current_strength),
            'indicator_status': {
                'bollinger': bb_status,
                'rsi': rsi_status,
                'macd': macd_status
            },
            'buy_signals': buy_signals,
            'sell_signals': sell_signals,
            'entry_points': entry_points,
            'exit_points': exit_points,
            'dataframe': df_combo
        }

        return result

    def analyze_darvas_box(self, df, lookback_period=20, breakout_threshold=0.02):
        """
        تحلیل استراتژی جعبه دارواس (Darvas Box)

        Args:
            df: دیتافریم داده‌ها
            lookback_period: دوره بررسی برای یافتن سقف و کف
            breakout_threshold: آستانه شکست مستطیل (درصد)

        Returns:
            نتایج استراتژی جعبه دارواس
        """
        # ایجاد کپی از دیتافریم
        df_darvas = df.copy()

        # ستون‌های جدید برای ذخیره اطلاعات جعبه دارواس
        df_darvas['darvas_top'] = np.nan
        df_darvas['darvas_bottom'] = np.nan
        df_darvas['box_active'] = False
        df_darvas['box_breakout_up'] = False
        df_darvas['box_breakout_down'] = False

        # یافتن اوج‌های جدید
        df_darvas['new_high'] = df_darvas['high'].rolling(window=lookback_period).max() == df_darvas['high']

        # یافتن کف‌های جدید
        df_darvas['new_low'] = df_darvas['low'].rolling(window=lookback_period).min() == df_darvas['low']

        # افزودن حجم به تحلیل (قوت بیشتر به شکست‌ها)
        df_darvas['volume_sma10'] = df_darvas['volume'].rolling(window=10).mean()
        df_darvas['high_volume'] = df_darvas['volume'] > df_darvas['volume_sma10'] * 1.5

        # اطلاعات جعبه‌های دارواس
        darvas_boxes = []
        active_box = None

        # شناسایی جعبه‌های دارواس
        for i in range(lookback_period, len(df_darvas)):
            # اگر جعبه فعال داریم
            if active_box:
                current_price = df_darvas['close'].iloc[i]

                # بررسی شکست از بالای جعبه
                if current_price > active_box['top'] * (1 + breakout_threshold):
                    # شکست صعودی
                    df_darvas['box_breakout_up'].iloc[i] = True

                    # اگر حجم بالاست، اعتبار شکست بیشتر است
                    breakout_strength = 80
                    if df_darvas['high_volume'].iloc[i]:
                        breakout_strength = 90

                    # افزودن سیگنال شکست
                    darvas_boxes.append({
                        'box_start_idx': active_box['start_idx'],
                        'box_end_idx': i,
                        'top': active_box['top'],
                        'bottom': active_box['bottom'],
                        'breakout_idx': i,
                        'breakout_price': current_price,
                        'breakout_direction': 'up',
                        'strength': breakout_strength,
                        'entry_price': current_price,
                        'stop_loss': active_box['bottom'],
                        'take_profit': current_price + (active_box['top'] - active_box['bottom']) * 1.5
                    })

                    # جعبه جدید با کف قبلی
                    active_box = {
                        'start_idx': i,
                        'top': current_price,
                        'bottom': active_box['bottom'],
                        'duration': 0
                    }

                # بررسی شکست از پایین جعبه
                elif current_price < active_box['bottom'] * (1 - breakout_threshold):
                    # شکست نزولی
                    df_darvas['box_breakout_down'].iloc[i] = True

                    # اگر حجم بالاست، اعتبار شکست بیشتر است
                    breakout_strength = 80
                    if df_darvas['high_volume'].iloc[i]:
                        breakout_strength = 90

                    # افزودن سیگنال شکست
                    darvas_boxes.append({
                        'box_start_idx': active_box['start_idx'],
                        'box_end_idx': i,
                        'top': active_box['top'],
                        'bottom': active_box['bottom'],
                        'breakout_idx': i,
                        'breakout_price': current_price,
                        'breakout_direction': 'down',
                        'strength': breakout_strength,
                        'entry_price': current_price,
                        'stop_loss': active_box['top'],
                        'take_profit': current_price - (active_box['top'] - active_box['bottom']) * 1.5
                    })

                    # جعبه جدید با سقف قبلی
                    active_box = {
                        'start_idx': i,
                        'top': active_box['top'],
                        'bottom': current_price,
                        'duration': 0
                    }

                # بروزرسانی جعبه فعلی
                else:
                    active_box['duration'] += 1

                    # ذخیره اطلاعات جعبه در دیتافریم
                    df_darvas['darvas_top'].iloc[i] = active_box['top']
                    df_darvas['darvas_bottom'].iloc[i] = active_box['bottom']
                    df_darvas['box_active'].iloc[i] = True

            # اگر جعبه فعال نداریم و یک اوج جدید پیدا کردیم
            elif df_darvas['new_high'].iloc[i]:
                active_box = {
                    'start_idx': i,
                    'top': df_darvas['high'].iloc[i],
                    'bottom': df_darvas['low'].iloc[max(0, i - 5):i + 1].min(),  # کف در 5 کندل اخیر
                    'duration': 0
                }

                # ذخیره اطلاعات جعبه در دیتافریم
                df_darvas['darvas_top'].iloc[i] = active_box['top']
                df_darvas['darvas_bottom'].iloc[i] = active_box['bottom']
                df_darvas['box_active'].iloc[i] = True

        # بررسی وضعیت فعلی
        current_signal = 'خنثی'
        current_strength = 50

        # اگر آخرین شکست در 5 کندل اخیر باشد، سیگنال را بازیابی می‌کنیم
        if darvas_boxes:
            last_box = darvas_boxes[-1]
            if len(df_darvas) - last_box['breakout_idx'] <= 5:
                if last_box['breakout_direction'] == 'up':
                    current_signal = 'خرید'
                    current_strength = last_box['strength']
                else:
                    current_signal = 'فروش'
                    current_strength = last_box['strength']

        # بررسی اینکه آیا قیمت فعلی نزدیک لبه یک جعبه فعال است
        last_idx = len(df_darvas) - 1
        if df_darvas['box_active'].iloc[last_idx]:
            top = df_darvas['darvas_top'].iloc[last_idx]
            bottom = df_darvas['darvas_bottom'].iloc[last_idx]
            current_price = df_darvas['close'].iloc[last_idx]

            # اگر قیمت نزدیک سقف جعبه است (احتمال شکست صعودی)
            if current_price > top * 0.98:
                if current_signal == 'خنثی':
                    current_signal = 'خرید'
                    current_strength = 70

            # اگر قیمت نزدیک کف جعبه است (احتمال شکست نزولی)
            elif current_price < bottom * 1.02:
                if current_signal == 'خنثی':
                    current_signal = 'فروش'
                    current_strength = 70

        # آماده‌سازی نقاط ورود و خروج
        entry_points = []
        for box in darvas_boxes:
            entry_type = 'long' if box['breakout_direction'] == 'up' else 'short'
            entry_points.append({
                'index': box['breakout_idx'],
                'price': box['entry_price'],
                'stop_loss': box['stop_loss'],
                'take_profit': box['take_profit'],
                'type': entry_type,
                'strength': box['strength']
            })

        # نتایج نهایی
        result = {
            'current_signal': current_signal,
            'current_strength': current_strength,
            'darvas_boxes': darvas_boxes,
            'entry_points': entry_points,
            'dataframe': df_darvas
        }

        return result

    def analyze_ichimoku(self, df):
        """
        استراتژی ایچیموکو (Ichimoku Kinko Hyo)

        Args:
            df: دیتافریم داده‌ها

        Returns:
            نتایج استراتژی ایچیموکو
        """
        # ایجاد کپی از دیتافریم
        df_ichimoku = df.copy()

        # محاسبه خطوط ایچیموکو
        # تنکان-سن (Tenkan-sen) - خط تبدیل - معمولاً میانگین بالاترین بالا و پایین‌ترین پایین در 9 کندل اخیر
        high_9 = df_ichimoku['high'].rolling(window=9).max()
        low_9 = df_ichimoku['low'].rolling(window=9).min()
        df_ichimoku['tenkan_sen'] = (high_9 + low_9) / 2

        # کیجون-سن (Kijun-sen) - خط پایه - معمولاً میانگین بالاترین بالا و پایین‌ترین پایین در 26 کندل اخیر
        high_26 = df_ichimoku['high'].rolling(window=26).max()
        low_26 = df_ichimoku['low'].rolling(window=26).min()
        df_ichimoku['kijun_sen'] = (high_26 + low_26) / 2

        # سنکو اسپن A (Senkou Span A) - خط اول ابر - میانگین تنکان-سن و کیجون-سن، رسم شده 26 کندل جلوتر
        df_ichimoku['senkou_span_a'] = ((df_ichimoku['tenkan_sen'] + df_ichimoku['kijun_sen']) / 2).shift(26)

        # سنکو اسپن B (Senkou Span B) - خط دوم ابر - میانگین بالاترین بالا و پایین‌ترین پایین در 52 کندل اخیر، رسم شده 26 کندل جلوتر
        high_52 = df_ichimoku['high'].rolling(window=52).max()
        low_52 = df_ichimoku['low'].rolling(window=52).min()
        df_ichimoku['senkou_span_b'] = ((high_52 + low_52) / 2).shift(26)

        # چیکو اسپن (Chikou Span) - خط تأخیری - قیمت بسته شدن، رسم شده 26 کندل عقب‌تر
        df_ichimoku['chikou_span'] = df_ichimoku['close'].shift(-26)

        # شناسایی کراس تنکان-سن و کیجون-سن (سیگنال‌های معامله)
        df_ichimoku['tenkan_cross_above_kijun'] = (
                (df_ichimoku['tenkan_sen'] > df_ichimoku['kijun_sen']) &
                (df_ichimoku['tenkan_sen'].shift(1) <= df_ichimoku['kijun_sen'].shift(1))
        )

        df_ichimoku['tenkan_cross_below_kijun'] = (
                (df_ichimoku['tenkan_sen'] < df_ichimoku['kijun_sen']) &
                (df_ichimoku['tenkan_sen'].shift(1) >= df_ichimoku['kijun_sen'].shift(1))
        )

        # تشخیص موقعیت قیمت نسبت به ابر
        df_ichimoku['price_above_cloud'] = (
                (df_ichimoku['close'] > df_ichimoku['senkou_span_a']) &
                (df_ichimoku['close'] > df_ichimoku['senkou_span_b'])
        )

        df_ichimoku['price_below_cloud'] = (
                (df_ichimoku['close'] < df_ichimoku['senkou_span_a']) &
                (df_ichimoku['close'] < df_ichimoku['senkou_span_b'])
        )

        df_ichimoku['price_in_cloud'] = ~(df_ichimoku['price_above_cloud'] | df_ichimoku['price_below_cloud'])

        # تشخیص رنگ ابر (ابر سبز یا قرمز)
        df_ichimoku['green_cloud'] = df_ichimoku['senkou_span_a'] > df_ichimoku['senkou_span_b']
        df_ichimoku['red_cloud'] = df_ichimoku['senkou_span_a'] < df_ichimoku['senkou_span_b']

        # تشخیص موقعیت چیکو اسپن نسبت به قیمت (سیگنال تأییدی)
        df_ichimoku['chikou_above_price'] = df_ichimoku['chikou_span'] > df_ichimoku['close']
        df_ichimoku['chikou_below_price'] = df_ichimoku['chikou_span'] < df_ichimoku['close']

        # شناسایی سیگنال‌های خرید قوی
        df_ichimoku['strong_buy_signal'] = (
                df_ichimoku['tenkan_cross_above_kijun'] &
                df_ichimoku['price_above_cloud'] &
                df_ichimoku['green_cloud']
        )

        # شناسایی سیگنال‌های فروش قوی
        df_ichimoku['strong_sell_signal'] = (
                df_ichimoku['tenkan_cross_below_kijun'] &
                df_ichimoku['price_below_cloud'] &
                df_ichimoku['red_cloud']
        )

        # جمع‌آوری سیگنال‌های خرید
        buy_signals = []
        for i in range(52, len(df_ichimoku)):
            if df_ichimoku['strong_buy_signal'].iloc[i]:
                # بررسی موقعیت چیکو اسپن برای تأیید بیشتر
                chikou_confirmation = False
                if i >= 26 and df_ichimoku['chikou_above_price'].iloc[i - 26]:
                    chikou_confirmation = True

                strength = 85
                if chikou_confirmation:
                    strength = 95

                description = 'سیگنال خرید قوی ایچیموکو: کراس صعودی تنکان-سن بالای کیجون-سن + قیمت بالای ابر سبز'
                if chikou_confirmation:
                    description += ' + تأیید چیکو اسپن'

                buy_signals.append({
                    'index': i,
                    'date': df_ichimoku.index[i],
                    'price': df_ichimoku['close'].iloc[i],
                    'strength': strength,
                    'description': description
                })

            # سیگنال خرید متوسط: کراس صعودی تنکان-سن بالای کیجون-سن + قیمت در حال خروج از ابر
            elif (df_ichimoku['tenkan_cross_above_kijun'].iloc[i] and
                  df_ichimoku['price_in_cloud'].iloc[i] and
                  df_ichimoku['close'].iloc[i] > df_ichimoku['senkou_span_a'].iloc[i]):

                buy_signals.append({
                    'index': i,
                    'date': df_ichimoku.index[i],
                    'price': df_ichimoku['close'].iloc[i],
                    'strength': 75,
                    'description': 'سیگنال خرید متوسط ایچیموکو: کراس صعودی تنکان-سن بالای کیجون-سن + قیمت در حال خروج از ابر'
                })

        # جمع‌آوری سیگنال‌های فروش
        sell_signals = []
        for i in range(52, len(df_ichimoku)):
            if df_ichimoku['strong_sell_signal'].iloc[i]:
                # بررسی موقعیت چیکو اسپن برای تأیید بیشتر
                chikou_confirmation = False
                if i >= 26 and df_ichimoku['chikou_below_price'].iloc[i - 26]:
                    chikou_confirmation = True

                strength = 85
                if chikou_confirmation:
                    strength = 95

                description = 'سیگنال فروش قوی ایچیموکو: کراس نزولی تنکان-سن زیر کیجون-سن + قیمت زیر ابر قرمز'
                if chikou_confirmation:
                    description += ' + تأیید چیکو اسپن'

                sell_signals.append({
                    'index': i,
                    'date': df_ichimoku.index[i],
                    'price': df_ichimoku['close'].iloc[i],
                    'strength': strength,
                    'description': description
                })

            # سیگنال فروش متوسط: کراس نزولی تنکان-سن زیر کیجون-سن + قیمت در حال خروج از ابر
            elif (df_ichimoku['tenkan_cross_below_kijun'].iloc[i] and
                  df_ichimoku['price_in_cloud'].iloc[i] and
                  df_ichimoku['close'].iloc[i] < df_ichimoku['senkou_span_a'].iloc[i]):

                sell_signals.append({
                    'index': i,
                    'date': df_ichimoku.index[i],
                    'price': df_ichimoku['close'].iloc[i],
                    'strength': 75,
                    'description': 'سیگنال فروش متوسط ایچیموکو: کراس نزولی تنکان-سن زیر کیجون-سن + قیمت در حال خروج از ابر'
                })

        # آماده‌سازی نقاط ورود و خروج
        entry_points = []

        for signal in buy_signals:
            # محاسبه حد ضرر و حد سود برای سیگنال‌های خرید
            # حد ضرر: کیجون-سن یا کف ابر (هرکدام که نزدیکتر است)
            kijun = df_ichimoku['kijun_sen'].iloc[signal['index']]
            cloud_bottom = min(
                df_ichimoku['senkou_span_a'].iloc[signal['index']],
                df_ichimoku['senkou_span_b'].iloc[signal['index']]
            )

            stop_loss = max(kijun, cloud_bottom)
            if stop_loss > signal['price'] * 0.97:
                stop_loss = signal['price'] * 0.97  # حداکثر 3% زیان

            # حد سود: 2 برابر فاصله تا حد ضرر
            risk = signal['price'] - stop_loss
            take_profit = signal['price'] + risk * 2

            entry_points.append({
                'index': signal['index'],
                'price': signal['price'],
                'stop_loss': stop_loss,
                'take_profit': take_profit,
                'type': 'long',
                'strength': signal['strength']
            })

        for signal in sell_signals:
            # محاسبه حد ضرر و حد سود برای سیگنال‌های فروش
            # حد ضرر: کیجون-سن یا سقف ابر (هرکدام که نزدیکتر است)
            kijun = df_ichimoku['kijun_sen'].iloc[signal['index']]
            cloud_top = max(
                df_ichimoku['senkou_span_a'].iloc[signal['index']],
                df_ichimoku['senkou_span_b'].iloc[signal['index']]
            )

            stop_loss = min(kijun, cloud_top)
            if stop_loss < signal['price'] * 1.03:
                stop_loss = signal['price'] * 1.03  # حداکثر 3% زیان

            # حد سود: 2 برابر فاصله تا حد ضرر
            risk = stop_loss - signal['price']
            take_profit = signal['price'] - risk * 2

            entry_points.append({
                'index': signal['index'],
                'price': signal['price'],
                'stop_loss': stop_loss,
                'take_profit': take_profit,
                'type': 'short',
                'strength': signal['strength']
            })

        # بررسی وضعیت فعلی
        current_signal = 'خنثی'
        current_strength = 50

        # بررسی آخرین وضعیت ایچیموکو
        last_idx = len(df_ichimoku) - 1

        # موقعیت قیمت نسبت به ابر
        if df_ichimoku['price_above_cloud'].iloc[last_idx]:
            if df_ichimoku['green_cloud'].iloc[last_idx]:
                current_signal = 'خرید'
                current_strength = 70
                current_status = 'قیمت بالای ابر سبز (روند صعودی قوی)'
            else:
                current_signal = 'خرید'
                current_strength = 60
                current_status = 'قیمت بالای ابر قرمز (احتمال تغییر روند به صعودی)'

        elif df_ichimoku['price_below_cloud'].iloc[last_idx]:
            if df_ichimoku['red_cloud'].iloc[last_idx]:
                current_signal = 'فروش'
                current_strength = 70
                current_status = 'قیمت زیر ابر قرمز (روند نزولی قوی)'
            else:
                current_signal = 'فروش'
                current_strength = 60
                current_status = 'قیمت زیر ابر سبز (احتمال تغییر روند به نزولی)'

        else:
            current_status = 'قیمت درون ابر (عدم قطعیت روند)'

        # سیگنال‌های کراس اخیر
        if df_ichimoku['tenkan_cross_above_kijun'].iloc[last_idx - 3:last_idx + 1].any():
            current_signal = 'خرید'
            current_strength = 75
            current_status = 'کراس صعودی اخیر تنکان-سن بالای کیجون-سن'

        elif df_ichimoku['tenkan_cross_below_kijun'].iloc[last_idx - 3:last_idx + 1].any():
            current_signal = 'فروش'
            current_strength = 75
            current_status = 'کراس نزولی اخیر تنکان-سن زیر کیجون-سن'

        # سیگنال‌های اخیر
        recent_buy = [s for s in buy_signals if last_idx - s['index'] <= 3]
        recent_sell = [s for s in sell_signals if last_idx - s['index'] <= 3]

        if recent_buy:
            current_signal = 'خرید'
            current_strength = max([s['strength'] for s in recent_buy])
            current_status = max(recent_buy, key=lambda x: x['strength'])['description']

        elif recent_sell:
            current_signal = 'فروش'
            current_strength = max([s['strength'] for s in recent_sell])
            current_status = max(recent_sell, key=lambda x: x['strength'])['description']

        # نتایج نهایی
        result = {
            'current_signal': current_signal,
            'current_strength': current_strength,
            'current_status': current_status,
            'buy_signals': buy_signals,
            'sell_signals': sell_signals,
            'entry_points': entry_points,
            'dataframe': df_ichimoku
        }

        return result

    def analyze_fibonacci(self, df, period=120, use_atr=True):
        """
        استراتژی فیبوناچی با بهترین ترکیب

        Args:
            df: دیتافریم داده‌ها
            period: دوره بررسی برای یافتن اوج‌ها و حضیض‌ها
            use_atr: استفاده از ATR برای تنظیم حد ضرر و حد سود

        Returns:
            نتایج استراتژی فیبوناچی
        """
        # ایجاد کپی از دیتافریم
        df_fib = df.copy()

        # محاسبه ATR
        if use_atr:
            df_fib['atr'] = ta.volatility.average_true_range(df_fib['high'], df_fib['low'], df_fib['close'], window=14)

        # یافتن اوج‌ها و حضیض‌های مهم در دوره مشخص شده
        df_fib['swing_high'] = df_fib['high'].rolling(window=period, center=True).apply(
            lambda x: 1 if x.iloc[period // 2] == max(x) else 0, raw=False)

        df_fib['swing_low'] = df_fib['low'].rolling(window=period, center=True).apply(
            lambda x: 1 if x.iloc[period // 2] == min(x) else 0, raw=False)

        # لیست اوج‌ها و حضیض‌ها
        swing_highs = []
        swing_lows = []

        for i in range(period, len(df_fib) - period):
            if df_fib['swing_high'].iloc[i] == 1:
                swing_highs.append((i, df_fib['high'].iloc[i]))
            if df_fib['swing_low'].iloc[i] == 1:
                swing_lows.append((i, df_fib['low'].iloc[i]))

        # سطوح فیبوناچی معمول
        fib_levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]

        # لیست برای ذخیره نتایج فیبوناچی
        fibonacci_retracements = []
        fibonacci_extensions = []

        # سطوح حمایت و مقاومت مهم فیبوناچی
        current_support_levels = []
        current_resistance_levels = []

        # برای هر جفت از اوج و حضیض مجاور
        # ابتدا فیبوناچی بازگشتی (retracement) را محاسبه می‌کنیم
        for i in range(1, min(len(swing_highs), len(swing_lows))):
            # بازگشت از اوج به حضیض (روند نزولی)
            if swing_highs[i - 1][0] < swing_lows[i][0]:
                high_val = swing_highs[i - 1][1]
                low_val = swing_lows[i][1]
                price_range = high_val - low_val

                # محاسبه سطوح فیبوناچی
                levels = {}
                for fib in fib_levels:
                    levels[fib] = high_val - price_range * fib

                fibonacci_retracements.append({
                    'type': 'bearish',
                    'high_idx': swing_highs[i - 1][0],
                    'low_idx': swing_lows[i][0],
                    'high_val': high_val,
                    'low_val': low_val,
                    'levels': levels
                })

                # بررسی سطوح مقاومت فعلی (در روند نزولی)
                current_price = df_fib['close'].iloc[-1]
                for fib, level in levels.items():
                    if level > current_price:
                        current_resistance_levels.append((fib, level))
                    elif level < current_price:
                        current_support_levels.append((fib, level))

            # بازگشت از حضیض به اوج (روند صعودی)
            elif swing_lows[i - 1][0] < swing_highs[i][0]:
                high_val = swing_highs[i][1]
                low_val = swing_lows[i - 1][1]
                price_range = high_val - low_val

                # محاسبه سطوح فیبوناچی
                levels = {}
                for fib in fib_levels:
                    levels[fib] = low_val + price_range * fib

                fibonacci_retracements.append({
                    'type': 'bullish',
                    'high_idx': swing_highs[i][0],
                    'low_idx': swing_lows[i - 1][0],
                    'high_val': high_val,
                    'low_val': low_val,
                    'levels': levels
                })

                # بررسی سطوح حمایت فعلی (در روند صعودی)
                current_price = df_fib['close'].iloc[-1]
                for fib, level in levels.items():
                    if level < current_price:
                        current_support_levels.append((fib, level))
                    elif level > current_price:
                        current_resistance_levels.append((fib, level))

        # مرتب‌سازی سطوح حمایت و مقاومت
        current_support_levels.sort(key=lambda x: x[1], reverse=True)
        current_resistance_levels.sort(key=lambda x: x[1])

        # بررسی واکنش قیمت به سطوح فیبوناچی
        price_reactions = []

        # ستون‌های برای ذخیره سطوح فیبوناچی نزدیک
        df_fib['nearest_fib_level'] = np.nan
        df_fib['nearest_fib_value'] = np.nan
        df_fib['fib_reaction'] = False

        for i in range(period, len(df_fib)):
            # بررسی همه سطوح فیبوناچی محاسبه شده
            for fib_obj in fibonacci_retracements:
                # اگر این کندل بعد از محاسبه فیبوناچی است
                if i > max(fib_obj['high_idx'], fib_obj['low_idx']):
                    levels = fib_obj['levels']

                    # بررسی واکنش قیمت به هر سطح
                    for fib, level in levels.items():
                        # اگر قیمت کندل به این سطح رسیده است
                        if (df_fib['low'].iloc[i] <= level <= df_fib['high'].iloc[i]):
                            # اگر کندل صعودی است و از سطح حمایت بالا رفته است
                            if df_fib['close'].iloc[i] > df_fib['open'].iloc[i] and df_fib['low'].iloc[i] <= level:
                                df_fib['nearest_fib_level'].iloc[i] = fib
                                df_fib['nearest_fib_value'].iloc[i] = level
                                df_fib['fib_reaction'].iloc[i] = True

                                # افزودن به لیست واکنش‌ها
                                reaction_strength = 70
                                if fib in [0.382, 0.618]:  # سطوح قوی‌تر
                                    reaction_strength = 85

                                price_reactions.append({
                                    'index': i,
                                    'date': df_fib.index[i],
                                    'price': df_fib['close'].iloc[i],
                                    'fib_level': fib,
                                    'fib_value': level,
                                    'reaction_type': 'support' if fib_obj['type'] == 'bullish' else 'resistance',
                                    'strength': reaction_strength
                                })

                            # اگر کندل نزولی است و از سطح مقاومت پایین رفته است
                            elif df_fib['close'].iloc[i] < df_fib['open'].iloc[i] and df_fib['high'].iloc[i] >= level:
                                df_fib['nearest_fib_level'].iloc[i] = fib
                                df_fib['nearest_fib_value'].iloc[i] = level
                                df_fib['fib_reaction'].iloc[i] = True

                                # افزودن به لیست واکنش‌ها
                                reaction_strength = 70
                                if fib in [0.382, 0.618]:  # سطوح قوی‌تر
                                    reaction_strength = 85

                                price_reactions.append({
                                    'index': i,
                                    'date': df_fib.index[i],
                                    'price': df_fib['close'].iloc[i],
                                    'fib_level': fib,
                                    'fib_value': level,
                                    'reaction_type': 'resistance' if fib_obj['type'] == 'bullish' else 'support',
                                    'strength': reaction_strength
                                })

        # شناسایی سیگنال‌های معاملاتی بر اساس واکنش قیمت
        buy_signals = []
        sell_signals = []

        for i in range(1, len(price_reactions)):
            reaction = price_reactions[i]

            # سیگنال خرید: واکنش مثبت به سطح حمایت فیبوناچی
            if reaction['reaction_type'] == 'support':
                # اگر قیمت از سطح حمایت بالا رفته است
                candle_idx = reaction['index']
                if (df_fib['close'].iloc[candle_idx] > df_fib['open'].iloc[candle_idx] and
                        df_fib['close'].iloc[candle_idx] > reaction['fib_value']):

                    # بررسی حجم برای تأیید بیشتر
                    volume_confirmation = df_fib['volume'].iloc[candle_idx] > df_fib['volume'].iloc[
                                                                              candle_idx - 5:candle_idx].mean()
                    strength = reaction['strength']

                    if volume_confirmation:
                        strength += 10

                    description = f"سیگنال خرید فیبوناچی: واکنش مثبت به سطح حمایت {reaction['fib_level']}"
                    if volume_confirmation:
                        description += " با تأیید حجم"

                    # محاسبه حد ضرر و حد سود
                    if use_atr:
                        atr = df_fib['atr'].iloc[candle_idx]
                        stop_loss = reaction['fib_value'] - 1.5 * atr
                        take_profit = reaction['price'] + 3 * atr
                    else:
                        # حد ضرر: سطح فیبوناچی قبلی
                        if reaction['fib_level'] == 0:
                            stop_loss = reaction['fib_value'] * 0.97
                        else:
                            prev_level = max([fib for fib in fib_levels if fib < reaction['fib_level']], default=0)
                            fib_range = fibonacci_retracements[-1]['high_val'] - fibonacci_retracements[-1]['low_val']
                            stop_loss = reaction['fib_value'] - fib_range * 0.1

                        # حد سود: سطح فیبوناچی بعدی
                        next_level = min([fib for fib in fib_levels if fib > reaction['fib_level']], default=1)
                        take_profit = fibonacci_retracements[-1]['low_val'] + fibonacci_retracements[-1]['high_val'] - \
                                      fibonacci_retracements[-1]['low_val'] * next_level

                    buy_signals.append({
                        'index': candle_idx,
                        'date': df_fib.index[candle_idx],
                        'price': df_fib['close'].iloc[candle_idx],
                        'stop_loss': stop_loss,
                        'take_profit': take_profit,
                        'strength': min(100, strength),
                        'description': description
                    })

            # سیگنال فروش: واکنش منفی به سطح مقاومت فیبوناچی
            elif reaction['reaction_type'] == 'resistance':
                # اگر قیمت از سطح مقاومت پایین رفته است
                candle_idx = reaction['index']
                if (df_fib['close'].iloc[candle_idx] < df_fib['open'].iloc[candle_idx] and
                        df_fib['close'].iloc[candle_idx] < reaction['fib_value']):

                    # بررسی حجم برای تأیید بیشتر
                    volume_confirmation = df_fib['volume'].iloc[candle_idx] > df_fib['volume'].iloc[
                                                                              candle_idx - 5:candle_idx].mean()
                    strength = reaction['strength']

                    if volume_confirmation:
                        strength += 10

                    description = f"سیگنال فروش فیبوناچی: واکنش منفی به سطح مقاومت {reaction['fib_level']}"
                    if volume_confirmation:
                        description += " با تأیید حجم"

                    # محاسبه حد ضرر و حد سود
                    if use_atr:
                        atr = df_fib['atr'].iloc[candle_idx]
                        stop_loss = reaction['fib_value'] + 1.5 * atr
                        take_profit = reaction['price'] - 3 * atr
                    else:
                        # حد ضرر: سطح فیبوناچی بعدی
                        if reaction['fib_level'] == 1:
                            stop_loss = reaction['fib_value'] * 1.03
                        else:
                            next_level = min([fib for fib in fib_levels if fib > reaction['fib_level']], default=1)
                            fib_range = fibonacci_retracements[-1]['high_val'] - fibonacci_retracements[-1]['low_val']
                            stop_loss = reaction['fib_value'] + fib_range * 0.1

                        # حد سود: سطح فیبوناچی قبلی
                        prev_level = max([fib for fib in fib_levels if fib < reaction['fib_level']], default=0)
                        take_profit = fibonacci_retracements[-1]['low_val'] + fibonacci_retracements[-1]['high_val'] - \
                                      fibonacci_retracements[-1]['low_val'] * prev_level

                    sell_signals.append({
                        'index': candle_idx,
                        'date': df_fib.index[candle_idx],
                        'price': df_fib['close'].iloc[candle_idx],
                        'stop_loss': stop_loss,
                        'take_profit': take_profit,
                        'strength': min(100, strength),
                        'description': description
                    })

        # بررسی وضعیت فعلی نسبت به سطوح فیبوناچی
        current_signal = 'خنثی'
        current_strength = 50

        # یافتن نزدیکترین سطح حمایت و مقاومت فیبوناچی
        current_price = df_fib['close'].iloc[-1]
        nearest_support = None
        nearest_resistance = None

        if current_support_levels:
            nearest_support = current_support_levels[0]

        if current_resistance_levels:
            nearest_resistance = current_resistance_levels[0]

        # اگر قیمت نزدیک یک سطح فیبوناچی است
        last_idx = len(df_fib) - 1
        if df_fib['fib_reaction'].iloc[last_idx - 3:last_idx + 1].any():
            # یافتن آخرین واکنش
            for i in range(last_idx, max(0, last_idx - 3), -1):
                if df_fib['fib_reaction'].iloc[i]:
                    fib_level = df_fib['nearest_fib_level'].iloc[i]
                    fib_value = df_fib['nearest_fib_value'].iloc[i]

                    # اگر واکنش به سطح حمایت بود
                    if current_price > fib_value:
                        current_signal = 'خرید'
                        current_strength = 70
                        if fib_level in [0.382, 0.618]:
                            current_strength = 80

                    # اگر واکنش به سطح مقاومت بود
                    else:
                        current_signal = 'فروش'
                        current_strength = 70
                        if fib_level in [0.382, 0.618]:
                            current_strength = 80

                    break

        # سیگنال‌های اخیر
        recent_buy = [s for s in buy_signals if last_idx - s['index'] <= 5]
        recent_sell = [s for s in sell_signals if last_idx - s['index'] <= 5]

        if recent_buy:
            current_signal = 'خرید'
            current_strength = max([s['strength'] for s in recent_buy])

        elif recent_sell:
            current_signal = 'فروش'
            current_strength = max([s['strength'] for s in recent_sell])

        # نتایج نهایی
        result = {
            'current_signal': current_signal,
            'current_strength': current_strength,
            'nearest_support': nearest_support,
            'nearest_resistance': nearest_resistance,
            'fibonacci_retracements': fibonacci_retracements,
            'price_reactions': price_reactions,
            'buy_signals': buy_signals,
            'sell_signals': sell_signals,
            'dataframe': df_fib
        }

        return result

    def golden_strategy(self, df):
        """
        استراتژی طلایی: ترکیب بهترین استراتژی‌ها برای دستیابی به بهترین عملکرد با محاسبه واقعی‌تر دقت

        Args:
            df: دیتافریم داده‌ها

        Returns:
            نتایج استراتژی طلایی
        """
        # اجرای تمام استراتژی‌های اصلی
        market_structure_result = self.analyze_market_structure(df, fib_factor=0.273)
        pressure_result = self.analyze_buying_selling_pressure(df)
        whale_result = self.detect_whale_activity(df)
        ema_result = self.analyze_ema_strategy(df)
        rsi_result = self.analyze_rsi_strategy(df)
        divergence_result = self.detect_divergence(df)
        bbrsi_macd_result = self.analyze_bbrsi_macd_strategy(df)
        ichimoku_result = self.analyze_ichimoku(df)
        fibonacci_result = self.analyze_fibonacci(df)
        special_result = self.special_strategy(df)

        # وزن‌دهی به هر استراتژی با بازنگری
        strategy_weights = {
            'market_structure': 0.22,  # وزن بیشتر به دلیل اهمیت ساختار بازار
            'pressure': 0.12,  # فشار خرید/فروش
            'whale': 0.08,  # فعالیت نهنگ‌ها
            'ema': 0.15,  # استراتژی EMA
            'rsi': 0.10,  # استراتژی RSI
            'divergence': 0.10,  # واگرایی
            'bbrsi_macd': 0.09,  # ترکیب باند بولینگر، RSI و MACD
            'ichimoku': 0.08,  # ایچیموکو
            'fibonacci': 0.06  # فیبوناچی (وزن کمتر به دلیل نوسان بیشتر)
        }

        # استخراج سیگنال‌ها و قدرت آن‌ها از هر استراتژی
        signals = {
            'market_structure': {
                'type': 'صعودی' if market_structure_result['market_structure']['trend_score'] > 0 else 'نزولی',
                'strength': abs(market_structure_result['market_structure']['strength']),
                'historical_accuracy': 0.78  # دقت تاریخی استراتژی براساس عملکرد گذشته
            },
            'pressure': {
                'type': pressure_result['pressure_status'],
                'strength': pressure_result['pressure_strength'],
                'historical_accuracy': 0.65
            },
            'whale': {
                'type': whale_result['whale_signal'],
                'strength': whale_result['whale_strength'],
                'historical_accuracy': 0.62
            },
            'ema': {
                'type': ema_result['latest_signal_type'],
                'strength': ema_result['latest_signal_strength'],
                'historical_accuracy': 0.75
            },
            'rsi': {
                'type': rsi_result['current_signal_type'],
                'strength': rsi_result['current_signal_strength'],
                'historical_accuracy': 0.72
            },
            'divergence': {
                'type': divergence_result['signal_type'],
                'strength': divergence_result['current_strength'],
                'historical_accuracy': 0.70
            },
            'bbrsi_macd': {
                'type': bbrsi_macd_result['current_signal'],
                'strength': bbrsi_macd_result['current_strength'],
                'historical_accuracy': 0.68
            },
            'ichimoku': {
                'type': ichimoku_result['current_signal'],
                'strength': ichimoku_result['current_strength'],
                'historical_accuracy': 0.66
            },
            'fibonacci': {
                'type': fibonacci_result['current_signal'],
                'strength': fibonacci_result['current_strength'],
                'historical_accuracy': 0.60
            }
        }

        # محاسبه امتیاز برای سیگنال‌های خرید و فروش
        buy_score = 0
        sell_score = 0
        buy_weights_sum = 0
        sell_weights_sum = 0

        # محاسبه دقت ترکیبی با در نظر گرفتن تاریخچه دقت و وزن‌دهی
        for strategy, signal in signals.items():
            weight = strategy_weights[strategy]
            accuracy = signal['historical_accuracy']

            # تعدیل قدرت سیگنال با توجه به دقت تاریخی
            adjusted_strength = signal['strength'] * accuracy

            if signal['type'] in ['خرید', 'صعودی', 'BUY', 'LONG']:
                buy_score += adjusted_strength * weight
                buy_weights_sum += weight
            elif signal['type'] in ['فروش', 'نزولی', 'SELL', 'SHORT']:
                sell_score += adjusted_strength * weight
                sell_weights_sum += weight

        # نرمال‌سازی امتیازها
        if buy_weights_sum > 0:
            buy_score = buy_score / buy_weights_sum
        if sell_weights_sum > 0:
            sell_score = sell_score / sell_weights_sum

        # فاکتور واقع‌گرایی: با توجه به نوسانات طبیعی بازار، یک ضریب کاهشی اعمال می‌کنیم
        realism_factor = 0.85  # کاهش 15% در دقت نهایی برای واقعی‌تر بودن

        # محاسبه شرایط فعلی بازار (نوسان، حجم و روند)
        market_volatility = df['volatility'].iloc[-10:].mean() if 'volatility' in df.columns else 0
        if market_volatility > 5:  # نوسان بالا
            realism_factor *= 0.92  # کاهش بیشتر دقت در شرایط پرنوسان

        # تعیین سیگنال نهایی
        final_signal = 'خنثی'
        final_strength = 50
        confidence_factor = 0

        if buy_score > sell_score and buy_score >= 65:
            final_signal = 'خرید'
            final_strength = buy_score * realism_factor
            confidence_factor = (buy_score - sell_score) / 10  # اختلاف امتیازها به عنوان فاکتور اطمینان
        elif sell_score > buy_score and sell_score >= 65:
            final_signal = 'فروش'
            final_strength = sell_score * realism_factor
            confidence_factor = (sell_score - buy_score) / 10
        else:
            # در حالت خنثی، قدرت استراتژی به میزان عدم قطعیت بستگی دارد
            final_strength = 50 - abs(buy_score - sell_score) / 2
            final_strength *= realism_factor  # اعمال فاکتور واقع‌گرایی

        # تعدیل نهایی قدرت سیگنال بر اساس تعداد استراتژی‌های همسو
        buy_strategies_count = sum(1 for s in signals.values() if s['type'] in ['خرید', 'صعودی', 'BUY', 'LONG'])
        sell_strategies_count = sum(1 for s in signals.values() if s['type'] in ['فروش', 'نزولی', 'SELL', 'SHORT'])

        # اگر بیش از 80% استراتژی‌ها همسو هستند، دقت را افزایش می‌دهیم
        total_strategies = len(signals)
        if final_signal == 'خرید' and buy_strategies_count >= 0.8 * total_strategies:
            final_strength += 5
        elif final_signal == 'فروش' and sell_strategies_count >= 0.8 * total_strategies:
            final_strength += 5

        # افزایش یا کاهش قدرت براساس میزان اطمینان
        final_strength += confidence_factor

        # محدود کردن قدرت نهایی به محدوده 1-100
        final_strength = max(1, min(100, final_strength))

        # تعیین نقاط ورود، حد ضرر و حد سود
        entry_price = df['close'].iloc[-1]
        stop_loss = 0
        take_profit = 0

        # ترکیب حد ضرر و حد سود از استراتژی‌های مختلف
        if final_signal == 'خرید':
            # برای سیگنال خرید
            stop_losses = []
            take_profits = []

            # استفاده از ساختار بازار
            if 'orderblock_bearish' in market_structure_result and market_structure_result['orderblock_bearish']:
                latest_block = market_structure_result['orderblock_bearish'][-1]
                stop_losses.append(latest_block['low'] * 0.98)

            # استفاده از حد ضرر استراتژی EMA
            if ema_result['entry_points'] and ema_result['entry_points'][-1]['type'] == 'long':
                stop_losses.append(ema_result['entry_points'][-1]['stop_loss'])
                take_profits.append(ema_result['entry_points'][-1]['take_profit'])

            # استفاده از حد ضرر ایچیموکو
            if ichimoku_result['entry_points'] and ichimoku_result['entry_points'][-1]['type'] == 'long':
                stop_losses.append(ichimoku_result['entry_points'][-1]['stop_loss'])
                take_profits.append(ichimoku_result['entry_points'][-1]['take_profit'])

            # استفاده از سطوح فیبوناچی
            if fibonacci_result['nearest_support'] and fibonacci_result['nearest_support'][1] < entry_price:
                stop_losses.append(fibonacci_result['nearest_support'][1] * 0.98)

            # اگر هیچ حد ضرری از استراتژی‌ها نیست، از ATR استفاده می‌کنیم
            if not stop_losses:
                atr = ta.volatility.average_true_range(df['high'], df['low'], df['close'], window=14).iloc[-1]
                stop_losses.append(entry_price - atr * 2)
                take_profits.append(entry_price + atr * 4)

            # انتخاب میانگین حد ضرر و حد سود
            stop_loss = sum(stop_losses) / len(stop_losses)

            # اگر حد سود از استراتژی‌ها تعیین نشده، 2 برابر ریسک در نظر می‌گیریم
            if take_profits:
                take_profit = sum(take_profits) / len(take_profits)
            else:
                take_profit = entry_price + (entry_price - stop_loss) * 2

        elif final_signal == 'فروش':
            # برای سیگنال فروش (کد مشابه شرط بالا)
            stop_losses = []
            take_profits = []

            # استفاده از ساختار بازار
            if 'orderblock_bullish' in market_structure_result and market_structure_result['orderblock_bullish']:
                latest_block = market_structure_result['orderblock_bullish'][-1]
                stop_losses.append(latest_block['high'] * 1.02)

            # استفاده از حد ضرر استراتژی EMA
            if ema_result['entry_points'] and ema_result['entry_points'][-1]['type'] == 'short':
                stop_losses.append(ema_result['entry_points'][-1]['stop_loss'])
                take_profits.append(ema_result['entry_points'][-1]['take_profit'])

            # استفاده از حد ضرر ایچیموکو
            if ichimoku_result['entry_points'] and ichimoku_result['entry_points'][-1]['type'] == 'short':
                stop_losses.append(ichimoku_result['entry_points'][-1]['stop_loss'])
                take_profits.append(ichimoku_result['entry_points'][-1]['take_profit'])

            # استفاده از سطوح فیبوناچی
            if fibonacci_result['nearest_resistance'] and fibonacci_result['nearest_resistance'][1] > entry_price:
                stop_losses.append(fibonacci_result['nearest_resistance'][1] * 1.02)

            # اگر هیچ حد ضرری از استراتژی‌ها نیست، از ATR استفاده می‌کنیم
            if not stop_losses:
                atr = ta.volatility.average_true_range(df['high'], df['low'], df['close'], window=14).iloc[-1]
                stop_losses.append(entry_price + atr * 2)
                take_profits.append(entry_price - atr * 4)

            # انتخاب میانگین حد ضرر و حد سود
            stop_loss = sum(stop_losses) / len(stop_losses)

            # اگر حد سود از استراتژی‌ها تعیین نشده، 2 برابر ریسک در نظر می‌گیریم
            if take_profits:
                take_profit = sum(take_profits) / len(take_profits)
            else:
                take_profit = entry_price - (stop_loss - entry_price) * 2

        # توضیحات استراتژی‌های موافق
        supporting_strategies = {}
        strategy_descriptions = []

        for strategy, info in signals.items():
            if (final_signal == 'خرید' and info['type'] in ['خرید', 'صعودی', 'BUY', 'LONG'] and info[
                'strength'] >= 60) or \
                    (final_signal == 'فروش' and info['type'] in ['فروش', 'نزولی', 'SELL', 'SHORT'] and info[
                        'strength'] >= 60):

                strategy_name = {
                    'market_structure': 'ساختار بازار',
                    'pressure': 'فشار خرید/فروش',
                    'whale': 'فعالیت نهنگ‌ها',
                    'ema': 'میانگین متحرک نمایی',
                    'rsi': 'شاخص قدرت نسبی',
                    'divergence': 'واگرایی',
                    'bbrsi_macd': 'باند بولینگر/RSI/MACD',
                    'ichimoku': 'ایچیموکو',
                    'fibonacci': 'فیبوناچی'
                }

                # محاسبه قدرت واقعی استراتژی با اعمال دقت تاریخی
                adjusted_strength = info['strength'] * info['historical_accuracy']
                supporting_strategies[strategy_name[strategy]] = min(100, adjusted_strength)

                # اضافه کردن توضیحات استراتژی
                if strategy == 'market_structure' and final_signal == 'خرید':
                    strategy_descriptions.append(f"ساختار بازار صعودی با شکست ساختاری و بلوک سفارش")
                elif strategy == 'market_structure' and final_signal == 'فروش':
                    strategy_descriptions.append(f"ساختار بازار نزولی با شکست ساختاری و بلوک سفارش")
                elif strategy == 'ema':
                    strategy_descriptions.append(f"استراتژی EMA: {ema_result['current_status']}")
                elif strategy == 'rsi' and 'current_status' in rsi_result:
                    strategy_descriptions.append(f"RSI در وضعیت {rsi_result['current_status']}")
                elif strategy == 'ichimoku' and 'current_status' in ichimoku_result:
                    strategy_descriptions.append(f"ایچیموکو: {ichimoku_result['current_status']}")

        # اضافه کردن توضیح واقع‌گرایی
        strategy_descriptions.append(f"دقت سیگنال با توجه به شرایط بازار و تاریخچه استراتژی‌ها محاسبه شده است")
        if market_volatility > 5:
            strategy_descriptions.append(f"نوسان بازار بالاست - دقت سیگنال کاهش یافته")

        # نتایج نهایی
        result = {
            'signal': final_signal,
            'strength': round(final_strength, 1),  # گرد کردن به یک رقم اعشار برای واقعی‌تر بودن
            'entry_price': entry_price,
            'stop_loss': stop_loss,
            'take_profit': take_profit,
            'supporting_strategies': supporting_strategies,
            'strategy_descriptions': strategy_descriptions,
            'risk_reward_ratio': abs(
                (take_profit - entry_price) / (entry_price - stop_loss)) if stop_loss != entry_price else 0,
            'buy_score': buy_score,
            'sell_score': sell_score,
            'signals': signals,
            'market_conditions': {
                'volatility': market_volatility,
                'confidence_factor': confidence_factor,
                'realism_factor': realism_factor
            }
        }

        return result

    def special_strategy(self, df):
        """
        استراتژی ویژه: ترکیب چندین اندیکاتور پیشرفته برای شناسایی بهترین فرصت‌های معاملاتی

        Args:
            df: دیتافریم داده‌ها

        Returns:
            نتایج استراتژی ویژه
        """
        # ایجاد کپی از دیتافریم
        df_special = df.copy()

        # محاسبه اندیکاتورهای پیشرفته
        # 1. Super Trend
        atr_period = 10
        atr_multiplier = 3.0

        # محاسبه ATR
        df_special['atr'] = ta.volatility.average_true_range(df_special['high'], df_special['low'], df_special['close'],
                                                             window=atr_period)

        # محاسبه باند‌های بالا و پایین
        df_special['upperband'] = ((df_special['high'] + df_special['low']) / 2) + (atr_multiplier * df_special['atr'])
        df_special['lowerband'] = ((df_special['high'] + df_special['low']) / 2) - (atr_multiplier * df_special['atr'])

        # تعیین روند با استفاده از Super Trend
        df_special['in_uptrend'] = True
        for i in range(1, len(df_special)):
            current_close = df_special['close'].iloc[i]
            prev_close = df_special['close'].iloc[i - 1]

            prev_upperband = df_special['upperband'].iloc[i - 1]
            prev_lowerband = df_special['lowerband'].iloc[i - 1]

            current_upperband = df_special['upperband'].iloc[i]
            current_lowerband = df_special['lowerband'].iloc[i]

            prev_in_uptrend = df_special['in_uptrend'].iloc[i - 1]

            if prev_in_uptrend:
                # اگر قبلاً در روند صعودی بودیم
                if current_close < current_lowerband:
                    df_special.loc[df_special.index[i], 'in_uptrend'] = False
                else:
                    df_special.loc[df_special.index[i], 'in_uptrend'] = True
            else:
                # اگر قبلاً در روند نزولی بودیم
                if current_close > current_upperband:
                    df_special.loc[df_special.index[i], 'in_uptrend'] = True
                else:
                    df_special.loc[df_special.index[i], 'in_uptrend'] = False

        # 2. Williams Vix Fix - شناسایی ترس و طمع بازار
        period = 22
        stddev = 2.0

        # محاسبه پایین‌ترین کف در دوره
        df_special['lowest_low'] = df_special['low'].rolling(window=period).min()

        # محاسبه بالاترین بالا در دوره
        df_special['highest_high'] = df_special['high'].rolling(window=period).max()

        # محاسبه Williams Vix Fix
        df_special['williams_vix'] = ((df_special['highest_high'] - df_special['close']) /
                                      (df_special['highest_high'] - df_special['lowest_low'])) * 100

        # میانگین متحرک Williams Vix Fix
        df_special['williams_vix_ma'] = df_special['williams_vix'].rolling(window=9).mean()

        # شناسایی نقاط اوج و حضیض Williams Vix Fix
        df_special['williams_vix_high'] = df_special['williams_vix'].rolling(window=10, center=True).max()
        df_special['williams_vix_low'] = df_special['williams_vix'].rolling(window=10, center=True).min()

        # 3. TTM Squeeze - شناسایی فشردگی نوسان و انفجار قیمت
        len_bb = 20
        len_kc = 20
        mult_bb = 2.0
        mult_kc = 1.5

        # محاسبه باند بولینگر
        df_special['bb_upper'] = ta.volatility.bollinger_hband(df_special['close'], window=len_bb, window_dev=mult_bb)
        df_special['bb_middle'] = ta.volatility.bollinger_mavg(df_special['close'], window=len_bb)
        df_special['bb_lower'] = ta.volatility.bollinger_lband(df_special['close'], window=len_bb, window_dev=mult_bb)

        # محاسبه کانال کلتنر
        df_special['kc_middle'] = df_special['close'].rolling(window=len_kc).mean()
        df_special['kc_range'] = (df_special['high'] - df_special['low']).rolling(window=len_kc).mean()
        df_special['kc_upper'] = df_special['kc_middle'] + mult_kc * df_special['kc_range']
        df_special['kc_lower'] = df_special['kc_middle'] - mult_kc * df_special['kc_range']

        # شناسایی فشردگی (Squeeze)
        df_special['squeeze_on'] = ((df_special['bb_lower'] > df_special['kc_lower']) &
                                    (df_special['bb_upper'] < df_special['kc_upper']))

        # شناسایی انفجار (Firing)
        df_special['squeeze_off'] = ((df_special['squeeze_on'].shift(1) == True) &
                                     (df_special['squeeze_on'] == False))

        # محاسبه مومنتوم برای تعیین جهت پس از انفجار
        df_special['momentum'] = df_special['close'] - df_special['close'].rolling(window=len_bb).mean()
        df_special['momentum_ma'] = df_special['momentum'].rolling(window=9).mean()

        # ترکیب استراتژی‌ها برای شناسایی سیگنال‌های قوی

        # سیگنال‌های خرید قوی:
        # 1. Super Trend تغییر به روند صعودی + TTM Squeeze پایان فشردگی + Williams Vix Fix در نقطه اوج
        df_special['strong_buy'] = (
                (df_special['in_uptrend'] == True) &
                (df_special['in_uptrend'].shift(1) == False) &
                (df_special['squeeze_off'] == True) &
                (df_special['momentum'] > 0) &
                (df_special['williams_vix'] > 80)
        )

        # 2. Super Trend در روند صعودی + TTM Squeeze در حال فشردگی + نزدیک شدن به کف Williams Vix Fix
        df_special['moderate_buy'] = (
                (df_special['in_uptrend'] == True) &
                (df_special['squeeze_on'] == True) &
                (df_special['williams_vix'] < 30) &
                (df_special['williams_vix'] < df_special['williams_vix'].shift(1))
        )

        # سیگنال‌های فروش قوی:
        # 1. Super Trend تغییر به روند نزولی + TTM Squeeze پایان فشردگی + Williams Vix Fix در نقطه حضیض
        df_special['strong_sell'] = (
                (df_special['in_uptrend'] == False) &
                (df_special['in_uptrend'].shift(1) == True) &
                (df_special['squeeze_off'] == True) &
                (df_special['momentum'] < 0) &
                (df_special['williams_vix'] < 20)
        )

        # 2. Super Trend در روند نزولی + TTM Squeeze در حال فشردگی + نزدیک شدن به اوج Williams Vix Fix
        df_special['moderate_sell'] = (
                (df_special['in_uptrend'] == False) &
                (df_special['squeeze_on'] == True) &
                (df_special['williams_vix'] > 70) &
                (df_special['williams_vix'] > df_special['williams_vix'].shift(1))
        )

        # جمع‌آوری سیگنال‌ها
        buy_signals = []
        sell_signals = []

        for i in range(max(len_bb, len_kc, period) + 10, len(df_special)):
            # سیگنال‌های خرید
            if df_special['strong_buy'].iloc[i]:
                buy_signals.append({
                    'index': i,
                    'date': df_special.index[i],
                    'price': df_special['close'].iloc[i],
                    'strength': 90,
                    'description': 'سیگنال خرید قوی: Super Trend صعودی + TTM Squeeze پایان فشردگی + Williams Vix Fix در نقطه اوج',
                    'type': 'strong_buy'
                })
            elif df_special['moderate_buy'].iloc[i]:
                buy_signals.append({
                    'index': i,
                    'date': df_special.index[i],
                    'price': df_special['close'].iloc[i],
                    'strength': 75,
                    'description': 'سیگنال خرید متوسط: Super Trend صعودی + TTM Squeeze در حال فشردگی + کاهش Williams Vix Fix',
                    'type': 'moderate_buy'
                })

            # سیگنال‌های فروش
            if df_special['strong_sell'].iloc[i]:
                sell_signals.append({
                    'index': i,
                    'date': df_special.index[i],
                    'price': df_special['close'].iloc[i],
                    'strength': 90,
                    'description': 'سیگنال فروش قوی: Super Trend نزولی + TTM Squeeze پایان فشردگی + Williams Vix Fix در نقطه حضیض',
                    'type': 'strong_sell'
                })
            elif df_special['moderate_sell'].iloc[i]:
                sell_signals.append({
                    'index': i,
                    'date': df_special.index[i],
                    'price': df_special['close'].iloc[i],
                    'strength': 75,
                    'description': 'سیگنال فروش متوسط: Super Trend نزولی + TTM Squeeze در حال فشردگی + افزایش Williams Vix Fix',
                    'type': 'moderate_sell'
                })

        # تعیین نقاط ورود و خروج
        entry_points = []

        for signal in buy_signals:
            # محاسبه حد ضرر و حد سود برای سیگنال‌های خرید
            i = signal['index']
            price = signal['price']

            # استفاده از Super Trend برای حد ضرر
            stop_loss = df_special['lowerband'].iloc[i]

            # حد سود: نسبت ریسک به ریوارد 1:2
            risk = price - stop_loss
            take_profit = price + (risk * 2)

            entry_points.append({
                'index': i,
                'price': price,
                'stop_loss': stop_loss,
                'take_profit': take_profit,
                'type': 'long',
                'strength': signal['strength']
            })

        for signal in sell_signals:
            # محاسبه حد ضرر و حد سود برای سیگنال‌های فروش
            i = signal['index']
            price = signal['price']

            # استفاده از Super Trend برای حد ضرر
            stop_loss = df_special['upperband'].iloc[i]

            # حد سود: نسبت ریسک به ریوارد 1:2
            risk = stop_loss - price
            take_profit = price - (risk * 2)

            entry_points.append({
                'index': i,
                'price': price,
                'stop_loss': stop_loss,
                'take_profit': take_profit,
                'type': 'short',
                'strength': signal['strength']
            })

        # بررسی وضعیت فعلی
        current_signal = 'خنثی'
        current_strength = 50
        last_idx = len(df_special) - 1

        # بررسی روند فعلی Super Trend
        if df_special['in_uptrend'].iloc[last_idx]:
            if df_special['in_uptrend'].iloc[last_idx - 1] == False:
                # تغییر روند به صعودی
                current_signal = 'خرید'
                current_strength = 80
            else:
                # ادامه روند صعودی
                current_signal = 'خرید'
                current_strength = 60
        else:
            if df_special['in_uptrend'].iloc[last_idx - 1] == True:
                # تغییر روند به نزولی
                current_signal = 'فروش'
                current_strength = 80
            else:
                # ادامه روند نزولی
                current_signal = 'فروش'
                current_strength = 60

        # بررسی TTM Squeeze
        if df_special['squeeze_off'].iloc[last_idx]:
            # فشردگی به پایان رسیده و قیمت در حال حرکت است
            if df_special['momentum'].iloc[last_idx] > 0:
                current_signal = 'خرید'
                current_strength += 15
            else:
                current_signal = 'فروش'
                current_strength += 15
        elif df_special['squeeze_on'].iloc[last_idx]:
            # در حال فشردگی هستیم - قیمت احتمالاً در محدوده معاملاتی است
            current_strength -= 10

        # بررسی Williams Vix Fix
        williams_vix_current = df_special['williams_vix'].iloc[last_idx]
        if williams_vix_current > 80:
            # سطح بالای ترس - احتمال برگشت صعودی
            if current_signal == 'خرید':
                current_strength += 10
        elif williams_vix_current < 20:
            # سطح پایین ترس - احتمال برگشت نزولی
            if current_signal == 'فروش':
                current_strength += 10

        # بررسی سیگنال‌های اخیر
        recent_buy = [s for s in buy_signals if last_idx - s['index'] <= 3]
        recent_sell = [s for s in sell_signals if last_idx - s['index'] <= 3]

        if recent_buy:
            current_signal = 'خرید'
            current_strength = max([s['strength'] for s in recent_buy])
        elif recent_sell:
            current_signal = 'فروش'
            current_strength = max([s['strength'] for s in recent_sell])

        # توضیحات استراتژی
        strategy_description = ""

        if current_signal == 'خرید':
            if df_special['in_uptrend'].iloc[last_idx] and df_special['in_uptrend'].iloc[last_idx - 1] == False:
                strategy_description += "تغییر Super Trend به روند صعودی. "

            if df_special['squeeze_off'].iloc[last_idx] and df_special['momentum'].iloc[last_idx] > 0:
                strategy_description += "TTM Squeeze: پایان فشردگی با مومنتوم مثبت. "

            if df_special['williams_vix'].iloc[last_idx] > 80:
                strategy_description += "Williams Vix Fix در سطح بالا - احتمال برگشت صعودی. "

        elif current_signal == 'فروش':
            if df_special['in_uptrend'].iloc[last_idx] == False and df_special['in_uptrend'].iloc[last_idx - 1]:
                strategy_description += "تغییر Super Trend به روند نزولی. "

            if df_special['squeeze_off'].iloc[last_idx] and df_special['momentum'].iloc[last_idx] < 0:
                strategy_description += "TTM Squeeze: پایان فشردگی با مومنتوم منفی. "

            if df_special['williams_vix'].iloc[last_idx] < 20:
                strategy_description += "Williams Vix Fix در سطح پایین - احتمال برگشت نزولی. "

        # نتایج نهایی
        result = {
            'current_signal': current_signal,
            'current_strength': min(100, current_strength),
            'strategy_description': strategy_description,
            'buy_signals': buy_signals,
            'sell_signals': sell_signals,
            'entry_points': entry_points,
            'dataframe': df_special
        }

        return result

    def run_analysis(self, symbol, timeframes=None):
        """
        اجرای تحلیل بر روی یک ارز و تایم‌فریم‌های مشخص با پشتیبانی از سیگنال‌های شرطی

        Args:
            symbol: نماد ارز
            timeframes: لیست تایم‌فریم‌ها (اختیاری)

        Returns:
            نتایج تحلیل
        """
        if timeframes is None:
            timeframes = ['15m', '1h', '4h', '1d']

        results = {}

        for timeframe in timeframes:
            logger.info(f"🔍 در حال تحلیل {symbol} در تایم‌فریم {timeframe}...")

            # بررسی زمان آخرین تحلیل
            last_time = get_last_analysis_time(symbol, timeframe)
            current_time = time.time()

            # اگر اخیراً این ارز را تحلیل کرده‌ایم، از کش استفاده می‌کنیم
            if last_time and current_time - last_time < 1800:  # 30 دقیقه
                logger.info(f"🕑 {symbol} در تایم‌فریم {timeframe} اخیراً تحلیل شده است. از نتایج قبلی استفاده می‌شود.")
                continue

            # دریافت داده‌ها
            df = force_latest_data(symbol, timeframe)
            if df is None:
                logger.warning(f"دریافت اجباری داده برای {symbol}/{timeframe} ناموفق بود - استفاده از روش جایگزین")
                df = fetch_data(symbol, timeframe, limit=1000, max_retries=5, force_fresh=True)

            if df is None or len(df) < 100:
                logger.error(f"❌ دریافت داده برای {symbol} در تایم‌فریم {timeframe} ناموفق بود")
                continue

            # اجرای تمام استراتژی‌ها
            try:
                # تحلیل ساختار بازار و بلوک‌های سفارش
                market_structure_result = self.analyze_market_structure(df, fib_factor=0.273)

                # تحلیل فشار خرید/فروش
                pressure_result = self.analyze_buying_selling_pressure(df)

                # تحلیل فعالیت نهنگ‌ها
                whale_result = self.detect_whale_activity(df)

                # تحلیل استراتژی EMA
                ema_result = self.analyze_ema_strategy(df)

                # تحلیل استراتژی RSI
                rsi_result = self.analyze_rsi_strategy(df)

                # تحلیل واگرایی
                divergence_result = self.detect_divergence(df)

                # تحلیل ترکیبی باند بولینگر، RSI و MACD
                bbrsi_macd_result = self.analyze_bbrsi_macd_strategy(df)

                # تحلیل جعبه دارواس
                darvas_result = self.analyze_darvas_box(df)

                # تحلیل ایچیموکو
                ichimoku_result = self.analyze_ichimoku(df)

                # تحلیل فیبوناچی
                fibonacci_result = self.analyze_fibonacci(df)

                # استراتژی طلایی
                golden_result = self.golden_strategy(df)

                # استراتژی ویژه
                special_result = self.special_strategy(df)

                # ثبت زمان تحلیل
                track_last_analysis_time(symbol, timeframe)

                # ======= بررسی سیگنال‌های شرطی براساس ساختار بازار =======
                if 'entry_points' in market_structure_result and market_structure_result['entry_points']:
                    for entry_point in market_structure_result['entry_points']:
                        # تعیین نوع سیگنال
                        signal_type = "خرید" if entry_point['type'].startswith('conditional_long') else "فروش"

                        # آماده کردن حدهای سود به صورت لیست
                        take_profit_levels = [
                            entry_point['take_profit1'],
                            entry_point['take_profit2'],
                            entry_point['take_profit3']
                        ]

                        # ترکیب استراتژی‌های تأییدکننده براساس تطابق سیگنال‌های مختلف
                        supporting_strategies = {
                            "ساختار بازار": entry_point['strength']
                        }

                        # اضافه کردن استراتژی‌های دیگر در صورت تطابق سیگنال
                        if signal_type == "خرید":
                            if pressure_result['pressure_status'] in ['خرید', 'BUY']:
                                supporting_strategies["فشار خرید/فروش"] = pressure_result['pressure_strength']

                            if whale_result['whale_signal'] in ['خرید', 'BUY']:
                                supporting_strategies["فعالیت نهنگ‌ها"] = whale_result['whale_strength']

                            if ema_result['latest_signal_type'] in ['خرید', 'صعودی']:
                                supporting_strategies["میانگین متحرک نمایی"] = ema_result['latest_signal_strength']

                            if rsi_result['current_signal_type'] in ['خرید', 'صعودی']:
                                supporting_strategies["شاخص قدرت نسبی"] = rsi_result['current_signal_strength']

                            if divergence_result['signal_type'] in ['خرید', 'صعودی']:
                                supporting_strategies["واگرایی"] = divergence_result['current_strength']

                            if bbrsi_macd_result['current_signal'] in ['خرید', 'صعودی']:
                                supporting_strategies["ترکیبی BB/RSI/MACD"] = bbrsi_macd_result['current_strength']

                        elif signal_type == "فروش":
                            if pressure_result['pressure_status'] in ['فروش', 'SELL']:
                                supporting_strategies["فشار خرید/فروش"] = pressure_result['pressure_strength']

                            if whale_result['whale_signal'] in ['فروش', 'SELL']:
                                supporting_strategies["فعالیت نهنگ‌ها"] = whale_result['whale_strength']

                            if ema_result['latest_signal_type'] in ['فروش', 'نزولی']:
                                supporting_strategies["میانگین متحرک نمایی"] = ema_result['latest_signal_strength']

                            if rsi_result['current_signal_type'] in ['فروش', 'نزولی']:
                                supporting_strategies["شاخص قدرت نسبی"] = rsi_result['current_signal_strength']

                            if divergence_result['signal_type'] in ['فروش', 'نزولی']:
                                supporting_strategies["واگرایی"] = divergence_result['current_strength']

                            if bbrsi_macd_result['current_signal'] in ['فروش', 'نزولی']:
                                supporting_strategies["ترکیبی BB/RSI/MACD"] = bbrsi_macd_result['current_strength']

                        # محاسبه نسبت ریسک به ریوارد
                        if signal_type == "خرید":
                            risk = entry_point['price'] - entry_point['stop_loss']
                            reward = entry_point['take_profit2'] - entry_point['price']
                            risk_reward_ratio = reward / risk if risk > 0 else 0
                        else:  # فروش
                            risk = entry_point['stop_loss'] - entry_point['price']
                            reward = entry_point['price'] - entry_point['take_profit2']
                            risk_reward_ratio = reward / risk if risk > 0 else 0

                        # ساخت توضیحات مفصل
                        description = f"سیگنال {signal_type} براساس تحلیل ساختار بازار و بلوک‌های سفارش\n\n"

                        # اضافه کردن جزئیات بیشتر براساس نوع سیگنال
                        if signal_type == "خرید":
                            description += f"• معامله در ناحیه بلوک سفارش صعودی\n"
                            description += f"• نسبت ریسک به ریوارد: {risk_reward_ratio:.2f}\n"
                            description += f"• وضعیت ساختار بازار: {market_structure_result['market_structure']['status']}\n"

                            # اضافه کردن روند قیمت فعلی نسبت به EMA
                            if 'current_status' in ema_result:
                                description += f"• روند EMA: {ema_result['current_status']}\n"

                            # اضافه کردن وضعیت RSI
                            if 'current_status' in rsi_result:
                                description += f"• وضعیت RSI: {rsi_result['current_status']}\n"

                            # استراتژی مدیریت سرمایه پیشنهادی
                            description += f"\nاستراتژی معاملاتی پیشنهادی:\n"
                            description += f"• ورود: زمانی که قیمت به ناحیه {entry_point['price']:.6f} برسد\n"
                            description += f"• حد ضرر: {entry_point['stop_loss']:.6f} ({(entry_point['stop_loss'] / entry_point['price'] - 1) * 100:.2f}%)\n"
                            description += f"• خروج اول: {entry_point['take_profit1']:.6f} ({(entry_point['take_profit1'] / entry_point['price'] - 1) * 100:.2f}%)\n"
                            description += f"• خروج دوم: {entry_point['take_profit2']:.6f} ({(entry_point['take_profit2'] / entry_point['price'] - 1) * 100:.2f}%)\n"
                            description += f"• خروج سوم: {entry_point['take_profit3']:.6f} ({(entry_point['take_profit3'] / entry_point['price'] - 1) * 100:.2f}%)\n"

                        else:  # فروش
                            description += f"• معامله در ناحیه بلوک سفارش نزولی\n"
                            description += f"• نسبت ریسک به ریوارد: {risk_reward_ratio:.2f}\n"
                            description += f"• وضعیت ساختار بازار: {market_structure_result['market_structure']['status']}\n"

                            # اضافه کردن روند قیمت فعلی نسبت به EMA
                            if 'current_status' in ema_result:
                                description += f"• روند EMA: {ema_result['current_status']}\n"

                            # اضافه کردن وضعیت RSI
                            if 'current_status' in rsi_result:
                                description += f"• وضعیت RSI: {rsi_result['current_status']}\n"

                            # استراتژی مدیریت سرمایه پیشنهادی
                            description += f"\nاستراتژی معاملاتی پیشنهادی:\n"
                            description += f"• ورود: زمانی که قیمت به ناحیه {entry_point['price']:.6f} برسد\n"
                            description += f"• حد ضرر: {entry_point['stop_loss']:.6f} ({(entry_point['stop_loss'] / entry_point['price'] - 1) * 100:.2f}%)\n"
                            description += f"• خروج اول: {entry_point['take_profit1']:.6f} ({(entry_point['take_profit1'] / entry_point['price'] - 1) * 100:.2f}%)\n"
                            description += f"• خروج دوم: {entry_point['take_profit2']:.6f} ({(entry_point['take_profit2'] / entry_point['price'] - 1) * 100:.2f}%)\n"
                            description += f"• خروج سوم: {entry_point['take_profit3']:.6f} ({(entry_point['take_profit3'] / entry_point['price'] - 1) * 100:.2f}%)\n"

                        # محاسبه اطمینان کلی براساس تعداد استراتژی‌های تأییدکننده
                        total_confidence = sum(supporting_strategies.values()) / len(supporting_strategies)

                        # رسم نمودار با تمام جزئیات و نمایش استراتژی‌های پیش‌بینی
                        chart_path = self.create_candlestick_chart(
                            df, symbol, timeframe,
                            strategies=supporting_strategies,
                            indicators={
                                'ema': True,  # نمایش EMA
                                'rsi': True,  # نمایش RSI
                                'macd': True,  # نمایش MACD
                                'bollinger': 'ترکیبی BB/RSI/MACD' in supporting_strategies,
                                'ichimoku': 'ایچیموکو' in supporting_strategies,
                                'fibonacci': 'فیبوناچی' in supporting_strategies
                            },
                            patterns={
                                'market_structure_break_bullish': market_structure_result.get('msb_bullish', []),
                                'market_structure_break_bearish': market_structure_result.get('msb_bearish', []),
                                'order_block_bullish': market_structure_result.get('orderblock_bullish', []),
                                'order_block_bearish': market_structure_result.get('orderblock_bearish', [])
                            },
                            entry_exit={
                                'entries': [
                                    {'index': entry_point.get('index', len(df) - 1), 'price': entry_point['price']}],
                                'stop_loss': entry_point['stop_loss'],
                                'take_profit': take_profit_levels
                            },
                            title=f"{symbol} - {timeframe} - سیگنال شرطی {signal_type} ({int(total_confidence)}% اطمینان)"
                        )

                        # ارسال سیگنال شرطی
                        self.send_conditional_signal(
                            symbol=symbol,
                            timeframe=timeframe,
                            signal_type=signal_type,
                            entry_condition=entry_point.get('condition', f"قیمت به {entry_point['price']:.6f} برسد"),
                            entry_price=entry_point['price'],
                            stop_loss=entry_point['stop_loss'],
                            take_profit_levels=take_profit_levels,
                            strategies=supporting_strategies,
                            confidence=total_confidence,
                            description=description,
                            chart_path=chart_path
                        )
                # ======= بررسی سیگنال‌های استراتژی طلایی و ارسال پیام تأییدیه =======
                if golden_result['signal'] in ['خرید', 'فروش'] and golden_result['strength'] >= BOT_SETTINGS[
                    'signal_confidence_threshold']:
                    # استراتژی‌های تأییدکننده
                    gold_strategies = golden_result['supporting_strategies']

                    # ایجاد نمودار
                    gold_chart_path = self.create_candlestick_chart(
                        df, symbol, timeframe,
                        strategies=gold_strategies,
                        indicators={
                            'ema': True,
                            'rsi': True,
                            'macd': True,
                            'bollinger': True,
                            'ichimoku': ('ایچیموکو' in gold_strategies),
                            'fibonacci': ('فیبوناچی' in gold_strategies)
                        },
                        patterns={
                            'market_structure_break_bullish': market_structure_result.get('msb_bullish', []),
                            'market_structure_break_bearish': market_structure_result.get('msb_bearish', []),
                            'order_block_bullish': market_structure_result.get('orderblock_bullish', []),
                            'order_block_bearish': market_structure_result.get('orderblock_bearish', [])
                        },
                        entry_exit={
                            'entries': [{'index': len(df) - 1, 'price': golden_result['entry_price']}],
                            'stop_loss': golden_result['stop_loss'],
                            'take_profit': golden_result['take_profit']
                        },
                        title=f"{symbol} - {timeframe} - ⭐️ تأییدیه نهایی ربات ⭐️ {golden_result['signal']} ({int(golden_result['strength'])}% اطمینان)"
                    )

                    # ساخت پیام تأییدیه
                    confirmation_message = f"⭐️ <b>تأییدیه نهایی ربات | {symbol} | {timeframe}</b> ⭐️\n\n"
                    confirmation_message += f"🤖 <b>تحلیل هوشمند ربات:</b> سیگنال {golden_result['signal']} تأیید می‌شود\n"
                    confirmation_message += f"⚡️ <b>درصد اطمینان:</b> {golden_result['strength']:.1f}%\n"
                    confirmation_message += f"💵 <b>قیمت ورود:</b> {golden_result['entry_price']:.6f}\n"
                    confirmation_message += f"🛑 <b>حد ضرر:</b> {golden_result['stop_loss']:.6f}\n"
                    confirmation_message += f"🎯 <b>حد سود:</b> {golden_result['take_profit']:.6f}\n\n"

                    confirmation_message += "<b>استراتژی‌های تأییدکننده:</b>\n"
                    for strategy, conf in gold_strategies.items():
                        emoji = "✅" if conf >= 80 else "⚠️"
                        confirmation_message += f"{emoji} {strategy}: {conf}%\n"

                    if golden_result['strategy_descriptions']:
                        confirmation_message += f"\n<b>توضیحات:</b>\n"
                        for desc in golden_result['strategy_descriptions']:
                            confirmation_message += f"• {desc}\n"

                    confirmation_message += f"\n⚠️ <b>توجه:</b> این پیام تأییدیه نهایی ربات است. سایر سیگنال‌ها صرفاً جهت راهنمایی ارسال می‌شوند.\n"
                    confirmation_message += f"❗️ <i>این سیگنال‌ها صرفاً جنبه آموزشی داشته و توصیه سرمایه‌گذاری نیست.</i>"

                    # ارسال پیام تأییدیه
                    self.send_telegram_message(confirmation_message, gold_chart_path)

                # ======= بررسی سیگنال‌های استراتژی طلایی =======
                if golden_result['signal'] in ['خرید', 'فروش'] and golden_result['strength'] >= BOT_SETTINGS[
                    'signal_confidence_threshold']:
                    # ساخت استراتژی‌های تأییدکننده
                    gold_strategies = {}
                    for strategy in golden_result['supporting_strategies']:
                        gold_strategies[strategy['name']] = strategy['strength']

                    # ایجاد نمودار
                    gold_chart_path = self.create_candlestick_chart(
                        df, symbol, timeframe,
                        strategies=gold_strategies,
                        indicators={
                            'ema': True,
                            'rsi': True,
                            'macd': True,
                            'bollinger': True,
                            'ichimoku': ('ایچیموکو' in [s['name'] for s in golden_result['supporting_strategies']]),
                            'fibonacci': ('فیبوناچی' in [s['name'] for s in golden_result['supporting_strategies']])
                        },
                        patterns={
                            'market_structure_break_bullish': market_structure_result.get('msb_bullish', []),
                            'market_structure_break_bearish': market_structure_result.get('msb_bearish', []),
                            'order_block_bullish': market_structure_result.get('orderblock_bullish', []),
                            'order_block_bearish': market_structure_result.get('orderblock_bearish', [])
                        },
                        entry_exit={
                            'entries': [{'index': len(df) - 1, 'price': golden_result['entry_price']}],
                            'stop_loss': golden_result['stop_loss'],
                            'take_profit': golden_result['take_profit']
                        },
                        title=f"{symbol} - {timeframe} - استراتژی طلایی ({int(golden_result['strength'])}% اطمینان)"
                    )

                    # ارسال سیگنال
                    self.send_signal(
                        symbol, timeframe, golden_result['signal'], golden_result['entry_price'],
                        golden_result['stop_loss'], golden_result['take_profit'],
                        gold_strategies, golden_result['strength'],
                        "\n".join(golden_result['strategy_descriptions']),
                        gold_chart_path
                    )

                # ======= بررسی سیگنال‌های استراتژی دارواس =======
                if 'current_signal' in darvas_result and darvas_result['current_signal'] in ['خرید', 'فروش'] and \
                        darvas_result['current_strength'] >= BOT_SETTINGS['signal_confidence_threshold']:
                    # ساخت استراتژی‌های تأییدکننده
                    darvas_strategies = {
                        "جعبه دارواس": darvas_result['current_strength']
                    }

                    # اضافه کردن استراتژی‌های همسو
                    if darvas_result['current_signal'] == 'خرید':
                        if ema_result['latest_signal_type'] in ['خرید', 'صعودی']:
                            darvas_strategies["میانگین متحرک نمایی"] = ema_result['latest_signal_strength']
                        if rsi_result['current_signal_type'] in ['خرید', 'صعودی']:
                            darvas_strategies["شاخص قدرت نسبی"] = rsi_result['current_signal_strength']
                        if bbrsi_macd_result['current_signal'] in ['خرید', 'صعودی']:
                            darvas_strategies["ترکیبی BB/RSI/MACD"] = bbrsi_macd_result['current_strength']
                    elif darvas_result['current_signal'] == 'فروش':
                        if ema_result['latest_signal_type'] in ['فروش', 'نزولی']:
                            darvas_strategies["میانگین متحرک نمایی"] = ema_result['latest_signal_strength']
                        if rsi_result['current_signal_type'] in ['فروش', 'نزولی']:
                            darvas_strategies["شاخص قدرت نسبی"] = rsi_result['current_signal_strength']
                        if bbrsi_macd_result['current_signal'] in ['فروش', 'نزولی']:
                            darvas_strategies["ترکیبی BB/RSI/MACD"] = bbrsi_macd_result['current_strength']

                    # تعیین قیمت ورود، حد ضرر و حد سود
                    entry_price = df['close'].iloc[-1]

                    # استفاده از نقاط ورود دارواس اگر موجود باشند
                    stop_loss = entry_price * 0.95  # پیش‌فرض: 5% زیر قیمت ورود
                    take_profit = entry_price * 1.10  # پیش‌فرض: 10% بالای قیمت ورود

                    if 'entry_points' in darvas_result and darvas_result['entry_points'] and len(
                            darvas_result['entry_points']) > 0:
                        latest_entry = darvas_result['entry_points'][-1]
                        if latest_entry['type'] == 'long' and darvas_result['current_signal'] == 'خرید':
                            entry_price = latest_entry['price']
                            stop_loss = latest_entry['stop_loss']
                            take_profit = latest_entry['take_profit']
                        elif latest_entry['type'] == 'short' and darvas_result['current_signal'] == 'فروش':
                            entry_price = latest_entry['price']
                            stop_loss = latest_entry['stop_loss']
                            take_profit = latest_entry['take_profit']

                    # ایجاد توضیحات
                    description = f"سیگنال {darvas_result['current_signal']} براساس استراتژی جعبه دارواس\n\n"

                    if 'darvas_boxes' in darvas_result and darvas_result['darvas_boxes']:
                        latest_box = darvas_result['darvas_boxes'][-1]
                        description += f"• جعبه دارواس شناسایی شده در قیمت {latest_box.get('breakout_price', entry_price):.6f}\n"
                        description += f"• جهت شکست: {latest_box.get('breakout_direction', 'نامشخص')}\n"

                    description += f"• سطح اطمینان: {darvas_result['current_strength']}%\n"
                    description += f"\nاستراتژی معاملاتی پیشنهادی:\n"
                    description += f"• ورود: {entry_price:.6f}\n"
                    description += f"• حد ضرر: {stop_loss:.6f}\n"
                    description += f"• حد سود: {take_profit:.6f}\n"

                    # ایجاد نمودار
                    darvas_chart_path = self.create_candlestick_chart(
                        df, symbol, timeframe,
                        strategies=darvas_strategies,
                        indicators={
                            'ema': True,
                            'rsi': True,
                            'macd': True,
                            'bollinger': True
                        },
                        patterns={
                            'market_structure_break_bullish': market_structure_result.get('msb_bullish', []),
                            'market_structure_break_bearish': market_structure_result.get('msb_bearish', []),
                            'order_block_bullish': market_structure_result.get('orderblock_bullish', []),
                            'order_block_bearish': market_structure_result.get('orderblock_bearish', [])
                        },
                        entry_exit={
                            'entries': [{'index': len(df) - 1, 'price': entry_price}],
                            'stop_loss': stop_loss,
                            'take_profit': take_profit
                        },
                        title=f"{symbol} - {timeframe} - استراتژی دارواس ({int(darvas_result['current_strength'])}% اطمینان)"
                    )

                    # ارسال سیگنال
                    self.send_signal(
                        symbol, timeframe, darvas_result['current_signal'], entry_price,
                        stop_loss, take_profit,
                        darvas_strategies, darvas_result['current_strength'],
                        description,
                        darvas_chart_path
                    )

                # ======= بررسی سیگنال‌های استراتژی ویژه =======
                if 'current_signal' in special_result and special_result['current_signal'] in ['خرید', 'فروش'] and \
                        special_result['current_strength'] >= BOT_SETTINGS['signal_confidence_threshold']:
                    # ساخت استراتژی‌های تأییدکننده
                    special_strategies = {
                        "استراتژی ویژه": special_result['current_strength']
                    }

                    # اضافه کردن استراتژی‌های همسو اگر موجود باشند
                    if special_result['current_signal'] == 'خرید':
                        if ema_result['latest_signal_type'] in ['خرید', 'صعودی']:
                            special_strategies["میانگین متحرک نمایی"] = ema_result['latest_signal_strength']
                        if rsi_result['current_signal_type'] in ['خرید', 'صعودی']:
                            special_strategies["شاخص قدرت نسبی"] = rsi_result['current_signal_strength']
                    elif special_result['current_signal'] == 'فروش':
                        if ema_result['latest_signal_type'] in ['فروش', 'نزولی']:
                            special_strategies["میانگین متحرک نمایی"] = ema_result['latest_signal_strength']
                        if rsi_result['current_signal_type'] in ['فروش', 'نزولی']:
                            special_strategies["شاخص قدرت نسبی"] = rsi_result['current_signal_strength']

                    # تعیین قیمت ورود، حد ضرر و حد سود
                    entry_price = df['close'].iloc[-1]

                    # استفاده از نقاط ورود اگر موجود باشند
                    stop_loss = entry_price * 0.95  # پیش‌فرض: 5% زیر قیمت ورود
                    take_profit = entry_price * 1.10  # پیش‌فرض: 10% بالای قیمت ورود

                    if 'entry_points' in special_result and special_result['entry_points'] and len(
                            special_result['entry_points']) > 0:
                        latest_entry = special_result['entry_points'][-1]
                        if latest_entry['type'] == 'long' and special_result['current_signal'] == 'خرید':
                            stop_loss = latest_entry['stop_loss']
                            take_profit = latest_entry['take_profit']
                        elif latest_entry['type'] == 'short' and special_result['current_signal'] == 'فروش':
                            stop_loss = latest_entry['stop_loss']
                            take_profit = latest_entry['take_profit']

                    # ایجاد توضیحات
                    description = f"سیگنال {special_result['current_signal']} براساس استراتژی ویژه\n\n"

                    if 'strategy_description' in special_result and special_result['strategy_description']:
                        description += f"• {special_result['strategy_description']}\n"

                    description += f"• سطح اطمینان: {special_result['current_strength']}%\n"
                    description += f"\nاستراتژی معاملاتی پیشنهادی:\n"
                    description += f"• ورود: {entry_price:.6f}\n"
                    description += f"• حد ضرر: {stop_loss:.6f}\n"
                    description += f"• حد سود: {take_profit:.6f}\n"

                    # ایجاد نمودار
                    special_chart_path = self.create_candlestick_chart(
                        df, symbol, timeframe,
                        strategies=special_strategies,
                        indicators={
                            'ema': True,
                            'rsi': True,
                            'macd': True,
                            'bollinger': True
                        },
                        patterns={
                            'market_structure_break_bullish': market_structure_result.get('msb_bullish', []),
                            'market_structure_break_bearish': market_structure_result.get('msb_bearish', []),
                            'order_block_bullish': market_structure_result.get('orderblock_bullish', []),
                            'order_block_bearish': market_structure_result.get('orderblock_bearish', [])
                        },
                        entry_exit={
                            'entries': [{'index': len(df) - 1, 'price': entry_price}],
                            'stop_loss': stop_loss,
                            'take_profit': take_profit
                        },
                        title=f"{symbol} - {timeframe} - استراتژی ویژه ({int(special_result['current_strength'])}% اطمینان)"
                    )

                    # ارسال سیگنال
                    self.send_signal(
                        symbol, timeframe, special_result['current_signal'], entry_price,
                        stop_loss, take_profit,
                        special_strategies, special_result['current_strength'],
                        description,
                        special_chart_path
                    )

                # ======= بررسی سیگنال‌های استراتژی ایچیموکو =======
                if 'current_signal' in ichimoku_result and ichimoku_result['current_signal'] in ['خرید', 'فروش'] and \
                        ichimoku_result['current_strength'] >= BOT_SETTINGS['signal_confidence_threshold']:
                    # ساخت استراتژی‌های تأییدکننده
                    ichimoku_strategies = {
                        "ایچیموکو": ichimoku_result['current_strength']
                    }

                    # اضافه کردن استراتژی‌های همسو
                    if ichimoku_result['current_signal'] == 'خرید':
                        if ema_result['latest_signal_type'] in ['خرید', 'صعودی']:
                            ichimoku_strategies["میانگین متحرک نمایی"] = ema_result['latest_signal_strength']
                        if rsi_result['current_signal_type'] in ['خرید', 'صعودی']:
                            ichimoku_strategies["شاخص قدرت نسبی"] = rsi_result['current_signal_strength']
                    elif ichimoku_result['current_signal'] == 'فروش':
                        if ema_result['latest_signal_type'] in ['فروش', 'نزولی']:
                            ichimoku_strategies["میانگین متحرک نمایی"] = ema_result['latest_signal_strength']
                        if rsi_result['current_signal_type'] in ['فروش', 'نزولی']:
                            ichimoku_strategies["شاخص قدرت نسبی"] = rsi_result['current_signal_strength']

                    # تعیین قیمت ورود، حد ضرر و حد سود
                    entry_price = df['close'].iloc[-1]

                    # استفاده از نقاط ورود ایچیموکو اگر موجود باشند
                    stop_loss = entry_price * 0.95  # پیش‌فرض: 5% زیر قیمت ورود
                    take_profit = entry_price * 1.10  # پیش‌فرض: 10% بالای قیمت ورود

                    if 'entry_points' in ichimoku_result and ichimoku_result['entry_points'] and len(
                            ichimoku_result['entry_points']) > 0:
                        latest_entry = ichimoku_result['entry_points'][-1]
                        if latest_entry['type'] == 'long' and ichimoku_result['current_signal'] == 'خرید':
                            entry_price = latest_entry['price']
                            stop_loss = latest_entry['stop_loss']
                            take_profit = latest_entry['take_profit']
                        elif latest_entry['type'] == 'short' and ichimoku_result['current_signal'] == 'فروش':
                            entry_price = latest_entry['price']
                            stop_loss = latest_entry['stop_loss']
                            take_profit = latest_entry['take_profit']

                    # ایجاد توضیحات
                    description = f"سیگنال {ichimoku_result['current_signal']} براساس استراتژی ایچیموکو\n\n"

                    if 'current_status' in ichimoku_result:
                        description += f"• {ichimoku_result['current_status']}\n"

                    description += f"• سطح اطمینان: {ichimoku_result['current_strength']}%\n"
                    description += f"\nاستراتژی معاملاتی پیشنهادی:\n"
                    description += f"• ورود: {entry_price:.6f}\n"
                    description += f"• حد ضرر: {stop_loss:.6f}\n"
                    description += f"• حد سود: {take_profit:.6f}\n"

                    # ایجاد نمودار
                    ichimoku_chart_path = self.create_candlestick_chart(
                        df, symbol, timeframe,
                        strategies=ichimoku_strategies,
                        indicators={
                            'ema': True,
                            'rsi': True,
                            'macd': True,
                            'ichimoku': True
                        },
                        patterns={
                            'market_structure_break_bullish': market_structure_result.get('msb_bullish', []),
                            'market_structure_break_bearish': market_structure_result.get('msb_bearish', []),
                            'order_block_bullish': market_structure_result.get('orderblock_bullish', []),
                            'order_block_bearish': market_structure_result.get('orderblock_bearish', [])
                        },
                        entry_exit={
                            'entries': [{'index': len(df) - 1, 'price': entry_price}],
                            'stop_loss': stop_loss,
                            'take_profit': take_profit
                        },
                        title=f"{symbol} - {timeframe} - استراتژی ایچیموکو ({int(ichimoku_result['current_strength'])}% اطمینان)"
                    )

                    # ارسال سیگنال
                    self.send_signal(
                        symbol, timeframe, ichimoku_result['current_signal'], entry_price,
                        stop_loss, take_profit,
                        ichimoku_strategies, ichimoku_result['current_strength'],
                        description,
                        ichimoku_chart_path
                    )

                # ======= بررسی سیگنال‌های استراتژی فیبوناچی =======
                if 'current_signal' in fibonacci_result and fibonacci_result['current_signal'] in ['خرید', 'فروش'] and \
                        fibonacci_result['current_strength'] >= BOT_SETTINGS['signal_confidence_threshold']:
                    # ساخت استراتژی‌های تأییدکننده
                    fib_strategies = {
                        "فیبوناچی": fibonacci_result['current_strength']
                    }

                    # اضافه کردن استراتژی‌های همسو
                    if fibonacci_result['current_signal'] == 'خرید':
                        if ema_result['latest_signal_type'] in ['خرید', 'صعودی']:
                            fib_strategies["میانگین متحرک نمایی"] = ema_result['latest_signal_strength']
                        if rsi_result['current_signal_type'] in ['خرید', 'صعودی']:
                            fib_strategies["شاخص قدرت نسبی"] = rsi_result['current_signal_strength']
                    elif fibonacci_result['current_signal'] == 'فروش':
                        if ema_result['latest_signal_type'] in ['فروش', 'نزولی']:
                            fib_strategies["میانگین متحرک نمایی"] = ema_result['latest_signal_strength']
                        if rsi_result['current_signal_type'] in ['فروش', 'نزولی']:
                            fib_strategies["شاخص قدرت نسبی"] = rsi_result['current_signal_strength']

                    # تعیین قیمت ورود، حد ضرر و حد سود
                    entry_price = df['close'].iloc[-1]

                    # استفاده از نقاط ورود فیبوناچی اگر موجود باشند
                    stop_loss = entry_price * 0.95  # پیش‌فرض: 5% زیر قیمت ورود
                    take_profit = entry_price * 1.10  # پیش‌فرض: 10% بالای قیمت ورود

                    if fibonacci_result['current_signal'] == 'خرید' and 'buy_signals' in fibonacci_result and \
                            fibonacci_result['buy_signals'] and len(fibonacci_result['buy_signals']) > 0:
                        latest_entry = fibonacci_result['buy_signals'][-1]
                        stop_loss = latest_entry.get('stop_loss', stop_loss)
                        take_profit = latest_entry.get('take_profit', take_profit)
                    elif fibonacci_result['current_signal'] == 'فروش' and 'sell_signals' in fibonacci_result and \
                            fibonacci_result['sell_signals'] and len(fibonacci_result['sell_signals']) > 0:
                        latest_entry = fibonacci_result['sell_signals'][-1]
                        stop_loss = latest_entry.get('stop_loss', stop_loss)
                        take_profit = latest_entry.get('take_profit', take_profit)

                    # ایجاد توضیحات
                    description = f"سیگنال {fibonacci_result['current_signal']} براساس استراتژی فیبوناچی\n\n"

                    # اطلاعات سطوح فیبوناچی
                    if fibonacci_result['current_signal'] == 'خرید' and fibonacci_result.get('nearest_support'):
                        fib_level, fib_value = fibonacci_result.get('nearest_support')
                        description += f"• واکنش به سطح فیبوناچی {fib_level} در قیمت {fib_value:.6f}\n"
                    elif fibonacci_result['current_signal'] == 'فروش' and fibonacci_result.get('nearest_resistance'):
                        fib_level, fib_value = fibonacci_result.get('nearest_resistance')
                        description += f"• واکنش به سطح فیبوناچی {fib_level} در قیمت {fib_value:.6f}\n"

                    description += f"• سطح اطمینان: {fibonacci_result['current_strength']}%\n"
                    description += f"\nاستراتژی معاملاتی پیشنهادی:\n"
                    description += f"• ورود: {entry_price:.6f}\n"
                    description += f"• حد ضرر: {stop_loss:.6f}\n"
                    description += f"• حد سود: {take_profit:.6f}\n"

                    # ایجاد نمودار
                    fib_chart_path = self.create_candlestick_chart(
                        df, symbol, timeframe,
                        strategies=fib_strategies,
                        indicators={
                            'ema': True,
                            'rsi': True,
                            'macd': True,
                            'fibonacci': True
                        },
                        patterns={
                            'market_structure_break_bullish': market_structure_result.get('msb_bullish', []),
                            'market_structure_break_bearish': market_structure_result.get('msb_bearish', []),
                            'order_block_bullish': market_structure_result.get('orderblock_bullish', []),
                            'order_block_bearish': market_structure_result.get('orderblock_bearish', [])
                        },
                        entry_exit={
                            'entries': [{'index': len(df) - 1, 'price': entry_price}],
                            'stop_loss': stop_loss,
                            'take_profit': take_profit
                        },
                        title=f"{symbol} - {timeframe} - استراتژی فیبوناچی ({int(fibonacci_result['current_strength'])}% اطمینان)"
                    )

                    # ارسال سیگنال
                    self.send_signal(
                        symbol, timeframe, fibonacci_result['current_signal'], entry_price,
                        stop_loss, take_profit,
                        fib_strategies, fibonacci_result['current_strength'],
                        description,
                        fib_chart_path
                    )

                # ======= ذخیره نتایج =======
                results[timeframe] = {
                    'market_structure': market_structure_result,
                    'pressure': pressure_result,
                    'whale': whale_result,
                    'ema': ema_result,
                    'rsi': rsi_result,
                    'divergence': divergence_result,
                    'bbrsi_macd': bbrsi_macd_result,
                    'darvas': darvas_result,
                    'ichimoku': ichimoku_result,
                    'fibonacci': fibonacci_result,
                    'golden': golden_result,
                    'special': special_result
                }

            except Exception as e:
                logger.error(f"❌ خطا در تحلیل {symbol} در تایم‌فریم {timeframe}: {str(e)}")
                logger.error(traceback.format_exc())

        return results

    def run(self):
        """اجرای ربات تحلیلگر با قابلیت مقاومت در برابر قطع اتصال و مدیریت پیشرفته حافظه"""
        logger.info("🚀 ربات تحلیلگر ارز دیجیتال شروع به کار کرد")

        # افزودن مکانیسم‌های بقا در محیط کولب
        self.add_colab_survival_mechanisms()

        # بررسی وضعیت اتصال و شمارنده‌ها
        self.diagnose_signals()

        # پاکسازی قدیمی کش دیسک در شروع برنامه
        cleanup_old_disk_cache()

        # مکانیسم بازیابی در صورت خطا
        max_errors = 5
        consecutive_errors = 0

        # شمارنده برای پایش مداوم منابع
        resource_check_counter = 0

        while True:
            try:
                # پایش منابع در هر چرخه
                resource_check_counter += 1
                if resource_check_counter >= 5:  # هر 5 چرخه یک بار پایش کامل منابع
                    system_status = monitor_resources(critical_threshold=90, warning_threshold=75)
                    logger.info(f"وضعیت سیستم: RAM {system_status.get('memory_percent', 0):.1f}%, "
                                f"CPU {system_status.get('cpu_percent', 0):.1f}%, "
                                f"کش {system_status.get('cache_items', 0)} آیتم")
                    resource_check_counter = 0
                else:
                    # پایش سریع در هر چرخه برای اطمینان از عدم کمبود حافظه
                    try:
                        import psutil
                        memory_percent = psutil.virtual_memory().percent
                        if memory_percent > 85:  # اگر حافظه بیش از 85% استفاده شده، پاکسازی انجام می‌دهیم
                            logger.warning(f"مصرف حافظه بالا: {memory_percent:.1f}% - پاکسازی")
                            clean_memory_cache(force_cleanup=True)
                    except:
                        pass

                # مکانیسم اجرای سیگنال‌دهی با توزیع بار
                symbols_batch_size = 5  # کاهش تعداد ارزهای قابل بررسی در هر بار برای کاهش مصرف حافظه

                # تقسیم لیست ارزها به گروه‌های کوچکتر
                for i in range(0, len(symbols), symbols_batch_size):
                    batch = symbols[i:i + symbols_batch_size]
                    logger.info(
                        f"📊 بررسی گروه ارزها: {i // symbols_batch_size + 1}/{math.ceil(len(symbols) / symbols_batch_size)}")

                    for symbol in batch:
                        self.run_analysis(symbol, timeframes)

                        # پاکسازی صریح حافظه بعد از هر ارز
                        gc.collect()

                        # مکث کوتاه بین هر ارز برای جلوگیری از استفاده زیاد منابع
                        time.sleep(2)

                    # پایش منابع بعد از هر گروه
                    try:
                        import psutil
                        memory_percent = psutil.virtual_memory().percent
                        if memory_percent > 80:  # اگر حافظه بیش از 80% استفاده شده، پاکسازی انجام می‌دهیم
                            logger.warning(f"مصرف حافظه بالا بعد از گروه: {memory_percent:.1f}% - پاکسازی")
                            clean_memory_cache(force_cleanup=True)
                    except:
                        # پاکسازی محتاطانه در صورت خطا
                        clean_memory_cache(force_cleanup=True)

                    # ذخیره وضعیت پس از هر گروه
                    self.save_state()

                    # پاکسازی کش حافظه بعد از هر گروه
                    clean_memory_cache()

                    # مکث بین گروه‌ها برای توزیع بار
                    time.sleep(15)

                # تحلیل عملکرد صرافی‌ها
                analyze_exchange_performance()

                # ذخیره وضعیت
                self.save_state()

                # پاکسازی کش قدیمی
                cleanup_old_disk_cache()

                # ریست شمارنده خطاها
                consecutive_errors = 0

                # مکانیسم خواب هوشمند: در ساعات کم معامله، زمان طولانی‌تری می‌خوابیم
                current_hour = datetime.datetime.now().hour
                if 2 <= current_hour <= 5:  # ساعات کم فعالیت بازار (2 تا 5 صبح)
                    logger.info("💤 ساعات کم فعالیت بازار - اجرای با تاخیر بیشتر...")
                    sleep_time = 600  # 10 دقیقه در ساعات کم فعالیت
                else:
                    sleep_time = 180  # 3 دقیقه در ساعات معمول

                # آزادسازی صریح حافظه در پایان هر چرخه
                gc.collect()

                # انتظار قبل از بررسی مجدد با نمایش شمارنده معکوس
                for remaining in range(sleep_time, 0, -15):
                    print(f"\r⏳ بررسی بعدی در {remaining} ثانیه دیگر...     ", end="")

                    # پایش منابع حتی در حالت انتظار
                    if remaining % 60 == 0:  # هر 60 ثانیه
                        try:
                            import psutil
                            memory_percent = psutil.virtual_memory().percent
                            if memory_percent > 90:  # اگر حافظه بیش از 90% استفاده شده، پاکسازی اضطراری انجام می‌دهیم
                                logger.warning(
                                    f"مصرف حافظه بحرانی در حالت انتظار: {memory_percent:.1f}% - پاکسازی اضطراری")
                                memory_cache.clear()
                                memory_cache_access_time.clear()
                                gc.collect()
                        except:
                            pass

                    time.sleep(15)
                print("\r                                              ", end="")

            except KeyboardInterrupt:
                # ذخیره نهایی وضعیت
                self.save_state()
                logger.info("👋 ربات متوقف شد - وضعیت ذخیره شد")
                break

            except Exception as e:
                # افزایش شمارنده خطاهای پی در پی
                consecutive_errors += 1

                logger.error(f"❌ خطای کلی ({consecutive_errors}/{max_errors}): {str(e)}")
                logger.error(traceback.format_exc())

                # ذخیره وضعیت حتی در صورت خطا
                self.save_state()

                # آزادسازی صریح حافظه در صورت خطا و پاکسازی اضطراری
                memory_cache.clear()
                memory_cache_access_time.clear()
                gc.collect()

                # اگر تعداد خطاهای پی در پی از حد مجاز بیشتر شد، کمی بیشتر صبر می‌کنیم
                if consecutive_errors >= max_errors:
                    logger.warning(f"⚠️ تعداد خطاهای پی در پی به حد مجاز رسید. انتظار برای بازیابی...")
                    time.sleep(300)  # 5 دقیقه انتظار برای بازیابی
                    consecutive_errors = 0
                else:
                    time.sleep(60)  # 1 دقیقه انتظار قبل از تلاش مجدد

    # اجرای اصلی برنامه
if __name__ == "__main__":
    # تنظیم توکن ربات تلگرام و آیدی کانال
    TELEGRAM_BOT_TOKEN = "7307185304:AAF9zG3vM0Pc74WkiIDVEnRxu41OJYdyql8"
    TELEGRAM_CHANNEL_ID = "@hosseinbtb2"  # مثال: "@your_channel_name"

    # ایجاد مسیرهای لازم
    for path in [f"{BASE_DIR}/cache/ohlcv", f"{BASE_DIR}/charts", f"{BASE_DIR}/logs"]:
        os.makedirs(path, exist_ok=True)

    # ایجاد نمونه تحلیلگر
    analyzer = CryptoAnalyzer(TELEGRAM_BOT_TOKEN, TELEGRAM_CHANNEL_ID)

    # اجرای ربات
    analyzer.run()

🌟 محیط گوگل کولب شناسایی شد - فعال‌سازی ذخیره‌سازی پایدار...
📡 ربات فعال - آخرین ذخیره: 06:59:25 - سیگنال‌ها: 680Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<IPython.core.display.Javascript object>

2025-05-07 07:04:08,684 - ERROR - ❌ اتصال به ربات تلگرام برقرار نیست
2025-05-07 07:04:08,684 - ERROR - ❌ اتصال به ربات تلگرام برقرار نیست
2025-05-07 07:04:08,684 - ERROR - ❌ اتصال به ربات تلگرام برقرار نیست
2025-05-07 07:04:08,684 - ERROR - ❌ اتصال به ربات تلگرام برقرار نیست
2025-05-07 07:04:08,684 - ERROR - ❌ اتصال به ربات تلگرام برقرار نیست
2025-05-07 07:04:08,684 - ERROR - ❌ اتصال به ربات تلگرام برقرار نیست
ERROR:__main__:❌ اتصال به ربات تلگرام برقرار نیست


📡 ربات فعال - آخرین ذخیره: 07:52:21 - سیگنال‌ها: 685

2025-05-07 07:52:47,692 - WARNING - داده‌های ناکافی از binance برای OMG/USDT/15m
2025-05-07 07:52:47,692 - WARNING - داده‌های ناکافی از binance برای OMG/USDT/15m
2025-05-07 07:52:47,692 - WARNING - داده‌های ناکافی از binance برای OMG/USDT/15m
2025-05-07 07:52:47,692 - WARNING - داده‌های ناکافی از binance برای OMG/USDT/15m
2025-05-07 07:52:47,692 - WARNING - داده‌های ناکافی از binance برای OMG/USDT/15m
2025-05-07 07:52:47,692 - WARNING - داده‌های ناکافی از binance برای OMG/USDT/15m


...

📡 ربات فعال - آخرین ذخیره: 07:52:21 - سیگنال‌ها: 685

2025-05-07 07:56:37,671 - WARNING - داده‌های ناکافی از binance برای SRM/USDT/1d
2025-05-07 07:56:37,671 - WARNING - داده‌های ناکافی از binance برای SRM/USDT/1d
2025-05-07 07:56:37,671 - WARNING - داده‌های ناکافی از binance برای SRM/USDT/1d
2025-05-07 07:56:37,671 - WARNING - داده‌های ناکافی از binance برای SRM/USDT/1d
2025-05-07 07:56:37,671 - WARNING - داده‌های ناکافی از binance برای SRM/USDT/1d
2025-05-07 07:56:37,671 - WARNING - داده‌های ناکافی از binance برای SRM/USDT/1d


📡 ربات فعال - آخرین ذخیره: 08:06:20 - سیگنال‌ها: 692
=== گزارش عملکرد صرافی‌ها ===
kucoin: نرخ موفقیت 99.4%, تعداد کل درخواست: 538, دفعات محدودیت: 0
gate: نرخ موفقیت 98.5%, تعداد کل درخواست: 591, دفعات محدودیت: 0
mexc: نرخ موفقیت 99.3%, تعداد کل درخواست: 1085, دفعات محدودیت: 0
📡 ربات فعال - آخرین ذخیره: 08:44:31 - سیگنال‌ها: 694

2025-05-07 08:47:50,453 - WARNING - مصرف حافظه بالا بعد از گروه: 82.0% - پاکسازی
2025-05-07 08:47:50,453 - WARNING - مصرف حافظه بالا بعد از گروه: 82.0% - پاکسازی
2025-05-07 08:47:50,453 - WARNING - مصرف حافظه بالا بعد از گروه: 82.0% - پاکسازی
2025-05-07 08:47:50,453 - WARNING - مصرف حافظه بالا بعد از گروه: 82.0% - پاکسازی
2025-05-07 08:47:50,453 - WARNING - مصرف حافظه بالا بعد از گروه: 82.0% - پاکسازی
2025-05-07 08:47:50,453 - WARNING - مصرف حافظه بالا بعد از گروه: 82.0% - پاکسازی


📡 ربات فعال - آخرین ذخیره: 08:44:31 - سیگنال‌ها: 694